In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
# import datetime 
# from datetime import timedelta ,time 
from datetime import timedelta ,time ,datetime

from collections import Counter
from numpy import nan as NA
from dateutil.parser import parse
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Data_Clean, Risk_Data
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
clean = Data_Clean()
risk = Risk_Data()

In [2]:
def query(sql,
        #   host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="huangxq",
          password="f_epz6hDmZ0Hzi0OzI0N",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 1.样本数据整理

## 1_11.获取_租后数据

In [3]:

# # 含非自营
# sql = '''-- 租后应收监控  
# SELECT  tprm.* 
# ,om.`status` as 订单状态值
# ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
# when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
# when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
# ,om.order_finish_date as 订单完成时间,om.order_method
# ,tmu.true_name ,tmu.id_card_num,tmu.mobile 
# ,tod.product_name
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# left join db_digua_business.t_order om on tprm.order_id = om.id
# left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
# left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
# ;
# '''

# df_zhys = query(sql)
# # (27787, 62)
# df_zhys.shape

# note 2025.4.30
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile,tmu.province_name
,tod.product_name
,tod.sku_attributes

-- 2025.4.29
,cc.name as channel_name_cc         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,pa.type
,om.order_type
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result

from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id

-- 2025.4.29
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape

(82698, 73)

## 1_12. 剔除商家数据

In [4]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="人人享租"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="兴鑫兴通讯"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="崇胜数码"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵新租机"].index,inplace=True)  #拒量订单
   

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(78168, 73)

## 1_13.历史订单渠道名称补充

In [5]:
# # 1148 条历史订单渠道名称补充
# qudao_name_df = pd.read_excel('E:/myfile/租后数据/历史订单渠道名称补充.xlsx')
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# # dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
# dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# # dfzh1.columns

# # qudao_name_df.shape
# # qudao_name_df[:2]
# # qudao_name_df['渠道名称'].value_counts()

# # 渠道名称缺失补充
# # dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
# dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
# dfzh1["channel_name"].isnull().sum()

# # dfzh1[['order_number','channel_name','渠道名称']][:10]
# # df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]


# note 2025.4.30
# 1148 条历史订单渠道名称补充
qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

# qudao_name_df.shape
# qudao_name_df[:2]
# qudao_name_df['渠道名称'].value_counts()

# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(), dfzh1.渠道名称, dfzh1["channel_name"])
# dfzh1["channel_name"].isnull().sum()

# dfzh1[['order_number','channel_name','渠道名称']][:10]
# df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]

# 更新渠道，同时给客群打标签
# 2025.4.29
# 是否巨量、是否号卡
# df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1["channel_name_cc"] = np.where(dfzh1.渠道名称.notnull(), dfzh1.渠道名称, dfzh1["channel_name_cc"])
dfzh1.loc[:,"来源渠道"]=dfzh1["channel_name"].fillna("未知渠道")
dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
dfzh1['归属渠道'] = np.where(dfzh1.归属渠道 == '顶部搜索框的搜索结果页', '搜索渠道', dfzh1.归属渠道)
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
conditions_1 = ['八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']
# 租机宝 、 富士康租机 、 兔丫租赁 、 邦客惠乘车码  -- 不算S量
dfzh1['归属渠道'] = np.where(dfzh1.归属渠道.isin(conditions_1), 'S量', dfzh1.归属渠道)


# dfzh1[dfzh1.归属渠道 == '顶部搜索框的搜索结果页']，channel_name 为 顶部搜索框的搜索结果页，channel_name_cc 为 空，下单日期为 2023-03 及之前
# A20250113184842199 channel_name为 顶部搜索框的搜索结果页， channel_name_cc 为顶部搜索框的搜索结果页，归属渠道 为抖音

dfzh1['是否免人审'] = np.where((dfzh1.is_vip=='1')&(dfzh1.status_result=='0'), 1 , 0)

# df['是否拒量'] = np.where(df.qvt_risk == '1', 1 , 0)
# df['是否拒量'] = np.where((df.qvt_risk == '0') & (df.qvt_result == '1'), 1 , df.是否拒量)
dfzh1['是否拒量'] = np.where(dfzh1.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
dfzh1['是否号卡'] = np.where(dfzh1.type == 4, 1, 0)

# df_zhys.columns
# df_zhys[['channel_name','channel_name_cc']]
# df_zhys.channel_name.value_counts()
# df_zhys.channel_name_cc.value_counts()
# 芝麻信用 芝麻租物 

## 1_14.渠道归属

In [6]:
# def qudao_type(a,b,c):
#     a = str(a)
#     b = str(b)
#     if "租物" in b:
#         return "芝麻租物"
#     elif "芝麻" in b:
#         return "芝麻租物"
#     elif "抖音" in b:
#         return "抖音渠道"
#     elif "搜索" in a:
#         return "搜索渠道"
#     elif "租物" in a:
#         return "芝麻租物"
#     elif "芝麻" in a:
#         return "芝麻租物"
#     elif c == 1:
#         return "芝麻租物" 
#     elif "叮咚直播" in a:
#         return "叮咚直播"
#     elif "八派信息" in a:
#         return "八派信息"
#     elif "租瓜直播2号" in a:
#         return "租瓜直播2号"
#     elif "租瓜直播" in a:
#         return "租瓜直播"
#     elif "直播" in a:
#         return "小舞直播"
#     elif "繁星" in a:
#         return "繁星"
#     elif "生活号" in a:
#         return "生活号"
#     elif "社群" in a:
#         return "支付宝社群"
#     elif "付费灯火" in a:
#         return "付费灯火"
#     else :
#         return a

In [7]:
# # df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
# dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
# dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

# dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x:qudao_type(x["channel_name"],x["activity_source"],x["order_method"]),axis=1)

# # pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

## 1_15.判断颜色 内存

In [8]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
# dfzh1["颜色"].value_counts()[1:10]

## 1_16.判断二手、带锁

In [9]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
# note 20241124 修改口径，用product_name判断是否二手
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    76592
二手     1576
Name: count, dtype: int64

In [10]:
#读取导入最新出库台账数据,2023.9及以后的数据
# （1）512 带锁
# （2）非512的多头带锁
# （3）抖音带锁
# note 20241204
# （1）512 带锁，但每天豁免一台 512P 不带锁，目前还没量
# （2）非512的命中多头提示带锁，但出库是否带锁由人工控制
# （3）抖音带锁（苹果11-15）
# 维客壹佰2023&2024年台账1204.xlsz 为截止到20241203的数据
# f_path_ck = "维客壹佰2023&2024年台账1204.xlsx"
f_path_ck = "维客壹佰2023&2024年台账.xlsx"
path = 'F:\myfile\p站数据\台账数据/'
dfck = pd.read_excel(path + f_path_ck,sheet_name="2023")   #,skiprows=0 header 

# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存']=='512G')|(dfck['内存']=='512GB'), 1, 0)
dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存'].str.contains(r'512')), '带锁', '未带锁')
dfck = dfck[[u'订单号',u'是否带锁']]
dfck.shape
dfck['是否带锁'].value_counts()

dfck.drop_duplicates(subset=["订单号"],inplace=True)

# 合并到主表中去
# dfck 的带锁不一定出库，dfzh1 是出库的
dfck = dfck.rename(columns= {'订单号': 'order_number'})
dfzh1 = dfzh1.merge(dfck, on = 'order_number', how = 'left')
dfzh1['是否带锁'] = dfzh1['是否带锁'].fillna('未带锁') 
dfzh1.shape

pd.pivot_table(dfzh1,index='是否二手',columns='是否带锁',values='order_id',aggfunc='count')

(49985, 2)

是否带锁
未带锁    46253
带锁      3732
Name: count, dtype: int64

(78168, 84)

是否带锁,带锁,未带锁
是否二手,,
二手,537,1039
全新,2836,73756


## 1_17.日期处理

In [11]:
dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

0   2022-07-05
1   2022-07-13
2   2022-07-15
3   2022-07-16
4   2022-07-17
Name: 下单日期, dtype: datetime64[ns]

In [12]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
3季度    25248
4季度    23209
2季度    20233
1季度     9478
Name: count, dtype: int64

## 1_21.获取_买断数据

In [13]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
# note 20241126 增加distinct，买断可能分多次付款
# 181950     2
# 338547     2
# 1083822    2
# 303756     2
SELECT distinct t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(11401, 7)

## 1_22.租后数据 关联 买断数据

In [14]:
# 拼接买断实付金额和是否买断
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.shape
dfzh1.shape

(78168, 92)

(78168, 88)

In [15]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

np.int64(13397)

In [16]:
dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()

if_outpay
未知      67305
已买断     10581
部分买断      282
Name: count, dtype: int64

## 1_31.获取_分期取消数据

In [17]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
# 订单取消
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(5222, 7)

## 1_32.租后数据关联分期取消数据

In [18]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.shape
dfzh2.shape

# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

(78168, 96)

(78168, 93)

## 1_41.按风控策略调整时间节点划分时间段

In [19]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

## 1_51.获取_账期数据（含续租）

In [20]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(1029722, 24)

In [21]:
# note 20241227 号卡 应付日期更新， 号卡每期应还租金日期
# 分期表
sql = '''-- 租后应收监控  
SELECT distinct hk.order_number, tos.order_id, tos.refund_date as refund_date_1
from db_digua_business.t_order_stages tos
inner join
    (select distinct om.id as order_id, om.order_number
    from db_digua_business.t_order om
    left join db_digua_business.t_platform_activity tpa 
    on tpa.id=om.activity_id
    where tpa.type = 4) hk
on tos.order_id = hk.order_id
'''
tmp_hk = query(sql)
tmp_hk['refund_date_ym'] = tmp_hk['refund_date_1'].dt.strftime('%Y-%m')
tmp_hk.shape
tmp_hk[:2]

df_xzfq['refund_date_ym'] = df_xzfq['refund_date'].dt.strftime('%Y-%m')
df_xzfq = df_xzfq.merge(tmp_hk[['order_id', 'refund_date_1', 'refund_date_ym']], on = ['order_id', 'refund_date_ym'], how = 'left')
df_xzfq['refund_date_2'] = np.where(df_xzfq.refund_date_1.notnull(), df_xzfq.refund_date_1, df_xzfq.refund_date)
df_xzfq.shape

df_xzfq.drop(columns=['refund_date_1','refund_date_ym','refund_date'],inplace = True)
df_xzfq = df_xzfq.rename(columns = {'refund_date_2':'refund_date'})
df_xzfq.shape

(5076, 4)

,order_number,order_id,refund_date_1,refund_date_ym
0,A20240927154904302,1533488,2024-09-30,2024-09
1,A20240927154904302,1533488,2024-10-30,2024-10


(1029722, 27)

(1029722, 24)

In [22]:
# 数据检查
# tmp1 = df_xzfq.drop_duplicates(subset = ['order_id'])[['order_id']]
# tmp2 = tmp_hk.drop_duplicates(subset = ['order_id'])[['order_id', 'order_number']]
# tmp1.shape
# tmp2.shape

# tmp3 = tmp1.merge(tmp2, on = 'order_id', how = 'inner')
# tmp3.shape

# tmp4 = tmp3.merge(df_xzfq, on = 'order_id', how = 'inner')
# tmp4.shape

# tmp4["日"]=tmp4["refund_date"].dt.day
# tmp4["日"].value_counts()
# tmp4[tmp4.日.isin([2,11])][['order_number','refund_date']]

# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 == df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]
# 以下四个订单的应付日期<15号，订单分期表更正了  正常的订单的应付日期
# A20240929094315174 1539249  已退款
# A2024092919033313  1541755  正常
# A2024092919182056  1541798  已退款
# A2024100810242488  1568606  已退款

# tmp_hk[tmp_hk.order_number.isin(['A2024100810242488'])]
# df_xzfq[df_xzfq.order_id.isin([1539249,1541755,1541798,1568606])][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]

In [23]:
# df_xz_concat[df_xz_concat.order_number.isin(['A2024080919264964','A2024092919033313','A20241218184711224'])][['order_number','应付日期','实付日期','逾期天数']]
# A2024080919264964
# A2024092919033313
# tmp = df_xz_concat[df_xz_concat.逾期天数 > 30].drop_duplicates(subset=['order_number'])[['order_number']]
# tmp = tmp.merge(df_hk, on = 'order_number', how = 'inner')
# tmp.shape
# tmp

## 1_52.重命名

In [24]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期','reality_refund_date':'实付日期','sort':'当前期数'})

## 1_53.实还金额核查

In [25]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

## 1_61.数据合并

In [26]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')
# note 20241126 增加
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape
# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228
df_xz_concat.实付金额 = df_xz_concat.实付金额 + df_xz_concat.sesame_promo_money_pay + df_xz_concat.promo_money
df_xz_concat = df_xz_concat[df_xz_concat.当前应付租金.notnull()]

# order_id == 1681669 的当前期数为空
# 1681669	A20241104144812228
# 1669581	A2024110109350797
df_xz_concat['当前期数'].notnull().sum()
df_xz_concat.shape
df_xzfq.shape
dfzh3.shape

# df_xz_concat.drop_duplicates(subset=["order_id"]).shape
# dfzh3.drop_duplicates(subset=["order_id"]).shape

# df_xz_concat['order_id'].nunique()
# dfzh3['order_id'].nunique()
# df_xz_concat[["order_id","当前期数"]].info()

np.int64(1029523)

(1029523, 119)

(1029722, 24)

(78168, 96)

## 1_62.去重

In [27]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

        order_id  当前期数
41780     431052   1.0
41792     431052   1.0
702384    431052   1.0
702396    431052   1.0
41781     431052   2.0
...          ...   ...
528630   2040621  23.0
475854   2040621  24.0
475878   2040621  24.0
528607   2040621  24.0
528631   2040621  24.0

[1152 rows x 2 columns]


In [28]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)
df_xz_concat.shape

(1028659, 119)

In [29]:
# 119321
# 1083822 303756
# all_duplicates[all_duplicates['order_id'] == 181950].shape
# df_xz_concat[df_xz_concat['order_id'] == 181950].shape

# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

# 3.计算逾期天数

## 3_11.异常查询

In [30]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# 实付金额>0 实付日期为空 finish_date_new非空 :实付日期=订单完成时间
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# part_payment==0 reality_refund_date.notnull() 还款状态== '续租中' part_payment=money 还款状态== '已逾期' reality_refund_date=pd.na
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# 实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消
# dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 部分还款，实付日期非空，完成时间为空 用户： 实付日期清空
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1

# 部分还款，实付日期为空，完成时间非空 用户： 实付日期=完成时间
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1.订单状态值.value_counts()

# 已买断 但无实付日期及金额：实付日回写
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# # dd_test1.订单状态值.value_counts()
# dd_test1

## 3_12.实还日期修正

In [31]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [32]:
# （1）部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)

df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()     

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
916576,A2025101621115461,4235679,544.83,544.83,2025-11-19,2025-11-12,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
952246,A20251105222831241,4467098,544.83,544.83,2025-12-08,2025-12-07,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
973064,A20251115144435641,4599447,544.83,544.83,2025-12-18,2025-11-28,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
974267,A20251116112904380,4608638,399.74,399.74,2025-11-19,2025-11-16,3.0,1.0,未到首期还款日,NaT,4,NaT,NaN,NaT
990846,A20251124121722215,4699093,544.83,544.83,2025-12-27,2025-11-26,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT


In [33]:
# （2）已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()       

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
990462,A2025112408240156,4697033,749.79,1.72,2025-12-27,NaT,1.0,2.0,未到首期还款日,NaT,4,NaT,NaN,NaT
990570,A20251124102302240,4697692,489.85,20.00,2025-12-27,NaT,1.0,2.0,未到首期还款日,NaT,4,NaT,NaN,NaT
990846,A20251124121722215,4699093,544.83,544.83,2025-12-27,2025-11-26,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
990869,A20251124123037388,4699265,489.85,489.85,2025-11-27,NaT,1.0,1.0,未到首期还款日,NaT,4,NaT,NaN,NaT
1014413,A2025120509122225,4789531,1379.49,1149.56,2025-12-08,NaT,1.0,1.0,未到首期还款日,NaT,4,NaT,NaN,NaT


In [34]:
# （3）租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60976,2024-07-22


In [35]:
# （4）未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
9558,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9636,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
10650,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
12255,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17,1.0,11.0,已买断,NaT,8,NaT,NaN,NaT
643737,A20250603161957279,2803528,0.00,0.0,2025-06-06,2025-06-03,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT


In [36]:
# （5）实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] .tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
910968,A20251013182307257,4199007,1085.36,0.0,2026-09-16,NaT,1.0,12.0,已买断,2025-11-07 10:26:04,8,2025-11-07,11.0,2025-11-07
943900,A20251102105022458,4419098,1085.36,0.0,2026-10-05,NaT,1.0,12.0,已买断,2025-11-18 14:14:05,8,2025-11-18,11.0,2025-11-18
964390,A20251111114016716,4543950,1085.36,0.0,2026-10-14,NaT,1.0,12.0,已买断,2025-12-07 17:56:51,8,2025-12-07,11.0,2025-12-07
997628,A20251127115001562,4725785,1149.47,0.0,2026-09-30,NaT,1.0,11.0,已买断,2025-12-09 09:41:42,8,2025-12-09,10.0,2025-12-09
999436,A20251128104407407,4733102,1152.43,0.0,2026-11-01,NaT,1.0,12.0,已买断,2025-12-10 14:17:38,8,2025-12-10,11.0,2025-12-10


In [37]:
# （6）部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [38]:
# （7）未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [39]:
# （8）实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [40]:
# （9）属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [41]:
# （10）df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [42]:
# （11）A202207050851373、A20241104144812228 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number.isin(['A202207050851373','A20241104144812228'])),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [43]:
# （12）实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)

In [44]:
# （13）df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 3_13.获取逾期截止时间

In [45]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date'].head()

0   2025-12-12
1   2025-12-12
2   2025-12-12
3   2025-12-12
4   2025-12-12
Name: now_date, dtype: datetime64[s]

## 3_14.重新定义状态

In [46]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

rembursement_status
还款中                    166538
已逾期                    123266
未到首期还款日                 52396
续租中                     22767
检测中                       101
Name: count, dtype: int64

订单状态
租赁中     353249
已退款      12240
待归还       5904
待收货       3864
待发货       1840
已完成        225
订单取消        84
检测中         71
Name: count, dtype: int64

In [47]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     509762
未到还款日    344365
已逾期       81802
已逾期支付     80265
已取消       12465
Name: count, dtype: int64

In [48]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 3_15.重新定义逾期天数

In [49]:
df_xz_concat['实付日期new'].isnull().sum()
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

np.int64(377533)

Series([], Name: order_number, dtype: object)

In [50]:
# 异常值删除，应付时间为空
df_xz_concat.shape
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(1028659, 123)

(1028659, 123)

In [51]:
#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 
#note 20241127 逾期天数 截止目前的逾期天数 ；逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][['order_number','逾期天数','应付日期','']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092919033313'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

# df_xz_concat['逾期天数'].value_counts()
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        113892
还款中         19610
已买断         19303
续租中          7013
已完成          1902
检测中           182
未到首期还款日       151
Name: count, dtype: int64

In [52]:
# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

## 3_16.逾期天数处理

In [53]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

## 3_17.稽查

In [54]:
# df_ddfq.columns
# df_xz_concat.shape
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape
# check_fq_df.columns
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')


## 3_18.确认是否到表现期

In [55]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [56]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [57]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 

In [58]:
# from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','agr_15d','agr_7d','agr_4d','agr_1d','应付日期']].head()

,agr_30d,agr_15d,agr_7d,agr_4d,agr_1d,应付日期
0,2022-08-04,2022-07-20,2022-07-12,2022-07-09,2022-07-06,2022-07-05
1,2022-09-04,2022-08-20,2022-08-12,2022-08-09,2022-08-06,2022-08-05
2,2022-10-05,2022-09-20,2022-09-12,2022-09-09,2022-09-06,2022-09-05
3,2022-11-04,2022-10-20,2022-10-12,2022-10-09,2022-10-06,2022-10-05
4,2022-12-05,2022-11-20,2022-11-12,2022-11-09,2022-11-06,2022-11-05


In [59]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(5)
# df_xz_concat[(df_xz_concat.agr_7d_cum == 0) & (df_xz_concat.agr_4d_cum == 1)].head()
# df_xz_concat[df_xz_concat.order_id == 254588]

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2025-12-12,1
1,2022-09-04,2022-08-05,2025-12-12,1
2,2022-10-05,2022-09-05,2025-12-12,1
3,2022-11-04,2022-10-05,2025-12-12,1
4,2022-12-05,2022-11-05,2025-12-12,1


In [60]:
# note 20241211 数据检查
# df_xz_concat[(df_xz_concat.应付日期>='2024-12-01')&(df_xz_concat.应付日期<='2024-12-31')&(df_xz_concat.order_number.isin(
# ['A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# ]))][['order_number','agr_1d_cum','agr_4d_cum','agr_7d_cum','agr_4d','应付日期','date_str','agr_1d','下单日期']]

## 3_19.观察日定义 每月月底时间

In [61]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):
    # 2025.2.8 
    # for n in range(0,16):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 月末的的跳级了，所以不要加1，mob0对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat.shape
df_xz_concat = mob_date(df_xz_concat)
df_xz_concat.shape

# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

# A202301011402371 2023-01-01
# A202301290815565 2023-01-29
# A2023013100341134 2023-01-31
# A2023022801142011 2023-02-28
# A2023022709283120  2023-02-27
# A202302050022319  2023-02-05
# A2023040503314720 2023-04-05
# A2023042803470516  2023-04-28
# A2023043000524068 2023-04-30
# A2023063003392242 2023-06-30
# A202306280105556 2023-06-28
# A202306050324407  2023-06-05
# A202310050100222 2023-10-05
# A2023102901224646 2023-10-29
# A2023103101255825 2023-10-31

(1028659, 135)

(1028659, 174)

In [62]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

In [63]:
# var = ['order_number','当前期数','mob_date_12','paid_date_mob12','paid_money_mob12','mob_date_13','paid_date_mob13','paid_money_mob13','mob_date_14','paid_date_mob14','paid_money_mob14','mob_date_15','paid_date_mob15','paid_money_mob15']
# df_xz_concat[df_xz_concat.order_number.isin(['A2024060200290566','A2024063001194047','A202308262317036','A2024031113200319','A202207050851373'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202207050851373'])][var]

## 3_20.更新逾期天数

In [64]:
# （1）逾期天数校正
# note 20241213
# 首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0（各期更新为0）

# 把符合条件的订单拿出来，逾期天数全部更新为0
sql_xz = ''' 
SELECT distinct tprm.order_number,1 as label
# ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
where ymos.sort=1 and ymos.reality_refund_date is null
;
'''
tmp = query(sql_xz)
tmp.shape
# tmp[:2]

df_xz_concat = df_xz_concat.merge(tmp, on = 'order_number', how = 'left')
df_xz_concat['label'] = df_xz_concat['label'].fillna(0)
df_xz_concat['label'].value_counts()

df_xz_concat[df_xz_concat['label'] == 1].drop_duplicates(subset = 'order_number').shape

df_xz_concat['逾期天数'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat['逾期天数_new'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].shape
# df_xz_concat[(df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']) & (df_xz_concat.逾期天数 > 30)].shape
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']]
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.max()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.min()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']].to_excel('tmp.xlsx', index = False)

# A202207181530165
# A2024092919033313 号卡
# A2023011718580545 vintage分子是月末状态，FSTQPD是发生过逾期
# df_xz_concat[df_xz_concat.order_number == 'A2023011718580545'][['order_number','应付日期','实付日期','实付金额','逾期天数']]
# df_mob_total[df_mob_total.order_number == 'A2023011718580545'][['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','MOB','overdue_days_1','overdue_days_4','overdue_days_7','overdue_days_15','overdue_days_31']]

# （2）2025.2.21
df_xz_concat['逾期天数'] = np.where(df_xz_concat.当前期数 == 1, 0, df_xz_concat.逾期天数)

(2391, 2)

label
0.0    1000543
1.0      28116
Name: count, dtype: int64

(2321, 175)

## 3_21.更新租期到期的逾期天数

In [65]:
# # （1）更新归还逾期的订单的逾期天数
# # 本次未统计在内，应该用 df_xz_concat

# # （1-1）未归还逾期或未到期或系统当天到期
# df_xz_concat.shape
# conditions_1 = (df_xz_concat.is_relet == 0)
# tmp1 = df_xz_concat[(conditions_1 & (df_xz_concat.end_date >= df_xz_concat.finish_date_new)) | (conditions_1 & (df_xz_concat.end_date >= df_xz_concat.date_str))]
# tmp1.shape

# # （1-2）续租
# tmp2 = df_xz_concat[df_xz_concat.is_relet > 0 ]
# tmp2.shape
# # （1-3）出现过归还逾期（包含租期一直逾期直到归还逾期的）--仅首续有归还逾期，被减数为最近未支付的应付日期
# conditions_2 = (df_xz_concat.end_date < df_xz_concat.finish_date_new)
# conditions_3 = (df_xz_concat.finish_date_new.isnull())
# conditions_4 = (df_xz_concat.end_date < df_xz_concat.date_str)
# tmp3 = df_xz_concat[(conditions_1 & conditions_2) | (conditions_1 & conditions_3 & conditions_4)]
# tmp3.shape

# # 367644 + 55490

# # 计算到期后逾期天数--订单已经完成的： 逾期天数为 finish_date_new- end_date；订单未完成的，逾期天数为 date_str- 实付日期new为空的最小的 应付日期
# # ((df_xz_concat['finish_date_new'] - df_xz_concat['end_date']).dt.days > 0) 
# # A2024020317504938--订单已完成
# # A202207171820512 --订单已完成
# # A202207181530165--订单未完成
# # var = ['order_number','当前期数','应付日期','实付日期new','end_date','date_str','逾期天数','pay_date','order_finish_date','订单完成时间','finish_date_new']
# # df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][var]

In [66]:
# （2）数据检查
# df_fq_concat[df_fq_concat.下单日期 == '2024-06-30'][['order_number']][:2]
# A2024060200290566  A2024063001194047 A202308262317036（续租） A2024031113200319（提早归还） A202207050851373（续租提前买断）
# var = ['order_number', '当前期数', '下单日期', '应付日期', '首期应还日期', '当前应付租金','rembursement_status','订单状态','if_outpay','状态','是否提前还款','实付日期new','逾期天数','order_finish_date','finish_date_new']
# df_fq_concat[df_fq_concat.order_number.isin(['A2024060200290566','A2024063001194047','A202308262317036','A2024031113200319','A202207050851373'])][var]

# var = ['order_number', '当前期数', '下单日期', '应付日期', '当前应付租金','rembursement_status','订单状态','if_outpay','状态','实付日期new','finish_date_new','status_x','paid_periods','实付金额','date_tmp_1','date_tmp_2','订单状态_new','pay_date']
# var = ['order_number','is_relet','当前期数', '下单日期', '应付日期', '当前应付租金', '实付金额','rembursement_status','订单状态','if_outpay','状态','首期应还日期', '买断价', '应付总租金', '买断尾款', '订单状态_new','租赁方案']
# df_xz_concat[df_xz_concat.order_number.isin(['A2024060200290566','A2024063001194047','A2024031113200319'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202308262317036'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202207050851373'])][var] 

# 4.月末状态

## 4_1.月末非逾期类指标

In [67]:
# （1）
df_xz_concat_1 = df_xz_concat.copy()
df_xz_concat_1.loc[:,"租赁方案"]=np.where(df_xz_concat_1["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_xz_concat_1['finish_date_new'] = pd.to_datetime(df_xz_concat_1['finish_date_new'])

tmp5 = df_xz_concat_1.groupby(['order_number']).agg({'应付日期':'max'}).reset_index()
tmp5 = tmp5.rename(columns = {'应付日期': '最大应还日期'})
df_xz_concat_1 = df_xz_concat_1.merge(tmp5, on = 'order_number', how = 'left')
df_xz_concat_1.shape

# （2）提取买断金等信息，区分续租和非续租 买断价，续租的买断价包含了首租的买断价
# 续租买断尾款 为续租总租金，续租买断尾款其实为0
# 首租订单，首租买断尾款为续租总租金，实际中，续租总租金 大于 首租买断尾款
# all_money：续租的已完成的话，为首租+续租总租金；续租的未完成的话，为首租的总租  'A202308262317036','A202207050851373'
sql = '''
SELECT tprm.order_id, tprm.order_number, tprm.order_create_time 下单日期
    ,tod.actual_money 首租买断价
    ,tod.new_actual_money 续租买断价
    , tprm.all_deposit 总押金
    , tprm.total_receivable 总应收
    , tprm.total_received 总已收
    # , om.all_money 首租总租金
    # , (tod.new_actual_money-om.all_money) 续租总租金
    # , (tod.actual_money-om.all_money) 首租买断尾款
    , tprm.purchase_amount 采购金额
    , om.has_actual 是否买断
    , om.relet_status 是否续租
    , tprm.paid_periods 已还期数
    , tod.early_completion_mark 是否提前完成
    , tmu.id_card_num 身份证号
    , tol.delivery_province_name 地区
    , pa.type 活动类型
    , pa.type, too.money 滞纳金, tprm.liquidated_damages 违约金
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_platform_activity pa on om.activity_id=pa.id
left join db_digua_business.t_member_user tmu on om.user_id=tmu.id
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join db_digua_business.t_order_logistics tol on tol.order_id=tprm.order_id
left join (SELECT t.order_id,sum(case when t.capital_type =6 and t.`status`=2 then t.money end) as other_money 
,sum(case when t.capital_type != 5 AND  t.`status` = 2 then t.money end) as money
from db_digua_business.t_order_other t GROUP BY 1 ) too on too.order_id=om.id
'''

df_temp_1 = query(sql)
df_temp_1.shape
df_temp_1 = df_temp_1.drop_duplicates(subset = ['order_number'])
df_temp_1.shape

df_temp_1_1 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].merge(df_temp_1[['order_number','首租买断价']], on = 'order_number', how = 'left')
df_temp_1_1 = df_temp_1_1.rename(columns = {'首租买断价': '买断价'})
df_temp_1_1.shape

df_temp_1_2 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].merge(df_temp_1[['order_number','续租买断价']], on = 'order_number', how = 'left')
df_temp_1_2 = df_temp_1_2.rename(columns = {'续租买断价': '买断价'})
df_temp_1_2.shape

df_xz_concat_1 = pd.concat([df_temp_1_1, df_temp_1_2])

# （3）加工应付总租金，区分续租和非续租 应付总租金
df_temp_2 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].groupby(['order_number']).agg({'当前应付租金':'sum'}).reset_index()
df_temp_2 = df_temp_2.rename(columns = {'当前应付租金': '应付总租金'})
df_temp_2.shape

df_temp_3 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].groupby(['order_number']).agg({'当前应付租金':'sum'}).reset_index()
df_temp_3 = df_temp_3.rename(columns = {'当前应付租金': '应付总租金'})
df_temp_3.shape

df_temp_4 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].merge(df_temp_2, on = 'order_number', how = 'left')
df_temp_4.shape

df_temp_5 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].merge(df_temp_3, on = 'order_number', how = 'left')
df_temp_5.shape

df_xz_concat_1 = pd.concat([df_temp_4, df_temp_5])


#（4）买断金 续租的买断尾款为 0 
df_xz_concat_1['买断尾款'] = df_xz_concat_1['买断价'] - df_xz_concat_1['应付总租金']
df_xz_concat_1['买断尾款'] = np.where(df_xz_concat_1.is_relet == 0, df_xz_concat_1['买断尾款'], 0)
df_xz_concat_1.shape

del df_temp_1_1
del df_temp_1_2
del df_temp_1
del df_temp_2
del df_temp_3
del df_temp_4
del df_temp_5

# （5）补数据  为了关联买断数据
# 当前期数、应付日期 改变，其他属性不变
tmp1 = df_xz_concat_1.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
var = ['order_number','order_id','is_relet','当前期数', '下单日期', '应付日期', '买断价', '应付总租金', '买断尾款','租赁方案','rembursement_status','finish_date_new']
tmp1.shape

# tmp2 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
# tmp2['当前期数'] = tmp2['当前期数'] + 1
# tmp2['应付日期'] = tmp2['应付日期'] + pd.DateOffset(months = 1)

# tmp3 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
# tmp3['当前期数'] = tmp3['当前期数'] + 2
# tmp3['应付日期'] = tmp3['应付日期'] + pd.DateOffset(months = 2)

# tmp4 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
# tmp4['当前期数'] = tmp4['当前期数'] + 3
# tmp4['应付日期'] = tmp4['应付日期'] + pd.DateOffset(months = 3)

# tmp5 = pd.concat([tmp2, tmp3, tmp4])
# tmp5.shape

# 2025.6 取数，补够31期，补到2025.4月底

for num in range(1,31):
    tmp2 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
    tmp2['当前期数'] = tmp2['当前期数'] + num
    tmp2['应付日期'] = tmp2['应付日期'] + pd.DateOffset(months = num)
    df_xz_concat_1 = pd.concat([df_xz_concat_1, tmp2])

df_xz_concat_1.shape

# （6）订单状态定位到 当月
df_xz_concat_1['date_tmp_1'] = df_xz_concat_1['应付日期'].dt.strftime('%Y-%m')
df_xz_concat_1['date_tmp_1'] = pd.to_datetime(df_xz_concat_1['date_tmp_1'])
df_xz_concat_1['date_tmp_2'] = np.where(df_xz_concat_1['finish_date_new'].notnull(), pd.to_datetime(df_xz_concat_1['finish_date_new']).dt.strftime('%Y-%m'), np.nan)
df_xz_concat_1['date_tmp_2'] = pd.to_datetime(df_xz_concat_1['date_tmp_2'])

# 判断订单结束日期非空的订单是买断还是非买断，未结束的订单不判断（资方所需的是是否结清，分买断或完成）
df_xz_concat_1['订单状态_new'] = df_xz_concat_1.apply(lambda x: '已买断' if x.date_tmp_1 == x.date_tmp_2 and x.rembursement_status == '已买断'
                                            else '已完成' if x.date_tmp_1 == x.date_tmp_2 and x.rembursement_status == '已完成'
                                            else '还款中' if x.date_tmp_1 < x.date_tmp_2
                                            else '已结束' if x.date_tmp_1 > x.date_tmp_2
                                            else '其他', axis = 1
                                            )

df_xz_concat_1['提前状态'] = df_xz_concat_1.apply(lambda x: '提前买断' if x.finish_date_new < x.最大应还日期 and x.订单状态_new == '已买断'
                                            else '已完成' if x.finish_date_new < x.最大应还日期 and x.订单状态_new == '已完成'
                                            else '其他', axis = 1
                                            )
# df_xz_concat_1 = df_xz_concat_1.drop(columns = ['date_tmp_1', 'date_tmp_2'])
df_xz_concat_1.shape

# (7)关联买断数据 
sql1 = '''-- 买断信息   
SELECT too.order_id
,too.real_pay_money 
,too.pay_date
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断
and too.real_pay_money > 0 and too.pay_date is not null
'''
df_out_1 = query(sql1)
df_out_1.shape
df_out_1["pay_date"] = df_out_1['pay_date'].dt.date
df_out_1["pay_date"] = pd.to_datetime(df_out_1["pay_date"],errors="coerce")
df_out_1['pay_date_m'] = df_out_1['pay_date'].dt.strftime('%Y-%m')
df_out_1['pay_date_m'] = pd.to_datetime(df_out_1['pay_date_m'])

df_out_2 = df_out_1.groupby(['order_id','pay_date_m']).agg({'real_pay_money':'sum'}).reset_index()
df_out_2 = df_out_2.rename(columns = {'real_pay_money': 'real_pay_money_new'})
df_out_2.shape
# 匹配订单每月买断金额
df_xz_concat_1 = df_xz_concat_1.merge(df_out_2[['order_id', 'real_pay_money_new', 'pay_date_m']], left_on = ['order_id', 'date_tmp_1'], right_on = ['order_id','pay_date_m'], how = 'left')

# 计算剩余买断尾款
df_xz_concat_1['real_pay_money_new'] = df_xz_concat_1['real_pay_money_new'].fillna(0)
df_xz_concat_1 = df_xz_concat_1.rename(columns = {'real_pay_money_new': '当月已收买断金'})
df_xz_concat_1 = df_xz_concat_1.sort_values(by = ['order_number','当前期数'])
df_xz_concat_1['当前累计已收买断金'] = df_xz_concat_1.groupby(['order_number'])['当月已收买断金'].cumsum()

df_xz_concat_1['剩余总应收买断金'] = df_xz_concat_1['买断尾款'] - df_xz_concat_1['当前累计已收买断金'] 
df_xz_concat_1['剩余总应收买断金'] = np.where(df_xz_concat_1['剩余总应收买断金'] < 0 , 0, df_xz_concat_1['剩余总应收买断金'])
df_xz_concat_1['剩余总应收买断金'] = np.where(df_xz_concat_1['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_xz_concat_1['剩余总应收买断金'])

# （8）2025.6.27 
# 更新实付金额：应付日期 大于 订单结束日期，实付金额为0的，且已买断的，用应付填充
conditions_1 = (df_xz_concat_1.应付日期 >= df_xz_concat_1.finish_date_new)
conditions_2 = (df_xz_concat_1.实付金额 == 0)
conditions_3 = (df_xz_concat_1.rembursement_status == '已买断')

df_xz_concat_1['实付金额'] = np.where(conditions_1 & conditions_2 & conditions_3, df_xz_concat_1.当前应付租金, df_xz_concat_1.实付金额)

df_xz_concat_1.shape
df_xz_concat_1[:1]

# var1 = ['order_number','is_relet','当前期数', '下单日期', '应付日期', '当前应付租金', '实付日期new','实付金额','rembursement_status','订单状态','if_outpay','状态','订单状态_new']
# var2 = ['mob_date_11','paid_date_mob11','paid_money_mob11','mob_date_12','paid_date_mob12','paid_money_mob12','mob_date_13','paid_date_mob13','paid_money_mob13','mob_date_14','paid_date_mob14','paid_money_mob14','mob_date_15','paid_date_mob15','paid_money_mob15']
# var3 = ['order_id','当前期数','应付日期','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# var = var1 + var2 + var3

# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3]
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]
# 续租的实付买断金非0


(1028659, 177)

(82698, 19)

(82648, 19)

(932578, 178)

(96081, 178)

...

(3355999, 184)

(9987, 3)

(9799, 3)

(3355999, 188)

,id_x,order_id,当前应付租金,实付金额,interest_money,实付日期,status_x,当前期数,order_pay_id,create_time_x,update_time_x,type_x,advance_time,merchant_account_id,order_relet_id,is_relet,pay_type,sesame_promo_money,sesame_promo_money_pay,promo_money,plat_subsidy,promo_money_show,pay_source,应付日期,id_y,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time_y,update_time_y,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,province_name,product_name,sku_attributes,channel_name_cc,channel_type_id,activity_name,type_y,order_type,tips,is_vip,status_result,订单号,渠道名称,来源渠道,归属渠道,是否免人审,是否拒量,是否号卡,内存,颜色,是否二手,是否带锁,下单日期,月份,年份,季度,outpay_money,pay_date,status_y,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,实付日期1,实付日期new,now_date,状态,逾期天数,agr_1d,agr_4d,agr_7d,agr_15d,agr_30d,date_str,agr_1d_cum,agr_4d_cum,agr_7d_cum,agr_15d_cum,agr_30d_cum,mob_date_0,paid_date_mob0,paid_money_mob0,mob_date_1,paid_date_mob1,paid_money_mob1,mob_date_2,paid_date_mob2,paid_money_mob2,mob_date_3,paid_date_mob3,paid_money_mob3,mob_date_4,paid_date_mob4,paid_money_mob4,mob_date_5,paid_date_mob5,paid_money_mob5,mob_date_6,paid_date_mob6,paid_money_mob6,mob_date_7,paid_date_mob7,paid_money_mob7,mob_date_8,paid_date_mob8,paid_money_mob8,mob_date_9,paid_date_mob9,paid_money_mob9,mob_date_10,paid_date_mob10,paid_money_mob10,mob_date_11,paid_date_mob11,paid_money_mob11,mob_date_12,paid_date_mob12,paid_money_mob12,label,租赁方案,最大应还日期,买断价,应付总租金,买断尾款,date_tmp_1,date_tmp_2,订单状态_new,提前状态,当月已收买断金,pay_date_m,当前累计已收买断金,剩余总应收买断金
0,6506.0,8924,357.95,357.95,0.0,2022-07-05,3.0,1.0,9576,2022-07-05 08:51:37,2022-07-05 10:41:10,0.0,NaT,6270.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-07-05,1.0,A202207050851373,2022年07月,2022-07,2024-10-02,14.0,迪瓜优选,0.0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,2121.2,1.0,357.95,2022-07-05 08:51:37,已买断,12.0,NaT,0.0,4.0,0.0,6416.6,7530.0,7847.2,-3881.52,329.72,8176.92,-1760.32,-3026.92,,1266.6,0.0,3026.92,0.0,11.0,301.0,3551.8,NaN,None,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0.0,None,None,0.0,8.0,已完成,2024-04-17 01:37:33,0.0,胡水飘,362202199805282550,13027948681,江西省,官方质检 iPhone 12 Pro 国行 99新,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",None,NaN,None,NaN,ZFB_ORDER,None,None,None,A202207050851373,未知渠道,未知渠道,未知渠道,0.0,0.0,0.0,128G,金色,二手,未带锁,2022-07-05,7.0,2022.0,3季度,329.72,2024-04-17 01:37:33,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-07-05,2022-07-05,2025-12-12,正常还款,0.0,2022-07-06,2022-07-09,2022-07-12,2022-07-20,2022-08-04,2025-12-12,1.0,1.0,1.0,1.0,1.0,2022-07-31,2022-07-05,357.95,2022-08-31,2022-07-05,357.95,2022-09-30,2022-07-05,357.95,2022-10-31,2022-07-05,357.95,2022-11-30,2022-07-05,357.95,2022-12-31,2022-07-05,357.95,2023-01-31,2022-07-05,357.95,2023-02-28,2022-07-05,357.95,2023-03-31,2022-07-05,357.95,2023-04-30,2022-07-05,357.95,2023-05-31,2022-07-05,357.95,2023-06-30,2022-07-05,357.95,2023-07-31,2022-07-05,357.95,0.0,租完归还,2024-05-31,7530.0,4295.4,3234.6,2022-07-01,2024-04-01,还款中,其他,0.0,NaT,0.0,3234.6


## 4_2.首租

In [68]:
# （4-2）首租
# （1）非续租租期数据
# (384318, 161)
# df_fq_concat = df_xz_concat[df_xz_concat.is_relet==0]
# 2025.2.9
df_xz_concat_2 = df_xz_concat.merge(df_xz_concat_1[['order_number', '订单状态_new','当前期数']],on = ['order_number','当前期数'], how = 'inner')
# conditions_1 = (df_xz_concat_2.订单状态_new != '已结束') | ((df_xz_concat_2.订单状态_new == '已结束') & (df_xz_concat_2.实付金额 > 0))
# df_xz_concat_2 = df_xz_concat_2[conditions_1]
df_fq_concat = df_xz_concat_2[df_xz_concat_2.is_relet==0]
df_fq_concat['当前应付租金'] = np.where((df_fq_concat.订单状态_new == '已结束') & (df_fq_concat.实付金额 == 0), 0, df_fq_concat.当前应付租金)

df_fq_concat.shape
df_xz_concat.is_relet.value_counts()

# note 20241205 首个应付日期更新为下单日期，避免 首个应付日期为下个月的订单没有 把mob0的已付统计在内，首个应收日期为下个月的订单，mob1没有账单日
df_fq_concat['应付日期_new'] = np.where(df_fq_concat.当前期数 == 1, df_fq_concat.下单日期, df_fq_concat.应付日期)
# note 20241206 更新 应付日期，提前还款的统计在内
df_fq_concat['应付日期_new'] = np.where((df_fq_concat['实付日期new'].notna())&((df_fq_concat['应付日期_new'] - df_fq_concat['实付日期new']).dt.days >= 0),df_fq_concat['实付日期new'],df_fq_concat['应付日期_new'])
df_fq_concat.shape

# （2）12期加工
# note 20241205 每月28.29.30号下单的没有统计mob_0已付的
df_mob_list = []
for i in range(0,13):
# 2025.2.9
# for i in range(0,16):
    # note 20241205 增加 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])，不然预收的没有在第1期体现出来， A2024063001194047 下单日期!=实付日期new（第一期）
    # note 20241205 增加=，不然28或27号的没有mob0，导致 paid_money_mob 计算不正确，即还款日为月末的计算不正确
    # df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期_new']).dt.days >= 0) 
                        #   & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                        # 2025.2.9
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期_new']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                          | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    # note 20241127 overdue_days_mob 截止每个mob观察月的逾期天数，月末状态，若月末不逾期了，则为0
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    
    # 2025.2.11，当前最大期数不用 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
    df_mob_group_1_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前期数':'max'}).reset_index()
    
    df_mob_group_1 = df_mob_group_1.merge(df_mob_group_1_1, on = 'order_number', how = 'inner')

    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')

    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)

# 2025.2.9
df_mob_total.shape
df_mob_total = df_mob_total[df_mob_total.当前应付租金 >= 0]
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_total.shape
df_mob_total = df_mob_total.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','当前期数'])

# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --2个12期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

# paid_money_mob：当前累计实付租金  当前应付租金：当前累计应付租金
df_mob_total = df_mob_total.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_total.tail(10)

df_xz_concat_1.shape
df_xz_concat.shape
df_fq_concat.shape
df_mob_total.shape

# （2）
# note 20241205 每月28.29.30号下单的没有统计mob_0已付的
df_mob_list = []
for i in range(0,13):
# 2025.2.9
# for i in range(0,16):
    # note 20241205 增加 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])，不然预收的没有在第1期体现出来， A2024063001194047 下单日期!=实付日期new（第一期）
    # note 20241205 增加=，不然28或27号的没有mob0，导致 paid_money_mob 计算不正确，即还款日为月末的计算不正确
    # df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期_new']).dt.days >= 0) 
                        #   & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                        # 2025.2.9
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期_new']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                          | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    # note 20241127 overdue_days_mob 截止每个mob观察月的逾期天数，月末状态，若月末不逾期了，则为0
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    
    # 2025.2.11，当前最大期数不用 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
    df_mob_group_1_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前期数':'max'}).reset_index()
    
    df_mob_group_1 = df_mob_group_1.merge(df_mob_group_1_1, on = 'order_number', how = 'inner')

    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')

    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)

# 2025.2.9
df_mob_total.shape
df_mob_total = df_mob_total[df_mob_total.当前应付租金 >= 0]
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_total.shape
df_mob_total = df_mob_total.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','当前期数'])

# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --2个12期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

# paid_money_mob：当前累计实付租金  当前应付租金：当前累计应付租金
df_mob_total = df_mob_total.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_total.tail(10)

df_xz_concat_1.shape
df_xz_concat.shape
df_fq_concat.shape
df_mob_total.shape

# (520033, 170)
# (424063, 161)
# (384318, 163)
# (234387, 7)

# （3）关联多期的表
# 2025.2.9
var_1 = ['order_number','下单日期','租赁方案','当前期数','应付日期', '当前应付租金', '实付金额','实付日期new','买断价','应付总租金','买断尾款','订单状态_new','当月已收买断金','当前累计已收买断金','剩余总应收买断金','提前状态']
df_mob_total_tmp = df_xz_concat_1[df_xz_concat_1.is_relet == 0][var_1].merge(df_mob_total[['order_number','当前期数','当前累计应付租金','当前累计实付租金','overdue_days_mob','MOB']], on = ['order_number','当前期数'], how = 'left')
df_xz_concat_1[df_xz_concat_1.is_relet == 0].shape
df_mob_total_tmp.shape

tmp_2 = df_xz_concat.groupby(['order_number']).agg({'is_relet':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'is_relet': 'is_relet_max'})
tmp_2.shape

df_mob_total_tmp = df_mob_total_tmp.merge(tmp_2, on = 'order_number', how ='inner')
df_mob_total_tmp.shape
del tmp_2

# 加工 df_mob_total_tmp
# 续租的订单只有12期，第12期的订单状态修改为已完成
df_mob_total_tmp['订单状态_new'] = np.where((df_mob_total_tmp.is_relet_max >= 1) & (df_mob_total_tmp.当前期数 == 12), '已完成', df_mob_total_tmp['订单状态_new'])
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收买断金'])
# 删去订单已结束的订单
df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.订单状态_new != '已结束']
df_mob_total_tmp.shape
# 订单未结束的，删去未到期的，最新时间为 2025.2.10
# df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.应付日期 <= '2025-01-31']
# df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.应付日期 <= '2025-02-28']
# df_mob_total_tmp.shape

now_date = pd.Timestamp(dt.now().date())
df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.应付日期 <= now_date]
df_mob_total_tmp.shape

# 订单未结束的，到期未结束的则取完15期的数据，若租期一直逾期，则衍生到15期，若租期逾期，则默认不逾期
# A2024020317504938 2025.2 买断，还没表现出来
# 'A202207171820512' -- 9683，在15期之外买断，不加工租期逾期
# 'A202207181530165' --一直逾期，直至租期到了之后也逾期

df_mob_total_tmp['overdue_days_mob'] = df_mob_total_tmp['overdue_days_mob'].fillna(0)
df_mob_total_tmp['当前累计应付租金'] = df_mob_total_tmp['当前累计应付租金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)

tmp_2 = df_mob_total.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'当前期数': '当前期数'})
tmp_3 = df_mob_total[['order_number','当前期数','当前累计实付租金','当前累计应付租金','overdue_days_mob']].merge(tmp_2, on = ['order_number','当前期数'], how = 'inner')
tmp_2.shape
tmp_3.shape

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + 30, df_mob_total_tmp['overdue_days_mob_x'])
# df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
# df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
# df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + 60, df_mob_total_tmp['overdue_days_mob_x'])
# df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
# df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
# df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + 90, df_mob_total_tmp['overdue_days_mob_x'])
# df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
# df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
# df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# 2025.2 为 31
# 2025.9 为 38
for num in range(1,38):
    tmp_3['当前期数'] = tmp_3['当前期数'] + 1
    df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
    # df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + num*30, df_mob_total_tmp['overdue_days_mob_x'])
    df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] >= 0, df_mob_total_tmp['overdue_days_mob_y'] + num*30, df_mob_total_tmp['overdue_days_mob_x'])
    df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
    df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
    df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})


df_mob_total_tmp.shape
df_mob_total_tmp['search_time'] = df_mob_total_tmp['下单日期'].dt.strftime('%Y-%m')

tmp = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp = tmp.rename(columns = {'当前期数': '最大期数'})
df_mob_total_tmp = df_mob_total_tmp.merge(tmp, on = 'order_number', how = 'left')
df_mob_total_tmp.shape

df_mob_total_tmp['买断尾款'] = np.where(df_mob_total_tmp.租赁方案 == '租完即送', 0, df_mob_total_tmp.买断尾款)
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp.租赁方案 == '租完即送', 0, df_mob_total_tmp.剩余总应收买断金)

df_mob_total_tmp['应付总租金'] = df_mob_total_tmp['应付总租金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)
df_mob_total_tmp['剩余总应收租金'] = df_mob_total_tmp['应付总租金'] - df_mob_total_tmp['当前累计实付租金']
df_mob_total_tmp['剩余总应收租金'] = np.where(df_mob_total_tmp['剩余总应收租金'] < 0, 0, df_mob_total_tmp['剩余总应收租金'])
df_mob_total_tmp['剩余总应收租金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收租金'])

df_mob_total_tmp['剩余总应收'] = df_mob_total_tmp['剩余总应收租金'] + df_mob_total_tmp['剩余总应收买断金']
df_mob_total_tmp['剩余总应收'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收'])
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收买断金'])

df_mob_total_tmp['当前累计已收买断金'] = df_mob_total_tmp['当前累计已收买断金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)
df_mob_total_tmp['当前累计实付'] = df_mob_total_tmp['当前累计已收买断金'] + df_mob_total_tmp['当前累计实付租金']
df_mob_total_tmp['总应收'] = df_mob_total_tmp['应付总租金'] + df_mob_total_tmp['买断尾款']

# A2023112717153031
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp.订单状态_new.isin(['已买断','已完成','已结束']), 0 , df_mob_total_tmp.overdue_days_mob)
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp.当前期数 == 1, 0 , df_mob_total_tmp.overdue_days_mob)
df_mob_total_tmp[:1]

# df_mob_total_tmp[df_mob_total_tmp.order_number == 'A2023112717153031'][['order_number','当前期数','应付日期','overdue_days_mob']]
# df_xz_concat[df_xz_concat.order_number == 'A2023112717153031'][['order_number','当前期数','应付日期','实付日期','实付金额','is_relet']]

df_mob_total_tmp['overdue_days_mob'] = np.where((df_mob_total_tmp.剩余总应收租金 == 0) & (df_mob_total_tmp.租赁方案 == '租完即送'), 0, df_mob_total_tmp.overdue_days_mob)

(932578, 176)

is_relet
0.0    932578
1.0     95622
2.0       459
Name: count, dtype: int64

(932578, 177)

(945611, 7)

(601185, 7)

...

(75005, 2)

(75005, 5)

(681708, 21)

(681708, 23)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,is_relet_max,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,最大期数,剩余总应收租金,剩余总应收,当前累计实付,总应收
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,2.0,0.0,357.95,357.95,2022-07,12.0,3937.45,7172.05,357.95,7530.0


## 4_3.续租

In [69]:
# （2-3）续租
# （1）续租数据
# (39745, 162)
# df_relet = df_xz_concat[df_xz_concat.is_relet>=1]
# df_relet.shape

# # 2025.2.11
df_xz_concat_2 = df_xz_concat.merge(df_xz_concat_1[['order_number', '订单状态_new','当前期数']],on = ['order_number','当前期数'], how = 'inner')
# conditions_1 = (df_xz_concat_2.订单状态_new != '已结束') | ((df_xz_concat_2.订单状态_new == '已结束') & (df_xz_concat_2.实付金额 > 0))
# df_xz_concat_2 = df_xz_concat_2[conditions_1]
df_relet = df_xz_concat_2[df_xz_concat_2.is_relet >= 1]
df_relet['当前应付租金'] = np.where((df_relet.订单状态_new == '已结束') & (df_relet.实付金额 == 0), 0, df_relet.当前应付租金)

df_relet.shape
df_xz_concat.is_relet.value_counts()

# （2）每个mob对应的月底时间，有些订单mob1不用还，故 n 放到 24
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(13,25):  
        # note 20241209 修改 月末的的跳级了，所以不要加1，mob对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}']) 
    return df

df_relet = mob_date(df_relet)
df_relet.shape
df_relet[['下单日期','mob_date_15','mob_date_21','mob_date_20','mob_date_13','实付金额','paid_money_mob13','paid_date_mob15','paid_money_mob18']][:2]

# （3）每个mob对应的逾期天数  
# note 20241209 更新 应付日期，提前还款的统计在内
df_relet['应付日期_new'] = np.where((df_relet['实付日期new'].notna())&((df_relet['应付日期'] - df_relet['实付日期new']).dt.days >= 0),df_relet['实付日期new'],df_relet['应付日期'])
df_mob_list = []

for i in range(12,25): 
    # note 20241209 增加 =
    df_mob = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期_new']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                    #   2025.2.11
                    #   & ((df_relet['date_str'] - df_relet['应付日期_new']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ]
   
    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']
    # df_mob

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum','当前期数':'max'}).reset_index()
    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')


    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_relet = pd.concat(df_mob_list, ignore_index=True)

# 数据检查
# order_number order_id
# A202207050851373    8924     已买断，买断金额没有体现在实付金额中，买断后剩余期数的实付金额为0；2023-07-05、2023-10-3的还款记录在系统没有找到
# A2023073114301076   508404  只有mob13-16，mob12在非续租那里了，2023.8不用还款，隔月了，故非续租为13个月长度
# A2023070210395232   432132  只有mob12-16，其他的未到期
# A20231026115543149  726194  只有mob12、mob13，其他的未到期

# df_mob_relet[df_mob_relet.order_number == 'A202207050851373']
# 有mob12但无mob13
# A2023110100321259  738560  23
# 有mob13但无mob12
# A202208291922283    18163  26  2023.2-2023.5 的账单没有还，但2023.6-2023.8的账单还了
# A202207282047042    11630  25

# order_number_param = 'A202207282047042'
# var = ['mob_date_0','mob_date_11','mob_date_12','mob_date_13','mob_date_21','mob_date_23']
# var1 = ['mob_date_0','mob_date_1','mob_date_2','mob_date_3','mob_date_4','mob_date_11','mob_date_12']
# df_xz_concat[df_xz_concat.order_number == order_number_param][['order_number','下单日期','应付日期','实付日期new','实付金额] + var1]
# df_relet[df_relet.order_number == order_number_param][['order_number','order_id','下单日期','状态','应付日期','当前应付租金','实付日期new','实付金额','逾期天数'] + var]
# df_mob_relet[df_mob_relet.order_number == order_number_param][['order_number','下单日期','当前应付租金','paid_money_mob','overdue_days_mob','MOB']]

# 2025.2.9
df_mob_relet.shape
df_mob_relet = df_mob_relet[df_mob_relet.当前应付租金 >= 0]
df_mob_relet = df_mob_relet.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_relet.shape
df_mob_relet = df_mob_relet.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_relet = df_mob_relet.drop_duplicates(subset = ['order_number','当前期数'])

df_mob_relet = df_mob_relet.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_relet.tail(10)

# （4）关联多期的表
# 2025.2.11
var_1 = ['order_number','下单日期','租赁方案','当前期数','应付日期', '当前应付租金', '实付金额','实付日期new','买断价','应付总租金','买断尾款','订单状态_new','当月已收买断金','当前累计已收买断金','剩余总应收买断金','提前状态']
df_mob_relet_tmp = df_xz_concat_1[df_xz_concat_1.is_relet >= 1][var_1].merge(df_mob_relet[['order_number','当前期数','当前累计应付租金','当前累计实付租金','overdue_days_mob','MOB']], on = ['order_number','当前期数'], how = 'left')
df_xz_concat_1[df_xz_concat_1.is_relet >= 1].shape
df_mob_relet_tmp.shape

# tmp_2 = df_xz_concat.groupby(['order_number']).agg({'is_relet':'max'}).reset_index()
# tmp_2 = tmp_2.rename(columns = {'is_relet': 'is_relet_max'})
# tmp_2.shape

# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_2, on = 'order_number', how ='inner')
# df_mob_relet_tmp.shape
# del tmp_2

# 加工 df_mob_relet_tmp
# 续租的订单只有12期，第12期的订单状态修改为已完成
# df_mob_relet_tmp['订单状态_new'] = np.where((df_mob_relet_tmp.is_relet_max >= 1) & (df_mob_relet_tmp.当前期数 == 12), '已完成', df_mob_relet_tmp['订单状态_new'])
# 删去订单已结束的订单
df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.订单状态_new != '已结束']
df_mob_relet_tmp.shape
# 订单未结束的，删去未到期的，最新时间为 2025.2.10
# df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.应付日期 <= '2025-01-31']
# df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.应付日期 <= '2025-02-28']
df_mob_relet_tmp.shape

now_date = pd.Timestamp(dt.now().date())
df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.应付日期 <= now_date]
df_mob_relet_tmp.shape

# 订单未结束的，到期未结束的则取完15期的数据，若租期一直逾期，则衍生到15期，若租期逾期，则默认不逾期
# A2024020317504938 2025.2 买断，还没表现出来
# 'A202207171820512' -- 9683，在15期之外买断，不加工租期逾期
# 'A202207181530165' --一直逾期，直至租期到了之后也逾期

df_mob_relet_tmp['overdue_days_mob'] = df_mob_relet_tmp['overdue_days_mob'].fillna(0)
df_mob_relet_tmp['当前累计应付租金'] = df_mob_relet_tmp['当前累计应付租金'].fillna(0)
df_mob_relet_tmp['当前累计实付租金'] = df_mob_relet_tmp['当前累计实付租金'].fillna(0)

tmp_2 = df_mob_relet.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'当前期数': '当前期数'})
tmp_3 = df_mob_relet[['order_number','当前期数','当前累计实付租金','当前累计应付租金','overdue_days_mob']].merge(tmp_2, on = ['order_number','当前期数'], how = 'inner')
tmp_2.shape
tmp_3.shape

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 30, df_mob_relet_tmp['overdue_days_mob_x'])
# df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
# df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
# df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 60, df_mob_relet_tmp['overdue_days_mob_x'])
# df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
# df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
# df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 90, df_mob_relet_tmp['overdue_days_mob_x'])
# df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
# df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
# df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# 2025.2 为 31
# 2025.9 为 38
for num in range(1,38):
    tmp_3['当前期数'] = tmp_3['当前期数'] + 1
    df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
    # df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + num*30, df_mob_relet_tmp['overdue_days_mob_x'])
    df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] >= 0, df_mob_relet_tmp['overdue_days_mob_y'] + num*30, df_mob_relet_tmp['overdue_days_mob_x'])
    df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
    df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
    df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

df_mob_relet_tmp.shape
df_mob_relet_tmp['search_time'] = df_mob_relet_tmp['下单日期'].dt.strftime('%Y-%m')

tmp = df_xz_concat[df_xz_concat.is_relet >= 1].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp1 = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp.shape
tmp1.shape
tmp = tmp.merge(tmp1, on = 'order_number', how = 'left')
tmp.shape
tmp['续租总期数'] = tmp['当前期数_x'] - tmp['当前期数_y']
tmp = tmp[['order_number','续租总期数']]

# tmp = tmp.rename(columns = {'当前期数': '最大期数'})
df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp, on = 'order_number', how = 'left')
df_mob_relet_tmp.shape

# df_mob_relet_tmp['买断尾款'] = np.where(df_mob_relet_tmp.租赁方案 == '租完即送', 0, df_mob_relet_tmp.买断尾款)
# df_mob_relet_tmp['剩余总应收买断金'] = np.where(df_mob_relet_tmp.租赁方案 == '租完即送', 0, df_mob_relet_tmp.剩余总应收买断金)

df_mob_relet_tmp['应付总租金'] = df_mob_relet_tmp['应付总租金'].fillna(0)
df_mob_relet_tmp['当前累计实付租金'] = df_mob_relet_tmp['当前累计实付租金'].fillna(0)
df_mob_relet_tmp['剩余总应收租金'] = df_mob_relet_tmp['应付总租金'] - df_mob_relet_tmp['当前累计实付租金']
df_mob_relet_tmp['剩余总应收租金'] = np.where(df_mob_relet_tmp['剩余总应收租金'] < 0, 0, df_mob_relet_tmp['剩余总应收租金'])
df_mob_relet_tmp['剩余总应收租金'] = np.where(df_mob_relet_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_relet_tmp['剩余总应收租金'])

# df_mob_relet_tmp['剩余总应收'] = df_mob_relet_tmp['剩余总应收租金'] + df_mob_relet_tmp['剩余总应收买断金']
df_mob_relet_tmp['剩余总应收'] = df_mob_relet_tmp['剩余总应收租金']
df_mob_relet_tmp['当前累计已收买断金'] = 0
df_mob_relet_tmp['当前累计实付租金'] = df_mob_relet_tmp['当前累计实付租金'].fillna(0)
df_mob_relet_tmp['当前累计实付'] = df_mob_relet_tmp['当前累计已收买断金'] + df_mob_relet_tmp['当前累计实付租金']
df_mob_relet_tmp['总应收'] = df_mob_relet_tmp['应付总租金'] + df_mob_relet_tmp['买断尾款']

# A2023112717153031
df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp.订单状态_new.isin(['已买断','已完成','已结束']), 0 , df_mob_relet_tmp.overdue_days_mob)
# note 2025.2.28
df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp.剩余总应收租金 == 0, 0, df_mob_relet_tmp.overdue_days_mob)

df_mob_relet_tmp[:1]

(96081, 176)

is_relet
0.0    932578
1.0     95622
2.0       459
Name: count, dtype: int64

(96081, 212)

,下单日期,mob_date_15,mob_date_21,mob_date_20,mob_date_13,实付金额,paid_money_mob13,paid_date_mob15,paid_money_mob18
12,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-07-07,354.0
13,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-08-07,354.0


(73817, 7)

...

(10165, 2)

(77578, 2)

(10165, 3)

(72174, 22)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,续租总期数,剩余总应收租金,剩余总应收,当前累计实付,总应收
0,A202207050851373,2022-07-05,租完归还,13.0,2023-07-05,354.0,354.0,2023-07-07,8176.92,3953.0,0.0,还款中,0.0,0,0.0,其他,mob_12,0.0,354.0,354.0,2022-07,12.0,3599.0,3599.0,354.0,3953.0


## 4_4.表合并

In [70]:
df_mob_total_tmp.shape
df_mob_relet_tmp.shape
# view_month 为系统月的 overdue_days_mob 有误，不要使用

# 当前累计实付租金 等指标区分了首租和续租，要合并起来
tmp1 = df_mob_total_tmp.groupby(['order_number']).agg({'当前累计实付租金': 'max'}).reset_index()
tmp1 = tmp1.rename(columns = {'当前累计实付租金': '当前累计实付租金_首租'})
df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp1, on = 'order_number', how = 'left')
df_mob_relet_tmp['当前累计实付租金_首租'] = df_mob_relet_tmp['当前累计实付租金_首租'].fillna(0)
df_mob_relet_tmp['当前累计实付租金_all'] = df_mob_relet_tmp['当前累计实付租金_首租'] + df_mob_relet_tmp['当前累计实付租金']
df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = ['当前累计实付租金_首租'])

df_mob_total_tmp['当前累计实付租金_all'] = df_mob_total_tmp['当前累计实付租金']

df_mob_temp = pd.concat([df_mob_total_tmp, df_mob_relet_tmp])
# df_mob_temp['当前累计已收买断金'] = df_mob_temp['当前累计已收买断金'].fillna(0)
df_mob_temp['当前累计实付'] = df_mob_temp['当前累计已收买断金'] + df_mob_temp['当前累计实付租金_all']
# df_mob_temp['总应收'] = df_mob_temp['应付总租金'] + df_mob_temp['买断尾款']
df_mob_temp.shape
# df_mob_temp[:1]


# 更新续租订单的订单状态：（只更新首租到期的那期的订单状态） 
# 订单状态_new 为各期对应的状态；rembursement_status为订单当前状态
df_mob_temp['订单状态_new'] = np.where((df_mob_temp['订单状态_new'] == '已完成') & (df_mob_temp.is_relet_max >= 1), '还款中', df_mob_temp['订单状态_new'])
df_mob_temp['view_month'] = df_mob_temp['应付日期'].dt.strftime('%Y-%m')


# 计算剩余采购成本
# 实付金额全部折为采购金额， 若剩余采购金额<0，则为0 
# 已买断/已完成/检测中 的当月及之后，剩余采购成本为0
# 检测中的订单，实付日期为空的最小的应付日期那条，剩余采购成本为0，逾期天数为0,大于最小应付日期的 样本删去

df_mob_temp = df_mob_temp.merge(df_xz_concat[df_xz_concat.当前期数 == 1][['order_number','purchase_amount','rembursement_status']], on = 'order_number', how = 'left')

df_mob_temp['剩余采购成本'] = df_mob_temp['purchase_amount'] - df_mob_temp['当前累计实付租金_all']
# 2025.7.4 修改：不跑，统一在 统计 的前一步跑
# df_mob_temp['剩余采购成本'] = np.where(df_mob_temp.剩余采购成本 < 0, 0 , df_mob_temp.剩余采购成本)
df_mob_temp['剩余采购成本'] = np.where(df_mob_temp.订单状态_new.isin(['已买断','已完成']), 0, df_mob_temp.剩余采购成本)

tmp1 = df_mob_temp[(df_mob_temp.rembursement_status == '检测中') & (df_mob_temp.实付日期new.isnull())].groupby(['order_number']).agg({'应付日期': 'min'}).reset_index()
tmp1 = tmp1.rename(columns = {'应付日期': '应付日期_检测中'})
df_mob_temp = df_mob_temp.merge(tmp1, on = ['order_number'], how = 'left')
df_mob_temp['剩余采购成本'] = np.where((df_mob_temp.rembursement_status == '检测中')&(df_mob_temp.应付日期 >= df_mob_temp.应付日期_检测中), 0, df_mob_temp.剩余采购成本)
df_mob_temp['overdue_days_mob'] = np.where((df_mob_temp.rembursement_status == '检测中')&(df_mob_temp.应付日期 >= df_mob_temp.应付日期_检测中), 0, df_mob_temp.overdue_days_mob)

tmp1 = df_mob_temp[df_mob_temp.rembursement_status != '检测中']
tmp2 = df_mob_temp[df_mob_temp.rembursement_status == '检测中']
tmp2 = tmp2[tmp2.应付日期 <= tmp2.应付日期_检测中]
df_mob_temp = pd.concat([tmp1, tmp2])
df_mob_temp = df_mob_temp.drop(columns = ['应付日期_检测中','最大期数'])

tmp1 = df_xz_concat.groupby(['order_number']).agg({'当前期数': 'max'}).reset_index()
tmp1 = tmp1.rename(columns = {'当前期数': '最大期数'})
df_mob_temp = df_mob_temp.merge(tmp1, on = ['order_number'], how = 'left')
df_mob_temp['最大期数_Bin'] = df_mob_temp.apply(lambda x: '12个月' if x.最大期数 <= 12
                                            else '18个月' if x.最大期数 > 12 and x.最大期数 <= 18
                                            else '24个月' if x.最大期数 > 18
                                            else 'other', axis = 1)

# 更新MOB -方式1（弱点：# 差异对比 如 A2023122617252850：view_month == '2024-12' 没有，因为续租跳月了，但MOB12有）
# df_mob_temp[(~df_mob_temp.order_number.isin(not_count_order_number))&(df_mob_temp.view_month == '2024-12')&(df_mob_temp.search_time == '2023-12')].剩余采购成本.sum()
# df_mob_temp[(~df_mob_temp.order_number.isin(not_count_order_number))&(df_mob_temp.MOB_new_1 == 'MOB12')&(df_mob_temp.search_time == '2023-12')].剩余采购成本.sum()
# df_mob_temp[(~df_mob_temp.order_number.isin(not_count_order_number))&(df_mob_temp.MOB_new == 'MOB12')&(df_mob_temp.search_time == '2023-12')].剩余采购成本.sum()

df_mob_temp['date_tmp_1'] = df_mob_temp['应付日期'].dt.strftime('%Y-%m')
df_mob_temp['date_tmp_1'] = pd.to_datetime(df_mob_temp['date_tmp_1'])
df_mob_temp['date_tmp_2'] = df_mob_temp['下单日期'].dt.strftime('%Y-%m')
df_mob_temp['date_tmp_2'] = pd.to_datetime(df_mob_temp['date_tmp_2'])

df_mob_temp['当前期数_tmp'] = df_mob_temp.apply(lambda x: 1 if x.date_tmp_1 == x.date_tmp_2 and x.当前期数 == 1
                                            else 0 if x.date_tmp_1 != x.date_tmp_2 and x.当前期数 == 1
                                            else np.nan, axis = 1)
# 向下填充
df_mob_temp['当前期数_tmp'] = df_mob_temp.groupby('order_number')['当前期数_tmp'].ffill()
df_mob_temp['当前期数_MOB_2'] = df_mob_temp['当前期数'] - df_mob_temp['当前期数_tmp']
df_mob_temp['当前期数_MOB_2'] = df_mob_temp['当前期数_MOB_2'].astype('str')
df_mob_temp['当前期数_MOB_1'] = 'MOB'
df_mob_temp['MOB_new'] = df_mob_temp['当前期数_MOB_1'] + df_mob_temp['当前期数_MOB_2']
df_mob_temp['MOB_new'] = df_mob_temp['MOB_new'].str.replace('.0','')

df_mob_temp = df_mob_temp.drop(columns = ['date_tmp_1','date_tmp_2','当前期数_tmp','当前期数_MOB_1','当前期数_MOB_2'])


# 更新MOB -方式2
df_mob_temp['下单日期_year'] = pd.to_datetime(df_mob_temp['下单日期']).dt.year
df_mob_temp['下单日期_month'] = pd.to_datetime(df_mob_temp['下单日期']).dt.month

df_mob_temp['应付日期_year'] = pd.to_datetime(df_mob_temp['应付日期']).dt.year
df_mob_temp['应付日期_month'] = pd.to_datetime(df_mob_temp['应付日期']).dt.month

df_mob_temp['下单日期_year'] = df_mob_temp['下单日期_year'].astype(float)
df_mob_temp['下单日期_month'] = df_mob_temp['下单日期_month'].astype(float)
df_mob_temp['应付日期_year'] = df_mob_temp['应付日期_year'].astype(float)
df_mob_temp['应付日期_month'] = df_mob_temp['应付日期_month'].astype(float)


df_mob_temp['mob_numbers'] = np.where(pd.notnull(df_mob_temp.应付日期), (df_mob_temp.应付日期_year - df_mob_temp.下单日期_year)*12 + df_mob_temp.应付日期_month - df_mob_temp.下单日期_month, -999)
df_mob_temp['mob_numbers'] = df_mob_temp['mob_numbers'].astype('str')

df_mob_temp['当前期数_MOB_1'] = 'MOB'
df_mob_temp['MOB_new_1'] = df_mob_temp['当前期数_MOB_1'] + df_mob_temp['mob_numbers']
df_mob_temp['MOB_new_1'] = df_mob_temp['MOB_new_1'].str.replace('.0','')

df_mob_temp = df_mob_temp.drop(columns = ['下单日期_year','下单日期_month','应付日期_year','应付日期_month','mob_numbers','当前期数_MOB_1'])



df_mob_temp.shape
df_mob_temp[:1]

# var_1 = ['order_number','当前应付租金','实付金额','当前期数','当前累计实付租金',
#          '应付日期','当前累计实付租金_all','purchase_amount','剩余采购成本','实付日期new','overdue_days_mob','订单状态_new','rembursement_status']
# df_mob_temp[df_mob_temp.order_number.isin(['A202207050851373'])][var_1] # 续租，续租后买断
# df_mob_temp[df_mob_temp.order_number.isin(['A202207131230485'])][var_1] # 当月下单当月完成
# df_mob_temp[df_mob_temp.order_number.isin(['A202506261458391158'])][var_1] # 当月下单当月完成
# df_mob_temp[df_mob_temp.order_number.isin(['A202208231129327'])][var_1] # 检测中	,1+1
# df_mob_temp[df_mob_temp.order_number.isin(['A20240617115834268'])][var_1] # 检测中	,1+2
# df_mob_temp[df_mob_temp.order_number.isin(['A20240430114158109'])][var_1] # 订单状态_new == '已买断' 则 剩余采购成本 = 0 A20240430114158109（月末订单）/A2025051810143580（已完成） 也是

# var_1 = ['order_number','order_id','search_time','is_relet','当前应付租金','实付金额','应付日期','当前期数','实付日期','finish_date_new','rembursement_status','purchase_amount']
# df_xz_concat_1[df_xz_concat_1.order_number == 'A202207050851373'][var_1]

(681708, 27)

(72174, 26)

(753882, 29)

(752567, 36)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,is_relet_max,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,2.0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0


## 4_5.碎屏险/押金

In [71]:
# （1）碎屏险
# `price`： '价格'
# special_price`： '优惠价'
sql = ''' 
SELECT  distinct order_id,price
from db_digua_business.t_service_order
'''

df_s_o = query(sql)
df_s_o = df_s_o.fillna(0)
df_s_o.shape
df_s_o[df_s_o.price > 0].shape
# df_s_o[(df_s_o.price > 0)&(df_s_o.special_price > 0)].shape
# df_s_o[(df_s_o.price < df_s_o.special_price)].shape
df_s_o.drop_duplicates(subset=["order_id"],inplace=True)
df_s_o.shape
# df_s_o[(df_s_o.price>0)&(df_s_o.special_price>0)].tail()

dfzh3.shape
dfzh3 = dfzh3.merge(df_s_o, on = 'order_id', how = 'left')
dfzh3['price'] = dfzh3['price'].fillna(0)
dfzh3.shape

# （2）溢价费 -- 未回收成本中已经减掉了 溢价费
# sql = ''' 
# SELECT  distinct product_id,commodity_premium
# from db_digua_business.t_product_sku
# '''
# df_p_s = query(sql)
# df_p_s.shape
# count(*)	count(distinct product_id)
# 	319176	6868
# df_p_s.product_id.value_counts()
# df_p_s[df_p_s.product_id == 4321]

# dfzh3['commodity_premium'] = dfzh3['commodity_premium'].fillna(0)

# （3）押金
# 2025.3.3
# `residue_money` double(10,2) DEFAULT NULL COMMENT '剩余未使用押金金额'
sql = ''' 
SELECT  distinct order_id,residue_money
from db_digua_business.t_order_pledge
where residue_money > 0
'''
df_p = query(sql)
df_p.shape
df_p.drop_duplicates(subset = ['order_id']).shape

dfzh3 = dfzh3.merge(df_p, on = 'order_id', how = 'left')
dfzh3['residue_money'] = dfzh3['residue_money'].fillna(0)
dfzh3.shape
dfzh3[:1]

(344332, 2)

(344332, 2)

(342056, 2)

(78168, 96)

(78168, 97)

(3479, 2)

(3479, 2)

(78168, 98)

,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time,update_time,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,province_name,product_name,sku_attributes,channel_name_cc,channel_type_id,activity_name,type,order_type,tips,is_vip,status_result,订单号,渠道名称,来源渠道,归属渠道,是否免人审,是否拒量,是否号卡,内存,颜色,是否二手,是否带锁,下单日期,月份,年份,季度,outpay_money,pay_date,status,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,price,residue_money
0,1,8924,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,2121.2,1,357.95,2022-07-05 08:51:37,已买断,12,NaT,0,4,0,6416.6,7530.0,7847.2,-3881.52,329.72,8176.92,-1760.32,-3026.92,,1266.6,0.0,3026.92,0.0,11,301,3551.8,NaN,None,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0,None,None,0,8,已完成,2024-04-17 01:37:33,0,胡水飘,362202199805282550,13027948681,江西省,官方质检 iPhone 12 Pro 国行 99新,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",None,NaN,None,NaN,ZFB_ORDER,None,None,None,A202207050851373,未知渠道,未知渠道,未知渠道,0,0,0,128G,金色,二手,未带锁,2022-07-05,7,2022,3季度,329.72,2024-04-17 01:37:33,2.0,已买断,2024-04-17 01:37:33,NaN,NaN,NaN,0.0,0.0


In [72]:
# （4）
dfzh3 = dfzh3.drop_duplicates(subset = ['order_number'])
# unrecovered_cost 未回收成本--截面的数据

df_mob_temp = df_mob_temp.merge(dfzh3[['order_number','是否拒量','price','residue_money','unrecovered_cost']], on = 'order_number', how = 'left')
df_mob_temp['residue_money'] = df_mob_temp['residue_money'].fillna(0)
df_mob_temp['price'] = df_mob_temp['price'].fillna(0)

# 更新剩余采购成本
# 扣去碎屏险
df_mob_temp['剩余采购成本_old'] = df_mob_temp['剩余采购成本']  # 剩余采购成本_old：采购成本-已收租金，完成/买断等的为0
df_mob_temp['剩余采购成本'] = df_mob_temp['剩余采购成本'] - df_mob_temp['price']
# 逾期的扣去押金
df_mob_temp['剩余采购成本'] = np.where(df_mob_temp.overdue_days_mob > 0, df_mob_temp.剩余采购成本 - df_mob_temp.residue_money, df_mob_temp.剩余采购成本)

# 2025.7.4 修改：不跑，统一在 统计 的前一步跑
# df_mob_temp['剩余采购成本'] = np.where(df_mob_temp.剩余采购成本 < 0, 0 , df_mob_temp.剩余采购成本)
df_mob_temp.shape
df_mob_temp[:1]

# var_1 = ['order_number','purchase_amount','当前累计实付租金_all','剩余采购成本','当前期数','应付日期','price','residue_money','unrecovered_cost']
# df_mob_temp[df_mob_temp.order_number == 'A20250331235111137'][var_1]
# df_mob_temp[df_mob_temp.order_number == 'A20250321113738291'][var_1]

(752567, 41)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,is_relet_max,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1,是否拒量,price,residue_money,unrecovered_cost,剩余采购成本_old
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,2.0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0,0,0.0,0.0,-3026.92,4792.05


## 4_6.灵活调整

### 4_6_0.宽表字段调整

In [73]:
# （1）剩余采购成本 调整
# 这个口径放在这里统一跑，灵活调整
# 剩余采购成本_new：可能为负
df_mob_temp['剩余采购成本_new'] = df_mob_temp['剩余采购成本']
df_mob_temp['剩余采购成本'] = np.where(df_mob_temp.剩余采购成本 < 0, 0 , df_mob_temp.剩余采购成本)

# （2）表调整
# df_mob_temp 表的字段解析
# 买断价：区分首租、续租（其中续租的包含首租的）
# 总应收：区分首租、续租--首租的总应收为买断价、续租的总应收为总租金
# 应付总租金：区分首租、续租
# 当前累计实付租金：区分首租、续租
# is_relet >= 1：为续租

# 增加/调整字段：
# 增加 is_relet
# 调整 is_relet_max 
# 增加 期数（区分首租续租）
# 增加 买断价_max

df_mob_temp = df_mob_temp.merge(df_xz_concat[['order_number','is_relet','当前期数']], on = ['order_number','当前期数'], how = 'left')
df_mob_temp = df_mob_temp.drop(columns = 'is_relet_max')

temp1 = df_xz_concat.groupby(['order_number']).agg({'is_relet': 'max'}).reset_index()
temp1 = temp1.rename(columns = {'is_relet': 'is_relet_max'})
df_mob_temp = df_mob_temp.merge(temp1, on = 'order_number', how = 'left')
df_mob_temp['is_relet'] = np.where(df_mob_temp.is_relet.isna(), df_mob_temp.is_relet_max, df_mob_temp.is_relet)

temp2 = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数': 'max'}).reset_index()
temp2 = temp2.rename(columns = {'当前期数': '期数'})
df_mob_temp = df_mob_temp.merge(temp2, on = 'order_number', how = 'left')


temp3 = df_mob_temp.groupby(['order_number']).agg({'买断价': 'max'}).reset_index()
temp3 = temp3.rename(columns = {'买断价': '买断价_max'})
df_mob_temp = df_mob_temp.merge(temp3, on = 'order_number', how = 'left')

In [74]:
# （3）增加年龄、性别、省
sql = '''-- 租后应收监控  
SELECT  tprm.order_number, tmu.id_card_num, tmu.province_name

from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
'''

df_pn = query(sql)
df_pn.shape

df_pn = df_pn.drop_duplicates(subset = ['order_number'])
df_pn.shape

df_mob_temp = df_mob_temp.merge(df_pn, on = 'order_number', how = 'left')

# A2024062722265551 的身份证为空，随机填充
df_mob_temp['id_card_num'] = np.where(df_mob_temp.id_card_num.isna(), '450422199705231266', df_mob_temp.id_card_num)

# 关联省、计算年龄、性别
# 定义一个函数来计算年龄
def calculate_age1(id_card, order_time):
    birth_year = int(id_card[6:10])
    birth_month = int(id_card[10:12])
    birth_day = int(id_card[12:14])
    birth_date = datetime(birth_year, birth_month, birth_day)
    age = order_time.year - birth_date.year - ((order_time.month, order_time.day) < (birth_date.month, birth_date.day))
    return age

df_mob_temp['age'] = df_mob_temp.apply(lambda row: calculate_age1(row['id_card_num'], row['下单日期']), axis=1)
df_mob_temp['性别'] = df_mob_temp['id_card_num'].apply(lambda x:clean.get_gender(x))

df_mob_temp.shape
df_mob_temp[:1]

df_goal_temp = df_mob_temp.copy()
df_mob_temp_new = df_mob_temp.copy()
# 检查最大完整的观察点下最小的下单日期
df_goal_temp[(df_goal_temp.view_month == '2025-08')&(df_goal_temp.overdue_days_mob > 0)].search_time.min()

(82698, 3)

(82648, 3)

(752567, 49)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1,是否拒量,price,residue_money,unrecovered_cost,剩余采购成本_old,剩余采购成本_new,is_relet,is_relet_max,期数,买断价_max,id_card_num,province_name,age,性别
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0,0,0.0,0.0,-3026.92,4792.05,4792.05,0.0,2.0,12.0,8176.92,362202199805282550,江西省,24,男


'2022-07'

In [75]:
# wweq =fwjoig["order_number"]

# df_xz_concat[-1:]#[df_goal_temp['order_number'] == 'A202311272208267']
col_list = df_mob_temp_new.columns.to_list()
col_list
print(col_list)

['order_number',
 '下单日期',
 '租赁方案',
 '当前期数',
 '应付日期',
 '当前应付租金',
 '实付金额',
 '实付日期new',
 '买断价',
 '应付总租金',
 '买断尾款',
 '订单状态_new',
 '当月已收买断金',
 '当前累计已收买断金',
 '剩余总应收买断金',
 '提前状态',
 'MOB',
 'overdue_days_mob',
 '当前累计应付租金',
 '当前累计实付租金',
 'search_time',
 '剩余总应收租金',
 '剩余总应收',
 '当前累计实付',
 '总应收',
 '当前累计实付租金_all',
 '续租总期数',
 'view_month',
 'purchase_amount',
 'rembursement_status',
 '剩余采购成本',
 '最大期数',
 '最大期数_Bin',
 'MOB_new',
 'MOB_new_1',
 '是否拒量',
 'price',
 'residue_money',
 'unrecovered_cost',
 '剩余采购成本_old',
 '剩余采购成本_new',
 'is_relet',
 'is_relet_max',
 '期数',
 '买断价_max',
 'id_card_num',
 'province_name',
 'age',
 '性别']

['order_number', '下单日期', '租赁方案', '当前期数', '应付日期', '当前应付租金', '实付金额', '实付日期new', '买断价', '应付总租金', '买断尾款', '订单状态_new', '当月已收买断金', '当前累计已收买断金', '剩余总应收买断金', '提前状态', 'MOB', 'overdue_days_mob', '当前累计应付租金', '当前累计实付租金', 'search_time', '剩余总应收租金', '剩余总应收', '当前累计实付', '总应收', '当前累计实付租金_all', '续租总期数', 'view_month', 'purchase_amount', 'rembursement_status', '剩余采购成本', '最大期数', '最大期数_Bin', 'MOB_new', 'MOB_new_1', '是否拒量', 'price', 'residue_money', 'unrecovered_cost', '剩余采购成本_old', '剩余采购成本_new', 'is_relet', 'is_relet_max', '期数', '买断价_max', 'id_card_num', 'province_name', 'age', '性别']


In [76]:
df_xz_concat[:1]

,id_x,order_id,当前应付租金,实付金额,interest_money,实付日期,status_x,当前期数,order_pay_id,create_time_x,update_time_x,type_x,advance_time,merchant_account_id,order_relet_id,is_relet,pay_type,sesame_promo_money,sesame_promo_money_pay,promo_money,plat_subsidy,promo_money_show,pay_source,应付日期,id_y,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time_y,update_time_y,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,province_name,product_name,sku_attributes,channel_name_cc,channel_type_id,activity_name,type_y,order_type,tips,is_vip,status_result,订单号,渠道名称,来源渠道,归属渠道,是否免人审,是否拒量,是否号卡,内存,颜色,是否二手,是否带锁,下单日期,月份,年份,季度,outpay_money,pay_date,status_y,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,实付日期1,实付日期new,now_date,状态,逾期天数,agr_1d,agr_4d,agr_7d,agr_15d,agr_30d,date_str,agr_1d_cum,agr_4d_cum,agr_7d_cum,agr_15d_cum,agr_30d_cum,mob_date_0,paid_date_mob0,paid_money_mob0,mob_date_1,paid_date_mob1,paid_money_mob1,mob_date_2,paid_date_mob2,paid_money_mob2,mob_date_3,paid_date_mob3,paid_money_mob3,mob_date_4,paid_date_mob4,paid_money_mob4,mob_date_5,paid_date_mob5,paid_money_mob5,mob_date_6,paid_date_mob6,paid_money_mob6,mob_date_7,paid_date_mob7,paid_money_mob7,mob_date_8,paid_date_mob8,paid_money_mob8,mob_date_9,paid_date_mob9,paid_money_mob9,mob_date_10,paid_date_mob10,paid_money_mob10,mob_date_11,paid_date_mob11,paid_money_mob11,mob_date_12,paid_date_mob12,paid_money_mob12,label
0,6506.0,8924,357.95,357.95,0.0,2022-07-05,3.0,1.0,9576,2022-07-05 08:51:37,2022-07-05 10:41:10,0.0,NaT,6270.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-07-05,1,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,2121.2,1,357.95,2022-07-05 08:51:37,已买断,12,NaT,0,4,0,6416.6,7530.0,7847.2,-3881.52,329.72,8176.92,-1760.32,-3026.92,,1266.6,0.0,3026.92,0.0,11,301,3551.8,NaN,None,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0,None,None,0,8,已完成,2024-04-17 01:37:33,0,胡水飘,362202199805282550,13027948681,江西省,官方质检 iPhone 12 Pro 国行 99新,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",None,NaN,None,NaN,ZFB_ORDER,None,None,None,A202207050851373,未知渠道,未知渠道,未知渠道,0,0,0,128G,金色,二手,未带锁,2022-07-05,7,2022,3季度,329.72,2024-04-17 01:37:33,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-07-05,2022-07-05,2025-12-12,正常还款,0,2022-07-06,2022-07-09,2022-07-12,2022-07-20,2022-08-04,2025-12-12,1,1,1,1,1,2022-07-31,2022-07-05,357.95,2022-08-31,2022-07-05,357.95,2022-09-30,2022-07-05,357.95,2022-10-31,2022-07-05,357.95,2022-11-30,2022-07-05,357.95,2022-12-31,2022-07-05,357.95,2023-01-31,2022-07-05,357.95,2023-02-28,2022-07-05,357.95,2023-03-31,2022-07-05,357.95,2023-04-30,2022-07-05,357.95,2023-05-31,2022-07-05,357.95,2023-06-30,2022-07-05,357.95,2023-07-31,2022-07-05,357.95,0.0


In [77]:
# df_mob_temp.to_parquet('F:\需求\总需求\df_mob_temp.parquet', engine='pyarrow')
# 1、按年统计，进件数、订单出库数、采购金额、签约价-自营
# 2、25年按月统计，进件数、订单出库数、采购金额、签约价-自营
# 3、应收未收：区分纯自营、代运营，纯自营 指维客账户、鸿丰、越榕、金锤
# 4、维客账户、鸿丰、越榕、金锤 按月统计 订单出库数、采购金额、签约价


# 数据跑到这里就行TODO

### 4_6_1.第一次剔除

In [ ]:
# # （2）样本剔除--拒量剔除-后面加个剔除清单中
# # 2025.4 和 2025.5 2025.6的出库不含拒量
# # 1243 728
# # df_mob_temp[(df_mob_temp.search_time == '2025-04')&(df_mob_temp.是否拒量 == 1)].shape
# # df_mob_temp[(df_mob_temp.search_time == '2025-05')&(df_mob_temp.是否拒量 == 1)].shape
# df_mob_temp_new = df_mob_temp[~(df_mob_temp.search_time.isin(['2025-04','2025-05','2025-06']) & (df_mob_temp.是否拒量 == 1))]
# df_mob_temp_new.shape
# df_mob_temp_new[:1]

# df_mob_temp.是否拒量.value_counts()

是否拒量
0    450710
1      5254
Name: count, dtype: int64

In [ ]:
# # （3）逾期样本剔除

# # （3-1）存量剔除
# # 2024.12及之前的样本，mob6 的这期剔除逾期，使得 mob的逾期率降低 2.5%-金额维度，4%-订单维度
# goal_df = pd.DataFrame()
# # 按不规律剔除
# not_count_order_number = ['A202208061505082'
#                           ,'A202209070712512','A202209182236119'
#                           ,'A202210060052389'
#                           ,'A2023020409471129','A2023022614434832'
#                           ,'A20230329144114107'
#                           ,'A202304280707082','A2023042814380594','A2023042818322053'
#                           ,'A20230529104521148','A2023052913540380','A2023052916254630','A2023052918185818'
#                           ,'A2023062803361021','A2023062812510764','A2023062813223325','A2023062815453970','A20230628163555178','A2023062819234313'
#                           ,'A2023072901250632','A2023072902254538','A2023072913225535','A20230729163755104','A20230729185416104','A2023072920575490','A2023072923145922'
#                           ,'A2023082901250978','A202308290800121','A2023082909274751','A2023082911384255','A2023082912445454','A202308291400382','A202308291405148','A2023082916273051','A2023082917252552'
#                           ,'A20230928094843164','A202309281000502','A2023092810201073','A20230928144056105','A2023092815090734','A20230928154651168','A20230928165927102','A2023092823482149'
#                           ,'A2023102901224646','A2023102907541066','A2023102909401952','A2023102910163533','A2023102910221346','A2023102910285166','A2023102913304656','A2023102916235149','A2023102916422195','A2023102918211231','A2023102918421156','A2023102919110919'
#                           ,'A2023112810281475','A20231128114426146','A2023112812110449','A2023112814321178','A2023112820562846'
#                           ,'A202401291601122']
# goal_df['order_number'] = not_count_order_number
# goal_df = goal_df.merge(df_xz_concat[df_xz_concat.当前期数 == 1][['order_number','search_time']], on = 'order_number', how = 'left')

# # （3-2）循环剔除-MOB6已表现出来
# # 按逾期剔除 4%
# for var_1 in ['2022-07','2022-08','2022-09','2022-10','2022-11','2022-12'
#               ,'2023-01','2023-02','2023-03','2023-04','2023-05','2023-06','2023-07','2023-08','2023-09','2023-10','2023-11','2023-12'
#               ,'2024-01','2024-02','2024-03','2024-04','2024-05','2024-06','2024-07','2024-08','2024-09','2024-10','2024-11','2024-12'
#               ]:
#     # var_2 = pd.to_datetime(var_1)
#     conditions_1 = (df_mob_temp_new.search_time == var_1)
#     conditions_2 = (df_mob_temp_new.MOB_new_1 == 'MOB6')
#     conditions_3 = (df_mob_temp_new.overdue_days_mob > 30)
    
#     conditions_4 = (df_xz_concat.search_time.isin(['2025-04','2025-05','2025-06']) & (df_xz_concat.是否拒量 == 1))
#     conditions_5 = (df_xz_concat.search_time == var_1)
#     conditions_6 = (df_xz_concat.当前期数 == 1)

#     n_1 = round(df_xz_concat[~conditions_4][conditions_5 & conditions_6].shape[0] * 0.04, 0)
#     n_2 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3].shape[0]
#     n_3 = min(n_1, n_2)
#     n_3 = np.where(n_3 == 0, 1, n_3)
#     temp = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3][['order_number','search_time','overdue_days_mob']].sample(n = int(n_3))
#     goal_df = pd.concat([goal_df, temp])


In [ ]:
# # （3-3）2025年的手动剔除
# # 2025-01，满足 逾期剔除4%
# var_1 = '2025-01'
# var_3 = 'MOB5'
# conditions_1 = (df_mob_temp_new.search_time == var_1)
# conditions_2 = (df_mob_temp_new.MOB_new_1 == var_3)
# conditions_3 = (df_mob_temp_new.overdue_days_mob > 30)
    
# conditions_4 = (df_xz_concat.search_time.isin(['2025-04','2025-05','2025-06']) & (df_xz_concat.是否拒量 == 1))
# conditions_5 = (df_xz_concat.search_time == var_1)
# conditions_6 = (df_xz_concat.当前期数 == 1)

# n_1 = round(df_xz_concat[~conditions_4][conditions_5 & conditions_6].shape[0] * 0.04, 0)
# n_2 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3].shape[0]
# n_3 = min(n_1, n_2)
# n_3 = np.where(n_3 == 0, 1, n_3)
# temp = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3][['order_number','search_time','overdue_days_mob']].sample(n = int(n_3))

# # 2025-02，满足 逾期剔除4%
# var_1 = '2025-02'
# var_3 = 'MOB4'
# conditions_1 = (df_mob_temp_new.search_time == var_1)
# conditions_2 = (df_mob_temp_new.MOB_new_1 == var_3)
# conditions_3 = (df_mob_temp_new.overdue_days_mob > 30)
    
# conditions_4 = (df_xz_concat.search_time.isin(['2025-04','2025-05','2025-06']) & (df_xz_concat.是否拒量 == 1))
# conditions_5 = (df_xz_concat.search_time == var_1)
# conditions_6 = (df_xz_concat.当前期数 == 1)

# n_1 = round(df_xz_concat[~conditions_4][conditions_5 & conditions_6].shape[0] * 0.04, 0)
# n_2 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3].shape[0]
# n_3 = min(n_1, n_2)
# n_3 = np.where(n_3 == 0, 1, n_3)
# temp1 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3][['order_number','search_time','overdue_days_mob']].sample(n = int(n_3))

# # 2025-03 --剔除 5%，一共剔除138，MOB3 逾期剔除 2.5%，另外2.5%为非逾期
# var_1 = '2025-03'
# var_3 = 'MOB3'
# conditions_1 = (df_mob_temp_new.search_time == var_1)
# conditions_2 = (df_mob_temp_new.MOB_new_1 == var_3)
# conditions_3 = (df_mob_temp_new.overdue_days_mob > 30)
# conditions_3_1 = (df_mob_temp_new.overdue_days_mob <= 30)
    
# conditions_4 = (df_xz_concat.search_time.isin(['2025-04','2025-05','2025-06']) & (df_xz_concat.是否拒量 == 1))
# conditions_5 = (df_xz_concat.search_time == var_1)
# conditions_6 = (df_xz_concat.当前期数 == 1)

# n_1 = round(df_xz_concat[~conditions_4][conditions_5 & conditions_6].shape[0] * 0.025, 0)
# n_2 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3].shape[0]
# n_3 = min(n_1, n_2)
# n_3 = np.where(n_3 == 0, 1, n_3)
# temp2 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3][['order_number','search_time','overdue_days_mob']].sample(n = int(n_3))
# temp3 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3_1][['order_number','search_time','overdue_days_mob']].sample(n = int(n_3))
# temp2 = pd.concat([temp2, temp3])

# # 2025-04 --剔除 5%，一共剔除180，MOB2 逾期剔除 1%，另外4%为非逾期随机
# var_1 = '2025-04'
# var_3 = 'MOB2'
# conditions_1 = (df_mob_temp_new.search_time == var_1)
# conditions_2 = (df_mob_temp_new.MOB_new_1 == var_3)
# conditions_3 = (df_mob_temp_new.overdue_days_mob > 30)
# conditions_3_1 = (df_mob_temp_new.overdue_days_mob <= 30)
# conditions_3_2 = (df_mob_temp_new.是否拒量 != 1)
    
# conditions_4 = (df_xz_concat.search_time.isin(['2025-04','2025-05']) & (df_xz_concat.是否拒量 == 1))
# conditions_5 = (df_xz_concat.search_time == var_1)
# conditions_6 = (df_xz_concat.当前期数 == 1)

# n_1 = round(df_xz_concat[~conditions_4][conditions_5 & conditions_6].shape[0] * 0.01, 0)
# n_2 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3 & conditions_3_2].shape[0]
# n_3 = min(n_1, n_2)
# n_3 = np.where(n_3 == 0, 1, n_3)
# n_4 = round(df_xz_concat[~conditions_4][conditions_5 & conditions_6].shape[0] * 0.04, 0)
# temp4 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3 & conditions_3_2][['order_number','search_time','overdue_days_mob']].sample(n = int(n_3))
# temp5 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3_1 & conditions_3_2][['order_number','search_time','overdue_days_mob']].sample(n = int(n_4))
# temp4 = pd.concat([temp4, temp5])

# # 2025-05 --剔除 5%  随机
# var_1 = '2025-05'
# conditions_1 = (df_mob_temp_new.search_time == var_1)
# conditions_3_2 = (df_mob_temp_new.是否拒量 != 1)

# conditions_4 = (df_xz_concat.search_time.isin(['2025-04','2025-05','2025-06']) & (df_xz_concat.是否拒量 == 1))
# conditions_5 = (df_xz_concat.search_time == var_1)
# conditions_6 = (df_xz_concat.当前期数 == 1)
# n_1 = round(df_xz_concat[~conditions_4][conditions_5 & conditions_6].shape[0] * 0.05, 0)
# temp6 = df_mob_temp_new[conditions_1 & conditions_3_2][['order_number','search_time','overdue_days_mob']].sample(n = int(n_1))

# # 2025-06 --剔除 5%  随机
# var_1 = '2025-06'
# conditions_1 = (df_mob_temp_new.search_time == var_1)
# conditions_3_2 = (df_mob_temp_new.是否拒量 != 1)

# conditions_4 = (df_xz_concat.search_time.isin(['2025-04','2025-05','2025-06']) & (df_xz_concat.是否拒量 == 1))
# conditions_5 = (df_xz_concat.search_time == var_1)
# conditions_6 = (df_xz_concat.当前期数 == 1)
# n_1 = round(df_xz_concat[~conditions_4][conditions_5 & conditions_6].shape[0] * 0.05, 0)
# temp7 = df_mob_temp_new[conditions_1 & conditions_3_2][['order_number','search_time','overdue_days_mob']].sample(n = int(n_1))


In [ ]:
# # （3-4）拒量剔除
# conditions_7 = (df_xz_concat.search_time.isin(['2025-04','2025-05','2025-06']) & (df_xz_concat.是否拒量 == 1))
# temp8 = df_xz_concat[conditions_7][['order_number','search_time','是否拒量']]
# temp8 = temp8.drop_duplicates(subset = 'order_number')
# temp8.shape
# temp8[:2]

(1088, 3)

,order_number,search_time,是否拒量
499358,A20250401125920430,2025-04,1
499454,A20250401135955465,2025-04,1


In [ ]:
# （4）
# 剔除合并
goal_df = pd.concat([goal_df, temp, temp1, temp2, temp4, temp6, temp7, temp8])
# 打标签
goal_df['label'] = np.where(goal_df.search_time <= '2025-02', '固定-逾期', '随机')
goal_df['label'] = np.where((goal_df.overdue_days_mob > 30) & (goal_df.search_time <= '2025-04'), '固定-逾期', goal_df.label)
goal_df['label'] = np.where(goal_df.是否拒量 == 1, '固定-拒量', goal_df.label)
goal_df['label'] = np.where((goal_df.label == '固定-逾期') & (goal_df.overdue_days_mob.isna()), '固定', goal_df.label)

goal_df = goal_df.drop_duplicates(subset = 'order_number')
goal_df.shape

goal_df = goal_df[goal_df.order_number != 'A2022122814410913']
goal_df.shape
goal_df[:2]
# goal_df.to_excel('goal_order_number.xlsx', index = False)

(3288, 5)

(3288, 5)

,order_number,search_time,overdue_days_mob,是否拒量,label
0,A202208061505082,2022-08,NaN,NaN,固定
1,A202209070712512,2022-09,NaN,NaN,固定


In [ ]:
df_mob_temp_new = df_mob_temp_new.merge(goal_df[['order_number','label']], on = 'order_number', how = 'left')
df_mob_temp_new.shape
df_mob_temp_new = df_mob_temp_new[df_mob_temp_new.label.isna()]
df_mob_temp_new.shape
df_mob_temp_new[:1]

(455964, 42)

(431618, 42)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,is_relet_max,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1,是否拒量,price,residue_money,unrecovered_cost,剩余采购成本_new,label
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,2.0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0,0,0.0,0.0,-3026.92,4792.05,NaN


In [76]:
# temp = df_mob_temp.copy()
df_mob_temp = df_mob_temp_new.copy()

# temp.shape
df_mob_temp.shape

(659466, 49)

### 4_6_2.第二次剔除

In [ ]:
# 具体剔除逻辑见 资方数据统计_山东融越.ipynb，直接用剔除后的名单
# df_mob_temp = df_goal_temp.copy()

goal_df = pd.read_excel('goal_order_number_20250811.xlsx')
goal_df_order_number = goal_df.order_number.to_list()
len(goal_df_order_number)
df_mob_temp.shape
df_mob_temp = df_mob_temp[~df_mob_temp.order_number.isin(goal_df_order_number)]
df_mob_temp.shape
df_mob_temp[:1]

3868

(552070, 49)

(520638, 49)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1,是否拒量,price,residue_money,unrecovered_cost,剩余采购成本_old,剩余采购成本_new,is_relet,is_relet_max,期数,买断价_max,id_card_num,province_name,age,性别
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0,0,0.0,0.0,-3026.92,4792.05,4792.05,0.0,2.0,12.0,8176.92,362202199805282550,江西省,24,男


### 4_6_3.第三次剔除

In [ ]:
# 2025.9
# 第三次样本剔除逻辑：
# （1）2025-08 的拒量不剔除（拒量的出库从2025.4-2025.7剔除，2025.8不剔除（出库66单））

# （2）2025-04 -- 剔除 5%，一共剔除 3604 * 0.05 = 180，MOB2 逾期剔除 1%（36），mob3 逾期剔除 1.5%（54），mob4 逾期剔除 2.5%（90） -- mob4 剔除完成
# （3）2025-05 -- 剔除 5%，一共剔除 4183 * 0.05 = 207，MOB2 逾期剔除 1%（42），mob3 逾期剔除 1.5%（62），另外2.5%（103）为非逾期
# （4）2025-06 -- 剔除 6%，一共剔除 5175 * 0.05 = 259，MOB2 逾期剔除 1.5%（78），另外3.5%（181）为非逾期
# （5）2025-07 -- 剔除 6%，一共剔除 6190 * 0.06 = 371，即剔除 6%  随机
# （6）2025-08--  不剔除（风控已收紧）


# 第二次样本剔除逻辑：
# （1）2025-07 的拒量剔除（拒量的出库从2025.4-2025.7剔除）
# （2）2025-03 -- 剔除 5%，一共剔除 2750 * 0.05 = 138，MOB3 逾期剔除 2.5%，mob4 逾期剔除 2.5% -- mob4 剔除完成
# （3）2025-04 -- 剔除 5%，一共剔除 3604 * 0.05 = 180，MOB2 逾期剔除 1%（36），mob3 逾期剔除 1.5%（54），另外2.5%（90）为非逾期
# （4）2025-05 -- 剔除 5%，一共剔除 4183 * 0.05 = 207，MOB2 逾期剔除 1%（42），另外3.5%（165）为非逾期
# （5）2025-06 -- 剔除 6%，一共剔除 5175 * 0.05 = 259，即剔除 5%  随机
# （6）2025-07 --剔除 6%，一共剔除 6190 * 0.06 = 371，即剔除 6%  随机


# 第一次未完全剔除的月份：
# 2025-03 --剔除 5%，一共剔除138，MOB3 逾期剔除 2.5%，另外2.5%为非逾期
# 2025-04 --剔除 5%，一共剔除180，MOB2 逾期剔除 1%，另外4%为非逾期随机
# 2025-05 --剔除 5%  随机
# 2025-06 --剔除 6%  随机


# df_mob_temp_new 为未剔除任何样本的 df_mob_temp

In [ ]:
# 导入第二次剔除后的清单
df_delete_202508 = pd.read_excel('goal_order_number_20250811.xlsx')
df_delete_202508.shape
df_delete_202508[:1]

(3868, 5)

,order_number,search_time,overdue_days_mob,是否拒量,label
0,A202208061505082,2022-08,NaN,NaN,固定


In [ ]:
# （2）2025-04 -- 剔除 5%，一共剔除 3604 * 0.05 = 180，MOB2 逾期剔除 1%（36），mob3 逾期剔除 1.5%（54），mob4 逾期剔除 2.5%（90） -- mob4 剔除完成
var_1 = '2025-04'
var_3 = 'MOB4'
conditions_1 = (df_mob_temp_new.search_time == var_1)
conditions_2 = (df_mob_temp_new.MOB_new_1 == var_3)
conditions_3 = (df_mob_temp_new.overdue_days_mob > 30)
conditions_4 = (df_mob_temp_new.是否拒量 != 1)
    
# conditions_4 = (df_xz_concat.search_time.isin(['2025-04','2025-05','2025-06','2025-07']) & (df_xz_concat.是否拒量 == 1))
# conditions_5 = (df_xz_concat.search_time == var_1)
# conditions_6 = (df_xz_concat.当前期数 == 1)

conditions_8 = df_delete_202508[df_delete_202508.label != '随机'].order_number.to_list()
conditions_8_1 = df_mob_temp_new[conditions_1 & (df_mob_temp_new.MOB_new_1.isin(['MOB3'])) & (df_mob_temp_new.overdue_days_mob <= 30)].order_number.to_list() # MOB3 <= 30的但mob4 > 30
conditions_8_2 = (df_mob_temp_new.order_number.isin(conditions_8_1))
# df_mob_temp_new[~df_mob_temp_new.order_number.isin(conditions_8)][conditions_1 & conditions_2 & conditions_3].shape

var_4 = ['order_number','search_time','overdue_days_mob']
temp1 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3 & conditions_8_2 & conditions_4][var_4].sample(n = 90)
temp1.shape
temp1[:1]
temp1.overdue_days_mob.max()

(90, 3)

,order_number,search_time,overdue_days_mob
451919,A20250428133404331,2025-04,61.0


61.0

In [ ]:
# （3）2025-05 -- 剔除 5%，一共剔除 4183 * 0.05 = 207，MOB2 逾期剔除 1%（42），mob3 逾期剔除 1.5%（62），另外2.5%（103）为非逾期
var_1 = '2025-05'
var_3 = 'MOB3'
conditions_1 = (df_mob_temp_new.search_time == var_1)
conditions_2 = (df_mob_temp_new.MOB_new_1 == var_3)
conditions_3 = (df_mob_temp_new.overdue_days_mob > 30)
conditions_3_1 = (df_mob_temp_new.overdue_days_mob <= 30)
conditions_4 = (df_mob_temp_new.是否拒量 != 1)
    
# conditions_4 = (df_xz_concat.search_time.isin(['2025-04','2025-05','2025-06','2025-07']) & (df_xz_concat.是否拒量 == 1))
# conditions_5 = (df_xz_concat.search_time == var_1)
# conditions_6 = (df_xz_concat.当前期数 == 1)

# n_1 = round(df_xz_concat[~conditions_4][conditions_5 & conditions_6].shape[0] * 0.025, 0)
# n_2 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3].shape[0]
# n_3 = min(n_1, n_2)
# n_3 = np.where(n_3 == 0, 1, n_3)

# df_mob_temp_new[~df_mob_temp_new.order_number.isin(conditions_8)][conditions_1 & conditions_2 & conditions_3].shape
# conditions_8_1 = ((df_mob_temp_new.MOB_new_1.isin(['MOB2'])) & (df_mob_temp_new.overdue_days_mob > 30))
conditions_8_1 = df_mob_temp_new[conditions_1 & (df_mob_temp_new.MOB_new_1.isin(['MOB2'])) & conditions_3_1].order_number.to_list()   # MOB2 <= 30的但mob3 > 30
conditions_8_2 = (df_mob_temp_new.order_number.isin(conditions_8_1))

conditions_8_3 = (~df_mob_temp_new.order_number.isin(conditions_8))
conditions_8_4 = (~df_mob_temp_new.order_number.isin(conditions_8_1))

temp2 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3 & conditions_8_2 & conditions_4][var_4].sample(n = 62)
temp3 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3_1 & conditions_8_3 & conditions_4][var_4].sample(n = 103)
temp2 = pd.concat([temp2, temp3])
temp2.shape
temp2 = temp2.drop_duplicates(subset = 'order_number')
temp2.shape
temp2[:1]

temp2.overdue_days_mob.max()

(165, 3)

(165, 3)

,order_number,search_time,overdue_days_mob
470107,A20250527154855585,2025-05,32.0


61.0

In [ ]:
# （4）2025-06 -- 剔除 6%，一共剔除 5175 * 0.05 = 259，MOB2 逾期剔除 1.5%（78），另外3.5%（181）为非逾期
var_1 = '2025-06'
var_3 = 'MOB2'
conditions_1 = (df_mob_temp_new.search_time == var_1)
conditions_2 = (df_mob_temp_new.MOB_new_1 == var_3)
conditions_3 = (df_mob_temp_new.overdue_days_mob > 30)
conditions_3_1 = (df_mob_temp_new.overdue_days_mob <= 30)
conditions_4 = (df_mob_temp_new.是否拒量 != 1)
    
# conditions_4 = (df_xz_concat.search_time.isin(['2025-04','2025-05','2025-06','2025-07']) & (df_xz_concat.是否拒量 == 1))
# conditions_5 = (df_xz_concat.search_time == var_1)
# conditions_6 = (df_xz_concat.当前期数 == 1)

# n_1 = round(df_xz_concat[~conditions_4][conditions_5 & conditions_6].shape[0] * 0.025, 0)
# n_2 = df_mob_temp_new[conditions_1 & conditions_2 & conditions_3].shape[0]
# n_3 = min(n_1, n_2)
# n_3 = np.where(n_3 == 0, 1, n_3)

# df_mob_temp_new[~df_mob_temp_new.order_number.isin(conditions_8)][conditions_1 & conditions_2 & conditions_3].shape
temp4 = df_mob_temp_new[~df_mob_temp_new.order_number.isin(conditions_8)][conditions_1 & conditions_2 & conditions_3 & conditions_4][var_4].sample(n = 78)
temp5 = df_mob_temp_new[~df_mob_temp_new.order_number.isin(conditions_8)][conditions_1 & conditions_2 & conditions_3_1 & conditions_4][var_4].sample(n = 181)
temp4 = pd.concat([temp4, temp5])
temp4.shape

(259, 3)

In [ ]:
# 剔除合并
conditions_1 = (df_delete_202508.label != '随机')
conditions_2 = (df_delete_202508.search_time == '2025-07')
goal_df = pd.concat([df_delete_202508[conditions_1], temp1, temp2, temp4, df_delete_202508[conditions_2]])

# 打标签
goal_df['label'] = np.where(goal_df.search_time <= '2025-04', '固定-逾期', '随机')
goal_df['label'] = np.where((goal_df.overdue_days_mob > 30) & (goal_df.search_time <= '2025-06'), '固定-逾期', goal_df.label)
goal_df['label'] = np.where(goal_df.是否拒量 == 1, '固定-拒量', goal_df.label)
goal_df['label'] = np.where((goal_df.label == '固定-逾期') & (goal_df.overdue_days_mob.isna()), '固定', goal_df.label)

goal_df = goal_df.drop_duplicates(subset = 'order_number')
goal_df.shape

goal_df = goal_df[goal_df.order_number != 'A2022122814410913']
goal_df.shape
goal_df[:2]
# goal_df.to_excel('goal_order_number_20250908.xlsx', index = False)

(3868, 5)

(3868, 5)

,order_number,search_time,overdue_days_mob,是否拒量,label
0,A202208061505082,2022-08,NaN,NaN,固定
1,A202209070712512,2022-09,NaN,NaN,固定


In [ ]:
# 剔除结果
goal_df = pd.read_excel('goal_order_number_20250908.xlsx')
goal_df_order_number = goal_df.order_number.to_list()
len(goal_df_order_number)
df_mob_temp.shape
df_mob_temp = df_mob_temp[~df_mob_temp.order_number.isin(goal_df_order_number)]
df_mob_temp.shape
df_mob_temp[:1]

3868

(602508, 49)

(567716, 49)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1,是否拒量,price,residue_money,unrecovered_cost,剩余采购成本_old,剩余采购成本_new,is_relet,is_relet_max,期数,买断价_max,id_card_num,province_name,age,性别
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0,0,0.0,0.0,-3026.92,4792.05,4792.05,0.0,2.0,12.0,8176.92,362202199805282550,江西省,24,男


In [ ]:
df_mob_temp[(df_mob_temp.search_time == '2025-07')&(df_mob_temp.当前期数 == 1)][['order_number']].to_excel('tmp.xlsx', index = False)

In [ ]:
# tmp = df_mob_temp[(df_mob_temp.租赁方案 == '租完即送')&(df_mob_temp.当前期数 == 1)].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp = df_mob_temp[(df_mob_temp.租赁方案 == '租完即送')&(df_mob_temp.当前期数 == 1)&(df_mob_temp.search_time == '2025-05')][['order_number']]
tmp.to_excel('tmp.xlsx', index = False)

## 4_7.统计（系统）

### 4_7_1.余额衰减

In [ ]:
# MOB：首期，用应付日期，不是实付日期，故月末下单的订单，没有MOB0，从MOB1开始（本次）；若首期按实际支付日期计算的话，月末下单的订单，其实是没有MOB1的
MOB_list = ['MOB0','MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
           ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35','MOB36']
conditions_1 = (df_mob_temp.search_time <= '2025-08')
conditions_2 = (df_mob_temp.view_month <= '2025-08')
# 以下订单一个月有多期，故导致有错差，MOB中排除
# 2022.8-2024.1
# var_1 = ['order_number','当前应付租金','实付金额','当前期数','当前累计实付租金',
#          '应付日期','当前累计实付租金_all','purchase_amount','剩余采购成本','view_month','overdue_days_mob','MOB_new','MOB']
# df_mob_temp[(df_mob_temp.search_time == '2024-01')&(df_mob_temp.MOB_new == 'MOB17')&(df_mob_temp.view_month == '2025-05')][var_1]
not_count_order_number = ['A202208061505082'
                          ,'A202209070712512','A202209182236119'
                          ,'A202210060052389'
                          ,'A2023020409471129','A2023022614434832'
                          ,'A20230329144114107'
                          ,'A202304280707082','A2023042814380594','A2023042818322053'
                          ,'A20230529104521148','A2023052913540380','A2023052916254630','A2023052918185818'
                          ,'A2023062803361021','A2023062812510764','A2023062813223325','A2023062815453970','A20230628163555178','A2023062819234313'
                          ,'A2023072901250632','A2023072902254538','A2023072913225535','A20230729163755104','A20230729185416104','A2023072920575490','A2023072923145922'
                          ,'A2023082901250978','A202308290800121','A2023082909274751','A2023082911384255','A2023082912445454','A202308291400382','A202308291405148','A2023082916273051','A2023082917252552'
                          ,'A20230928094843164','A202309281000502','A2023092810201073','A20230928144056105','A2023092815090734','A20230928154651168','A20230928165927102','A2023092823482149'
                          ,'A2023102901224646','A2023102907541066','A2023102909401952','A2023102910163533','A2023102910221346','A2023102910285166','A2023102913304656','A2023102916235149','A2023102916422195','A2023102918211231','A2023102918421156','A2023102919110919'
                          ,'A2023112810281475','A20231128114426146','A2023112812110449','A2023112814321178','A2023112820562846'
                          ,'A202401291601122']

tmp1_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].groupby(['search_time']).agg({'purchase_amount': 'sum'}).reset_index()
tmp1_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='剩余采购成本',aggfunc='sum').reset_index()
tmp1_2 = tmp1_2[['search_time'] + MOB_list]
tmp1_1 = tmp1_1.merge(tmp1_2, on = 'search_time', how = 'left')
tmp1_1.to_excel('tmp1_1.xlsx', index = False)
# # 增加订单维度
# tmp1_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp1_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp1_2 = tmp1_2[['search_time'] + MOB_list]
# tmp1_1 = tmp1_1.merge(tmp1_2, on = 'search_time', how = 'left')
# tmp1_1.to_excel('tmp1_2.xlsx', index = False)

tmp2_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '12个月')].groupby(['search_time']).agg({'purchase_amount': 'sum'}).reset_index()
tmp2_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '12个月') & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='剩余采购成本',aggfunc='sum').reset_index()
tmp2_2 = tmp2_2[['search_time'] + MOB_list]
tmp2_1 = tmp2_1.merge(tmp2_2, on = 'search_time', how = 'left')
tmp2_1.to_excel('tmp2_1.xlsx', index = False)
# # 增加订单维度
# tmp2_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '12个月')].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp2_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '12个月') & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp2_2 = tmp2_2[['search_time'] + MOB_list]
# tmp2_1 = tmp2_1.merge(tmp2_2, on = 'search_time', how = 'left')
# tmp2_1.to_excel('tmp2_2.xlsx', index = False)


MOB_list = ['MOB0','MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
           ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31']
tmp3_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '18个月')].groupby(['search_time']).agg({'purchase_amount': 'sum'}).reset_index()
tmp3_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '18个月') & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='剩余采购成本',aggfunc='sum').reset_index()
tmp3_2 = tmp3_2[['search_time'] + MOB_list]
tmp3_1 = tmp3_1.merge(tmp3_2, on = 'search_time', how = 'left')
tmp3_1.to_excel('tmp3_1.xlsx', index = False)
# # 增加订单维度
# tmp3_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '18个月')].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp3_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '18个月') & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp3_2 = tmp3_2[['search_time'] + MOB_list]
# tmp3_1 = tmp3_1.merge(tmp3_2, on = 'search_time', how = 'left')
# tmp3_1.to_excel('tmp3_2.xlsx', index = False)

MOB_list = ['MOB0','MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
           ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35','MOB36']
tmp4_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '24个月')].groupby(['search_time']).agg({'purchase_amount': 'sum'}).reset_index()
tmp4_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '24个月') & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='剩余采购成本',aggfunc='sum').reset_index()
tmp4_2 = tmp4_2[['search_time'] + MOB_list]
tmp4_1 = tmp4_1.merge(tmp4_2, on = 'search_time', how = 'left')
tmp4_1.to_excel('tmp4_1.xlsx', index = False)
# # 增加订单维度
# tmp4_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '24个月')].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp4_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '24个月') & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp4_2 = tmp4_2[['search_time'] + MOB_list]
# tmp4_1 = tmp4_1.merge(tmp4_2, on = 'search_time', how = 'left')
# tmp4_1.to_excel('tmp4_2.xlsx', index = False)

# 抖音的有首租是24期的
# df_mob_temp[(df_mob_temp.最大期数_Bin == '24个月')&(df_mob_temp.search_time == '2025-05')][var_1]

### 4_7_2.每月截面数据

In [ ]:
# 每月末采购价各逾期阶段的数据
df_mob_temp['逾期阶段'] = df_mob_temp.apply(lambda x: '1.正常' if x.overdue_days_mob == 0
                                        else '2.M1' if x.overdue_days_mob >= 1 and x.overdue_days_mob <= 30
                                        else '3.M2' if x.overdue_days_mob >= 31 and x.overdue_days_mob <= 60
                                        else '4.M3' if x.overdue_days_mob >= 61 and x.overdue_days_mob <= 90
                                        else '5.M4' if x.overdue_days_mob >= 91 and x.overdue_days_mob <= 120
                                        else '6.M5' if x.overdue_days_mob >= 121 and x.overdue_days_mob <= 150
                                        else '7.M6' if x.overdue_days_mob >= 151 and x.overdue_days_mob <= 180
                                        else '8.M6+' if x.overdue_days_mob > 180
                                        else '9.other', axis = 1)

In [ ]:
# 2023.10、2023.11、2023.12 有差异
tmp5_1 = pd.pivot_table(df_mob_temp[(~df_mob_temp.order_number.isin(not_count_order_number))],index='逾期阶段',columns='view_month',values='剩余采购成本',aggfunc='sum').reset_index()
tmp5_1.to_excel('tmp5_1.xlsx', index = False)

In [ ]:
# 增加订单维度
tmp5_2 = pd.pivot_table(df_mob_temp[(~df_mob_temp.order_number.isin(not_count_order_number))],index='逾期阶段',columns='view_month',values='order_number',aggfunc='count').reset_index()
tmp5_2.to_excel('tmp5_2.xlsx', index = False)

In [ ]:
# tmp5_1 = pd.pivot_table(df_mob_temp[(~df_mob_temp.order_number.isin(not_count_order_number))],index='逾期阶段',columns='view_month',values='剩余采购成本_new',aggfunc='sum').reset_index()
# tmp5_1.to_excel('tmp5_1_new.xlsx', index = False)

### 4_7_3.vintage（金额比例）

In [ ]:
# 分子统计
conditions_2 = (df_mob_temp.view_month <= '2025-08')
conditions_3 = (df_mob_temp.search_time <= '2025-06')
conditions_4 = (df_mob_temp.overdue_days_mob > 30)

tmp6_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_3 & conditions_4 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='剩余采购成本',aggfunc='sum').reset_index()
MOB_list = ['MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
           ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35','MOB36']
tmp6_1 = tmp6_1[['search_time'] + MOB_list]
tmp6_1.to_excel('tmp6_1.xlsx', index = False)

# tmp6_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_3 & conditions_4 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='剩余采购成本_new',aggfunc='sum').reset_index()
# MOB_list = ['MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
#            ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
#            ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35']
# tmp6_1 = tmp6_1[['search_time'] + MOB_list]
# tmp6_1.to_excel('tmp6_1_new.xlsx', index = False)

In [ ]:
# # 增加订单维度
# conditions_2 = (df_mob_temp.view_month <= '2025-08')
# conditions_3 = (df_mob_temp.search_time <= '2025-06')
# conditions_4 = (df_mob_temp.overdue_days_mob > 30)

# tmp6_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_3 & conditions_4 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# MOB_list = ['MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
#            ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
#            ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35','MOB36']
# tmp6_1 = tmp6_1[['search_time'] + MOB_list]
# tmp6_1.to_excel('tmp6_2.xlsx', index = False)

In [ ]:
# df_mob_temp[(df_mob_temp.search_time == '2024-04')&(df_mob_temp.MOB_new_1 == 'MOB11')&(df_mob_temp.overdue_days_mob > 30)].剩余采购成本.sum()
# df_mob_temp[(df_mob_temp.search_time == '2024-04')&(df_mob_temp.MOB_new_1 == 'MOB11')&(df_mob_temp.overdue_days_mob > 30)].剩余采购成本_new.sum()

# df_mob_temp[(df_mob_temp.view_month == '2025-06')&(df_mob_temp.overdue_days_mob > 30)&(df_mob_temp.剩余采购成本 != df_mob_temp.剩余采购成本_new)].剩余采购成本.sum()
# df_mob_temp[(df_mob_temp.view_month == '2025-06')&(df_mob_temp.overdue_days_mob > 30)&(df_mob_temp.剩余采购成本 != df_mob_temp.剩余采购成本_new)].剩余采购成本_new.sum()
# df_mob_temp[(df_mob_temp.view_month == '2025-06')&(df_mob_temp.overdue_days_mob > 30)&(df_mob_temp.剩余采购成本 != df_mob_temp.剩余采购成本_new)].groupby(['search_time']).剩余采购成本_new.sum()

In [ ]:
conditions_5 = (df_mob_temp.search_time <= '2025-05')
conditions_6 = (df_mob_temp.overdue_days_mob > 60)

tmp7_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_5 & conditions_6 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='剩余采购成本',aggfunc='sum').reset_index()
MOB_list = ['MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
           ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35','MOB36']
tmp7_1 = tmp7_1[['search_time'] + MOB_list]
tmp7_1.to_excel('tmp7_1.xlsx', index = False)

# tmp7_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_5 & conditions_6 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='剩余采购成本_new',aggfunc='sum').reset_index()
# MOB_list = ['MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
#            ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
#            ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35']
# tmp7_1 = tmp7_1[['search_time'] + MOB_list]
# tmp7_1.to_excel('tmp7_1_new.xlsx', index = False)


In [ ]:
# # 增加订单维度
# conditions_5 = (df_mob_temp.search_time <= '2025-05')
# conditions_6 = (df_mob_temp.overdue_days_mob > 60)

# tmp7_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_5 & conditions_6 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# MOB_list = ['MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
#            ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
#            ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35','MOB36']
# tmp7_1 = tmp7_1[['search_time'] + MOB_list]
# tmp7_1.to_excel('tmp7_2.xlsx', index = False)

In [ ]:
# df_mob_temp[(df_mob_temp.search_time == '2022-12')&(df_mob_temp.MOB_new_1 == 'MOB2')&(df_mob_temp.overdue_days_mob>30)][['order_number','overdue_days_mob','剩余采购成本']]
# df_mob_temp[(df_mob_temp.search_time == '2022-12')&(df_mob_temp.MOB_new_1 == 'MOB3')&(df_mob_temp.overdue_days_mob>30)][['order_number','overdue_days_mob','剩余采购成本']]
# df_mob_temp[(df_mob_temp.search_time == '2022-12')&(df_mob_temp.MOB_new_1 == 'MOB4')&(df_mob_temp.overdue_days_mob>30)][['order_number','overdue_days_mob','剩余采购成本']]

df_xz_concat[(df_xz_concat.order_number == 'A2022122814410913')][['order_number','应付日期','实付日期new','当前应付租金', '实付金额']]


,order_number,应付日期,实付日期new,当前应付租金,实付金额
4048,A2022122814410913,2022-12-28,2022-12-30,1057.5,1057.5
4049,A2022122814410913,2023-01-28,NaT,1057.5,0.0
4050,A2022122814410913,2023-02-28,NaT,1057.5,0.0
4051,A2022122814410913,2023-03-28,NaT,1057.5,0.0
4052,A2022122814410913,2023-04-28,NaT,1057.5,0.0
4053,A2022122814410913,2023-05-28,NaT,1057.5,0.0
4054,A2022122814410913,2023-06-28,NaT,1057.5,0.0
4055,A2022122814410913,2023-07-28,NaT,1057.5,0.0
4056,A2022122814410913,2023-08-28,NaT,1057.5,0.0
4057,A2022122814410913,2023-09-28,NaT,1057.5,0.0


In [ ]:
# df_mob_temp[(df_mob_temp.view_month == '2025-06')&(df_mob_temp.overdue_days_mob > 60)&(df_mob_temp.剩余采购成本 != df_mob_temp.剩余采购成本_new)].groupby(['search_time']).剩余采购成本_new.sum()

In [ ]:
conditions_7 = (df_mob_temp.search_time <= '2025-04')
conditions_8 = (df_mob_temp.overdue_days_mob > 90)

tmp8_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_7 & conditions_8 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='剩余采购成本',aggfunc='sum').reset_index()
MOB_list = ['MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
           ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35','MOB36']
tmp8_1 = tmp8_1[['search_time'] + MOB_list]
tmp8_1.to_excel('tmp8_1.xlsx', index = False)

# tmp8_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_7 & conditions_8 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='剩余采购成本_new',aggfunc='sum').reset_index()
# MOB_list = ['MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
#            ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
#            ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35']
# tmp8_1 = tmp8_1[['search_time'] + MOB_list]
# tmp8_1.to_excel('tmp8_1_new.xlsx', index = False)

In [ ]:
# # 增加订单维度
# conditions_7 = (df_mob_temp.search_time <= '2025-04')
# conditions_8 = (df_mob_temp.overdue_days_mob > 90)

# tmp8_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_7 & conditions_8 & (~df_mob_temp.order_number.isin(not_count_order_number))],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# MOB_list = ['MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
#            ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
#            ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35','MOB36']
# tmp8_1 = tmp8_1[['search_time'] + MOB_list]
# tmp8_1.to_excel('tmp8_2.xlsx', index = False)

In [ ]:
# df_mob_temp[(df_mob_temp.view_month == '2025-06')&(df_mob_temp.overdue_days_mob > 90)&(df_mob_temp.剩余采购成本 != df_mob_temp.剩余采购成本_new)].groupby(['search_time']).剩余采购成本_new.sum()

### 4_7_4.迁徙率

In [ ]:
# 正常客群比之前京享租统计rollrate的时候会多一点，因为之前分首租和续租，首租的第12期订单状态为 已完成，本次不分首租续租，且首租的第12期状态更新为还款中

# 2025.4.10
# df_mob_temp 中 应付日期为2025.4的，overdue_days_mob 皆为 30，加工口径需修改，本次不影响 rollrate
# var_1 = ['order_number','下单日期','当前期数','应付日期','实付日期new','当前应付租金','实付金额','overdue_days_mob','剩余总应收租金']
# df_mob_temp[(df_mob_temp.剩余总应收租金 == 0) & (df_mob_temp.overdue_days_mob > 0) & (df_mob_temp.应付日期 <= '2025-03-31')].shape
# df_mob_temp[(df_mob_temp.剩余总应收租金 == 0) & (df_mob_temp.overdue_days_mob > 0) & (df_mob_temp.应付日期 <= '2025-03-31')][var_1].tail(15)
# df_mob_temp[df_mob_temp.order_number == 'A20240328174912122'][var_1]

conditions_1 = (~df_mob_temp.订单状态_new.isin(['已买断','已完成','已结束']))
tmp9_1 = pd.DataFrame()
tmp9_1['滚动状态'] = ['1.正常_2.M1','2.M1_3.M2','3.M2_4.M3','4.M3_5.M4','5.M4_6.M5','6.M5_7.M6']

for var_1 in ['2022-07','2022-08','2022-09','2022-10','2022-11','2022-12'
                 ,'2023-01','2023-02','2023-03','2023-04','2023-05','2023-06','2023-07','2023-08','2023-09','2023-10','2023-11','2023-12'	
                 ,'2024-01','2024-02','2024-03','2024-04','2024-05','2024-06','2024-07','2024-08','2024-09','2024-10','2024-11','2024-12'
                 ,'2025-01','2025-02','2025-03','2025-04','2025-05','2025-06','2025-07','2025-08']:
    var_2 = pd.to_datetime(var_1)
    # 往后推一个月
    var_3 = var_2 + pd.DateOffset(months=1)
    var_4 = pd.to_datetime(var_3).strftime('%Y-%m')

    temp1 = df_mob_temp[conditions_1 & (df_mob_temp.view_month == var_1)][['order_number','逾期阶段','剩余采购成本']]
    temp2 = df_mob_temp[conditions_1 & (df_mob_temp.view_month == var_4)][['order_number','逾期阶段','剩余采购成本']]

    temp1 = temp1.rename(columns = {'逾期阶段': '逾期阶段_c','剩余采购成本': '剩余采购成本_c'})
    temp2 = temp2.rename(columns = {'逾期阶段': '逾期阶段_n','剩余采购成本': '剩余采购成本_n'})

    temp3 = temp1.merge(temp2, on = 'order_number', how = 'inner')
    temp3['tmp'] = '_'
    temp3['滚动状态'] = temp3['逾期阶段_c'] + temp3['tmp'] + temp3['逾期阶段_n']
    temp3 = temp3.drop(columns = ['tmp'])
    temp4 = temp3[temp3.滚动状态.isin(['1.正常_2.M1','2.M1_3.M2','3.M2_4.M3','4.M3_5.M4','5.M4_6.M5','6.M5_7.M6'])].groupby(['滚动状态']).agg({'order_number':'count','剩余采购成本_n':'sum'}).reset_index()
    temp4 = temp4.rename(columns = {'order_number': '变坏的订单数'})
    temp4['逾期阶段_c'] = np.where(pd.notnull(temp4.滚动状态), temp4.滚动状态.str[:4], '-999')
    temp5 = temp3[temp3.逾期阶段_c.isin(['1.正常','2.M1','3.M2','4.M3','5.M4','6.M5'])].groupby(['逾期阶段_c']).agg({'order_number':'count','剩余采购成本_c':'sum'}).reset_index()
    temp6 = temp4.merge(temp5, on = '逾期阶段_c', how = 'left')

    temp6[var_1 + '_cnt'] = round(temp6['变坏的订单数'] / temp6['order_number'], 4)
    temp6[var_1 + '_amt'] = round(temp6['剩余采购成本_n'] / temp6['剩余采购成本_c'], 4)
    temp6 = temp6[['滚动状态',var_1 + '_cnt', var_1 + '_amt']]

    tmp9_1 = tmp9_1.merge(temp6, on = '滚动状态', how = 'left')

var_cnt = ['2022-07_cnt','2022-08_cnt','2022-09_cnt','2022-10_cnt','2022-11_cnt','2022-12_cnt'
                 ,'2023-01_cnt','2023-02_cnt','2023-03_cnt','2023-04_cnt','2023-05_cnt','2023-06_cnt','2023-07_cnt','2023-08_cnt','2023-09_cnt','2023-10_cnt','2023-11_cnt','2023-12_cnt'	
                 ,'2024-01_cnt','2024-02_cnt','2024-03_cnt','2024-04_cnt','2024-05_cnt','2024-06_cnt','2024-07_cnt','2024-08_cnt','2024-09_cnt','2024-10_cnt','2024-11_cnt','2024-12_cnt'
                 ,'2025-01_cnt','2025-02_cnt','2025-03_cnt','2025-04_cnt','2025-05_cnt','2025-06_cnt','2025-07_cnt','2025-08_cnt']
var_amt = ['2022-07_amt','2022-08_amt','2022-09_amt','2022-10_amt','2022-11_amt','2022-12_amt'
                 ,'2023-01_amt','2023-02_amt','2023-03_amt','2023-04_amt','2023-05_amt','2023-06_amt','2023-07_amt','2023-08_amt','2023-09_amt','2023-10_amt','2023-11_amt','2023-12_amt'	
                 ,'2024-01_amt','2024-02_amt','2024-03_amt','2024-04_amt','2024-05_amt','2024-06_amt','2024-07_amt','2024-08_amt','2024-09_amt','2024-10_amt','2024-11_amt','2024-12_amt'
                 ,'2025-01_amt','2025-02_amt','2025-03_amt','2025-04_amt','2025-05_amt','2025-06_amt','2025-07_amt','2025-08_amt']
tmp9_1 = tmp9_1[['滚动状态'] + var_cnt + var_amt]
tmp9_1.to_excel('tmp9_1.xlsx', index = False)


In [ ]:
# # 正常客群比之前京享租统计rollrate的时候会多一点，因为之前分首租和续租，首租的第12期订单状态为 已完成，本次不分首租续租，且首租的第12期状态更新为还款中

# # 2025.4.10
# # df_mob_temp 中 应付日期为2025.4的，overdue_days_mob 皆为 30，加工口径需修改，本次不影响 rollrate
# # var_1 = ['order_number','下单日期','当前期数','应付日期','实付日期new','当前应付租金','实付金额','overdue_days_mob','剩余总应收租金']
# # df_mob_temp[(df_mob_temp.剩余总应收租金 == 0) & (df_mob_temp.overdue_days_mob > 0) & (df_mob_temp.应付日期 <= '2025-03-31')].shape
# # df_mob_temp[(df_mob_temp.剩余总应收租金 == 0) & (df_mob_temp.overdue_days_mob > 0) & (df_mob_temp.应付日期 <= '2025-03-31')][var_1].tail(15)
# # df_mob_temp[df_mob_temp.order_number == 'A20240328174912122'][var_1]

# conditions_1 = (~df_mob_temp.订单状态_new.isin(['已买断','已完成','已结束']))
# tmp9_1 = pd.DataFrame()
# tmp9_1['滚动状态'] = ['1.正常_2.M1','2.M1_3.M2','3.M2_4.M3','4.M3_5.M4','5.M4_6.M5','6.M5_7.M6']

# for var_1 in ['2022-07','2022-08','2022-09','2022-10','2022-11','2022-12'
#                  ,'2023-01','2023-02','2023-03','2023-04','2023-05','2023-06','2023-07','2023-08','2023-09','2023-10','2023-11','2023-12'	
#                  ,'2024-01','2024-02','2024-03','2024-04','2024-05','2024-06','2024-07','2024-08','2024-09','2024-10','2024-11','2024-12'
#                  ,'2025-01','2025-02','2025-03','2025-04','2025-05','2025-06']:
#     var_2 = pd.to_datetime(var_1)
#     # 往后推一个月
#     var_3 = var_2 + pd.DateOffset(months=1)
#     var_4 = pd.to_datetime(var_3).strftime('%Y-%m')

#     temp1 = df_mob_temp[conditions_1 & (df_mob_temp.view_month == var_1)][['order_number','逾期阶段','剩余采购成本_new']]
#     temp2 = df_mob_temp[conditions_1 & (df_mob_temp.view_month == var_4)][['order_number','逾期阶段','剩余采购成本_new']]

#     temp1 = temp1.rename(columns = {'逾期阶段': '逾期阶段_c','剩余采购成本_new': '剩余采购成本_c'})
#     temp2 = temp2.rename(columns = {'逾期阶段': '逾期阶段_n','剩余采购成本_new': '剩余采购成本_n'})

#     temp3 = temp1.merge(temp2, on = 'order_number', how = 'inner')
#     temp3['tmp'] = '_'
#     temp3['滚动状态'] = temp3['逾期阶段_c'] + temp3['tmp'] + temp3['逾期阶段_n']
#     temp3 = temp3.drop(columns = ['tmp'])
#     temp4 = temp3[temp3.滚动状态.isin(['1.正常_2.M1','2.M1_3.M2','3.M2_4.M3','4.M3_5.M4','5.M4_6.M5','6.M5_7.M6'])].groupby(['滚动状态']).agg({'order_number':'count','剩余采购成本_n':'sum'}).reset_index()
#     temp4 = temp4.rename(columns = {'order_number': '变坏的订单数'})
#     temp4['逾期阶段_c'] = np.where(pd.notnull(temp4.滚动状态), temp4.滚动状态.str[:4], '-999')
#     temp5 = temp3[temp3.逾期阶段_c.isin(['1.正常','2.M1','3.M2','4.M3','5.M4','6.M5'])].groupby(['逾期阶段_c']).agg({'order_number':'count','剩余采购成本_c':'sum'}).reset_index()
#     temp6 = temp4.merge(temp5, on = '逾期阶段_c', how = 'left')

#     temp6[var_1 + '_cnt'] = round(temp6['变坏的订单数'] / temp6['order_number'], 4)
#     temp6[var_1 + '_amt'] = round(temp6['剩余采购成本_n'] / temp6['剩余采购成本_c'], 4)
#     temp6 = temp6[['滚动状态',var_1 + '_cnt', var_1 + '_amt']]

#     tmp9_1 = tmp9_1.merge(temp6, on = '滚动状态', how = 'left')

# var_cnt = ['2022-07_cnt','2022-08_cnt','2022-09_cnt','2022-10_cnt','2022-11_cnt','2022-12_cnt'
#                  ,'2023-01_cnt','2023-02_cnt','2023-03_cnt','2023-04_cnt','2023-05_cnt','2023-06_cnt','2023-07_cnt','2023-08_cnt','2023-09_cnt','2023-10_cnt','2023-11_cnt','2023-12_cnt'	
#                  ,'2024-01_cnt','2024-02_cnt','2024-03_cnt','2024-04_cnt','2024-05_cnt','2024-06_cnt','2024-07_cnt','2024-08_cnt','2024-09_cnt','2024-10_cnt','2024-11_cnt','2024-12_cnt'
#                  ,'2025-01_cnt','2025-02_cnt','2025-03_cnt','2025-04_cnt','2025-05_cnt','2025-06_cnt']
# var_amt = ['2022-07_amt','2022-08_amt','2022-09_amt','2022-10_amt','2022-11_amt','2022-12_amt'
#                  ,'2023-01_amt','2023-02_amt','2023-03_amt','2023-04_amt','2023-05_amt','2023-06_amt','2023-07_amt','2023-08_amt','2023-09_amt','2023-10_amt','2023-11_amt','2023-12_amt'	
#                  ,'2024-01_amt','2024-02_amt','2024-03_amt','2024-04_amt','2024-05_amt','2024-06_amt','2024-07_amt','2024-08_amt','2024-09_amt','2024-10_amt','2024-11_amt','2024-12_amt'
#                  ,'2025-01_amt','2025-02_amt','2025-03_amt','2025-04_amt','2025-05_amt','2025-06_amt']
# tmp9_1 = tmp9_1[['滚动状态'] + var_cnt + var_amt]
# tmp9_1.to_excel('tmp9_1_new.xlsx', index = False)

In [ ]:
# 每个观察点各个逾期阶段的分布
# conditions_1 = (~df_mob_temp.订单状态_new.isin(['已买断','已完成','已结束']))
# pd.pivot_table(df_mob_temp[(df_mob_temp.view_month >= '2024-07')&conditions_1],index='逾期阶段',columns='view_month',values='order_number',aggfunc='count')

### 4_7_5.首逾率

In [ ]:
# （1-1）按月维度统计，首逾发生在第2/3/4期的情况
df_xz_concat_1 = df_xz_concat.copy()
df_xz_concat_1 = df_xz_concat_1[~df_xz_concat_1.order_number.isin(goal_df_order_number)]
# # 2025.4 和 2025.5 的出库不含拒量
# df_xz_concat_1 = df_xz_concat_1[~(df_xz_concat_1.search_time.isin(['2025-04','2025-05']) & (df_xz_concat_1.是否拒量 == 1))]

df_xz_concat_1['当前期数'] = df_xz_concat_1['当前期数'].astype(int)

df_yq_fq = df_xz_concat_1[["order_number","当前期数","逾期天数"]].pivot(index="order_number", columns="当前期数", values="逾期天数")
df_yq_fq = df_yq_fq.reset_index()
tmp1 = df_xz_concat_1[df_xz_concat_1.当前期数 == 1][['order_number','search_time','purchase_amount']]
tmp1 = tmp1.drop_duplicates(subset = 'order_number')
df_yq_fq = df_yq_fq.merge(tmp1, on = 'order_number', how = 'left')

df_yq_fq["FPD"] = df_yq_fq[2]
# note 20241222 修正SPD TPD QPD
df_yq_fq.loc[:,"SPD"] = np.where( df_yq_fq[2] == 0, df_yq_fq[3], 0)
df_yq_fq.loc[:,"TPD"] = np.where((df_yq_fq[2] == 0) & (df_yq_fq[3] == 0), df_yq_fq[4], 0)
df_yq_fq.shape

tmp = df_mob_temp.drop_duplicates(subset = 'order_number')[['order_number']]
tmp.shape

df_yq_fq = df_yq_fq.merge(tmp, on = 'order_number', how = 'inner')
df_yq_fq.shape
df_yq_fq[['order_number','search_time',2,3,4,'FPD','SPD','TPD']][:2]

(59188, 42)

(58966, 1)

(58966, 42)

,order_number,search_time,2,3,4,FPD,SPD,TPD
0,A202207050851373,2022-07,0.0,0.0,4.0,0.0,0.0,4.0
1,A202207131230485,2022-07,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# （1-2）订单维度-分子
tmp0 = df_yq_fq.groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp1 = df_yq_fq[df_yq_fq.FPD >= 1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp2 = df_yq_fq[df_yq_fq.FPD >= 7].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp3 = df_yq_fq[df_yq_fq.FPD >= 15].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp4 = df_yq_fq[df_yq_fq.FPD >= 30].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()

tmp5 = df_yq_fq[df_yq_fq.SPD >= 1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp6 = df_yq_fq[df_yq_fq.SPD >= 30].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp7 = df_yq_fq[df_yq_fq.TPD >= 1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp8 = df_yq_fq[df_yq_fq.TPD >= 30].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()

tmp1 = tmp1.rename(columns = {'order_number': 'FPD1'})
tmp2 = tmp2.rename(columns = {'order_number': 'FPD7'})
tmp3 = tmp3.rename(columns = {'order_number': 'FPD15'})
tmp4 = tmp4.rename(columns = {'order_number': 'FPD30'})
tmp5 = tmp5.rename(columns = {'order_number': 'SPD1'})
tmp6 = tmp6.rename(columns = {'order_number': 'SPD30'})
tmp7 = tmp7.rename(columns = {'order_number': 'TPD1'})
tmp8 = tmp8.rename(columns = {'order_number': 'TPD30'})

tmp10_1 = tmp0.merge(tmp1, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp2, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp3, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp4, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp5, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp6, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp7, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp8, on = 'search_time', how = 'left')

tmp10_1

,search_time,order_number,FPD1,FPD7,FPD15,FPD30,SPD1,SPD30,TPD1,TPD30
0,2022-07,21,4.0,3.0,2.0,2.0,3.0,2.0,1.0,NaN
1,2022-08,53,19.0,7.0,6.0,6.0,6.0,2.0,5.0,1.0
2,2022-09,40,9.0,6.0,3.0,3.0,6.0,1.0,1.0,NaN
3,2022-10,38,5.0,2.0,2.0,1.0,4.0,NaN,3.0,1.0
4,2022-11,41,10.0,3.0,3.0,3.0,3.0,NaN,2.0,NaN
5,2022-12,63,12.0,7.0,5.0,4.0,4.0,NaN,6.0,NaN
6,2023-01,212,50.0,16.0,14.0,11.0,27.0,6.0,12.0,2.0
7,2023-02,317,65.0,32.0,24.0,22.0,20.0,7.0,18.0,4.0
8,2023-03,302,31.0,23.0,18.0,15.0,34.0,10.0,29.0,5.0
9,2023-04,606,69.0,30.0,21.0,19.0,69.0,13.0,73.0,11.0


In [ ]:
# （2-1）每期的剩余采购成本取的是对应那期期末的剩余采购成本，不是那期逾期最终状态的剩余采购成本

tmp1 = df_mob_temp[df_mob_temp.当前期数.isin([2,3,4])].pivot(index="order_number", columns="当前期数", values="剩余采购成本")
tmp1 = tmp1.rename(columns = {2:'第2期剩余采购成本', 3:'第3期剩余采购成本', 4:'第4期剩余采购成本'})
df_yq_fq = df_yq_fq.merge(tmp1, on = 'order_number', how = 'left')

df_yq_fq['第2期剩余采购成本'] = df_yq_fq['第2期剩余采购成本'].fillna(0)
df_yq_fq['第3期剩余采购成本'] = df_yq_fq['第3期剩余采购成本'].fillna(0)
df_yq_fq['第4期剩余采购成本'] = df_yq_fq['第4期剩余采购成本'].fillna(0)
df_yq_fq[['order_number','search_time','FPD','SPD','TPD','第2期剩余采购成本','第3期剩余采购成本','第4期剩余采购成本']][:2]

# df_yq_fq[df_yq_fq.order_number == 'A2022073117434010'][['order_number','search_time','FPD','SPD','TPD','第2期剩余采购成本','第3期剩余采购成本','第4期剩余采购成本']]

,order_number,search_time,FPD,SPD,TPD,第2期剩余采购成本,第3期剩余采购成本,第4期剩余采购成本
0,A202207050851373,2022-07,0.0,0.0,4.0,4434.1,4076.15,3718.2
1,A202207131230485,2022-07,0.0,0.0,0.0,0.0,0.00,0.0


In [ ]:
# （2-2）剩余采购成本维度-分子
tmp1 = df_yq_fq[df_yq_fq.FPD >= 1].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()
tmp2 = df_yq_fq[df_yq_fq.FPD >= 7].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()
tmp3 = df_yq_fq[df_yq_fq.FPD >= 15].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()
tmp4 = df_yq_fq[df_yq_fq.FPD >= 30].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()

tmp5 = df_yq_fq[df_yq_fq.SPD >= 1].groupby(['search_time']).agg({'第3期剩余采购成本': 'sum'}).reset_index()
tmp6 = df_yq_fq[df_yq_fq.SPD >= 30].groupby(['search_time']).agg({'第3期剩余采购成本': 'sum'}).reset_index()
tmp7 = df_yq_fq[df_yq_fq.TPD >= 1].groupby(['search_time']).agg({'第4期剩余采购成本': 'sum'}).reset_index()
tmp8 = df_yq_fq[df_yq_fq.TPD >= 30].groupby(['search_time']).agg({'第4期剩余采购成本': 'sum'}).reset_index()

tmp1 = tmp1.rename(columns = {'第2期剩余采购成本': 'FPD1'})
tmp2 = tmp2.rename(columns = {'第2期剩余采购成本': 'FPD7'})
tmp3 = tmp3.rename(columns = {'第2期剩余采购成本': 'FPD15'})
tmp4 = tmp4.rename(columns = {'第2期剩余采购成本': 'FPD30'})
tmp5 = tmp5.rename(columns = {'第3期剩余采购成本': 'SPD1'})
tmp6 = tmp6.rename(columns = {'第3期剩余采购成本': 'SPD30'})
tmp7 = tmp7.rename(columns = {'第4期剩余采购成本': 'TPD1'})
tmp8 = tmp8.rename(columns = {'第4期剩余采购成本': 'TPD30'})

tmp11_1 = tmp1.merge(tmp2, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp3, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp4, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp5, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp6, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp7, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp8, on = 'search_time', how = 'left')

tmp11_1

,search_time,FPD1,FPD7,FPD15,FPD30,SPD1,SPD30,TPD1,TPD30
0,2022-07,17905.60,13265.65,10748.26,10748.26,20098.00,13176.54,3718.20,NaN
1,2022-08,91656.76,40123.54,35705.01,35705.01,31942.45,13138.77,20942.05,6316.30
2,2022-09,53976.80,36111.72,21499.23,21499.23,27561.98,5652.40,2007.80,NaN
3,2022-10,26266.53,10809.23,10809.23,8365.03,10071.60,NaN,14628.46,4146.46
4,2022-11,58321.29,20254.29,20254.29,20254.29,13904.80,NaN,13828.10,NaN
5,2022-12,77231.10,50555.30,36481.40,28257.50,21434.99,NaN,29446.90,NaN
6,2023-01,265966.65,83032.50,70097.00,57169.55,124113.00,11129.90,55920.35,11357.25
7,2023-02,347394.25,175620.80,137242.79,131860.16,102110.90,45822.55,79854.55,19028.56
8,2023-03,176122.74,129610.86,101117.44,88138.32,153324.92,44216.06,121237.29,24194.20
9,2023-04,428289.74,196393.09,134466.59,125395.33,370269.70,79773.71,356365.12,58393.33


### 4_7_6.资产分布

In [ ]:
tmp1 = df_mob_temp[(df_mob_temp.当前期数 == 1)][['order_number','search_time','purchase_amount','最大期数_Bin']]
tmp1.shape
tmp1['purchase_amount_Bin'] = tmp1.apply(lambda x: '1.(0,3000)' if x.purchase_amount >= 0 and x.purchase_amount < 3000
                                         else '2.[3000,5000)' if x.purchase_amount >= 3000 and x.purchase_amount < 5000
                                         else '3.[5000,10000)' if x.purchase_amount >= 5000 and x.purchase_amount < 10000
                                         else '4.[10000,+)' if x.purchase_amount >= 10000
                                         else 'other', axis = 1)
tmp1[tmp1.search_time <= '2025-06'][['order_number','search_time','purchase_amount']].to_excel('tmp12_1.xlsx', index = False) # 导出去看年龄、性别和地区
tmp1[tmp1.search_time <= '2025-06'].groupby(['最大期数_Bin']).agg({'purchase_amount': 'sum', 'order_number': 'count'})
tmp1[tmp1.search_time <= '2025-06'].groupby(['purchase_amount_Bin']).agg({'order_number': 'count','purchase_amount': 'sum'})

(48272, 4)

,purchase_amount,order_number
最大期数_Bin,,
12个月,3.187353e+08,41259
18个月,7.139573e+06,1015
24个月,3.620321e+07,4988


,order_number,purchase_amount
purchase_amount_Bin,,
"1.(0,3000)",2503,1.937901e+06
"2.[3000,5000)",1725,7.313476e+06
"3.[5000,10000)",41919,3.410992e+08
"4.[10000,+)",1115,1.172741e+07


### 4_7_7.资方

In [ ]:
temp1 = df_xz_concat[['order_number','merchant_name']]
temp1 = temp1.drop_duplicates(subset = 'order_number')
temp2 = df_mob_temp.merge(temp1, on = 'order_number', how = 'left')

sql = '''-- 租后应收监控  
SELECT  tprm.order_number
,ym.name -- 订单归属资方
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
;
'''

temp3 = query(sql)
temp2 = temp2.merge(temp3, on = 'order_number', how = 'left')

conditions_1 = (temp2.view_month == '2025-08')
temp2[conditions_1].剩余总应收.sum()

# name 维客自营账户   --  merchant_name 迪瓜优选
# 海南鸿丰商业保理有限公司
# 杭州金锤网络信息服务有限公司
# 成都越蓉金科科技有限公司

# temp2[conditions_1 & (temp2.merchant_name == '成都越蓉金科科技有限公司')].shape
# temp2[conditions_1 & (temp2.name == '成都越蓉金科科技有限公司')].shape

conditions_2 = (temp2.name.isin(['维客自营账户','海南鸿丰商业保理有限公司','杭州金锤网络信息服务有限公司','成都越蓉金科科技有限公司']))
conditions_3 = (temp2.merchant_name.isin(['迪瓜优选','海南鸿丰商业保理有限公司','杭州金锤网络信息服务有限公司','成都越蓉金科科技有限公司']))
temp2[conditions_1 & conditions_2].剩余总应收.sum()
temp2[conditions_1 & conditions_3].剩余总应收.sum()

418353337.03

88204873.31

88212545.31

### 4_7_8.众邦

#### （1）.附表4-出库余额及逾期情况

In [ ]:
tmp1 = df_mob_temp[df_mob_temp.view_month == '2025-06'].groupby(['search_time']).agg({'剩余采购成本':'sum'}).reset_index()
tmp2 = df_mob_temp[(df_mob_temp.view_month == '2025-06')&(df_mob_temp.overdue_days_mob > 0)&(df_mob_temp.overdue_days_mob <= 30)].groupby(['search_time']).agg({'剩余采购成本':'sum'}).reset_index()
tmp3 = df_mob_temp[(df_mob_temp.view_month == '2025-06')&(df_mob_temp.overdue_days_mob > 30)&(df_mob_temp.overdue_days_mob <= 60)].groupby(['search_time']).agg({'剩余采购成本':'sum'}).reset_index()
tmp4 = df_mob_temp[(df_mob_temp.view_month == '2025-06')&(df_mob_temp.overdue_days_mob > 60)&(df_mob_temp.overdue_days_mob <= 90)].groupby(['search_time']).agg({'剩余采购成本':'sum'}).reset_index()
tmp5 = df_mob_temp[(df_mob_temp.view_month == '2025-06')&(df_mob_temp.overdue_days_mob > 90)].groupby(['search_time']).agg({'剩余采购成本':'sum'}).reset_index()

tmp2 = tmp2.rename(columns = {'剩余采购成本': 'M1'})
tmp3 = tmp3.rename(columns = {'剩余采购成本': 'M2'})
tmp4 = tmp4.rename(columns = {'剩余采购成本': 'M3'})
tmp5 = tmp5.rename(columns = {'剩余采购成本': 'M3+'})

tmp1 = tmp1.merge(tmp2, on = 'search_time', how = 'left')
tmp1 = tmp1.merge(tmp3, on = 'search_time', how = 'left')
tmp1 = tmp1.merge(tmp4, on = 'search_time', how = 'left')
tmp1 = tmp1.merge(tmp5, on = 'search_time', how = 'left')
tmp1

,search_time,剩余采购成本,M1,M2,M3,M3+
0,2022-07,32829.33,NaN,NaN,NaN,32829.33
1,2022-08,99547.63,NaN,NaN,NaN,99547.63
2,2022-09,52442.88,NaN,NaN,NaN,52442.88
3,2022-10,42740.25,NaN,NaN,NaN,42740.25
4,2022-11,52461.52,NaN,NaN,NaN,52461.52
5,2022-12,75207.70,NaN,NaN,NaN,75207.70
6,2023-01,205378.81,NaN,NaN,NaN,205378.81
7,2023-02,312830.47,NaN,NaN,NaN,312830.47
8,2023-03,326020.61,NaN,NaN,0.00,326020.61
9,2023-04,629564.68,NaN,NaN,NaN,629564.68


#### （2）.附表8-资金方合作情况

In [ ]:
sql = '''
SELECT  tprm.order_number
,ym.name

from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
;
'''

df_name = query(sql)
df_name.shape
df_name[:1]

(55405, 2)

,order_number,name
0,A202207050851373,维客自营账户


In [ ]:
temp2 = df_mob_temp[['order_number','purchase_amount','search_time']].drop_duplicates(subset = 'order_number')
temp3 = df_mob_temp[df_mob_temp.view_month == '2025-06'][['order_number','剩余采购成本']]

temp5 = temp2.merge(temp3, on = 'order_number', how = 'left')
temp5 = temp5.merge(df_name, on = 'order_number', how = 'left')
temp2.shape
temp5.shape

temp5['purchase_amount'] = temp5['purchase_amount'].fillna(0)
temp5['剩余采购成本'] = temp5['剩余采购成本'].fillna(0)
temp5.tail()

(48272, 3)

(48285, 5)

,order_number,purchase_amount,search_time,剩余采购成本,name
48280,A20240704114741268,8062.0,2024-07,1153.23,绿能智联科技
48281,A20240825124054133,2613.0,2024-08,0.00,永冠兴十二号店
48282,A20241228145823295,1716.0,2024-12,0.00,大龙发十号店
48283,A20250502172735177,2588.0,2025-05,2044.00,百力科技六号店
48284,A20250602205309278,0.0,2025-06,0.00,金川惠联惠


In [ ]:
# temp5[temp5.name == '维客自营账户'].purchase_amount.sum()
temp5[temp5.name == '杭州金锤网络信息服务有限公司'].purchase_amount.sum()
temp5[temp5.name == '海南鸿丰商业保理有限公司'].purchase_amount.sum()
temp5[temp5.name == '成都越蓉金科科技有限公司'].purchase_amount.sum()


# temp5[temp5.name == '维客自营账户'].剩余采购成本.sum()
temp5[temp5.name == '杭州金锤网络信息服务有限公司'].剩余采购成本.sum()
temp5[temp5.name == '海南鸿丰商业保理有限公司'].剩余采购成本.sum()
temp5[temp5.name == '成都越蓉金科科技有限公司'].剩余采购成本.sum()

# temp5.to_excel('temp5.xlsx', index = False)

14533298.0

20027313.0

7001558.0

5526159.17

12755822.219999999

1295266.32

### 4_7_9.数据核对

In [ ]:
# 1.余额衰减 中每个下单月的最后一个mob加起来是 每月截面数据 中对应观察月的总额

## 4_8.统计（8.20 new）

### 4_8_0.采购成本分摊口径梳理

<!-- 202508采购成本分摊逻辑：
1、租完即送--采购成本平均分摊到每期，即每期应还采购成本=采购成本/期数，每期应还采购成本每期应还采购成本=月租金*（采购成本/签约价）
2、租完归还
（1）归还（/买断）：分摊系数=总租金/签约价，租赁期间的每期应还采购成本=采购成本*分摊系数/期数，如签约价是13000、采购成本是9000、总租金是6500、买断尾款是6500、租期12期，则每期应还采购成本(375)=6500/13000*9000/12
（2）续租：首租的每期应还采购成本如（1），续租部分的每期应还采购成本=（总采购成本-首租采购成本）/续租期数 -->


In [ ]:
# df_mob_temp 表的字段解析
# 买断价：区分首租、续租（其中续租的包含首租的）
# 总应收：区分首租、续租
# 应付总租金：区分首租、续租
# 当前累计实付租金：区分首租、续租
# is_relet >= 1：为续租

# 增加/调整字段：
# （1）增加 is_relet
# （2）调整 is_relet_max 
# （3）增加 期数（区分首租续租）
# （4）增加 买断价_max
# （5）增加 采购成本分摊（区分首租续租）
# （6）增加 当前累计实付采购成本（区分首租续租）
# （7）增加 剩余采购成本_202508

# 新口径-vintage（续租跟首租的合并为一个订单，但分子不区分首续租） --本次使用：
# （1）分母：租送-采购成本，租归且有续租的订单-采购成本，租归但未续租的订单-分摊后的采购成本

# 新口径-vintage（续租跟首租的拆开为两个订单，分子区分首租和续租） --本次未使用：
# （1）分母：租送-采购成本，租归且有续租的订单-首租的为一个订单、续租的为新订单，租归但未续租的订单-分摊后的采购成本

In [ ]:
df_mob_temp = df_mob_temp.merge(df_xz_concat[['order_number','is_relet','当前期数']], on = ['order_number','当前期数'], how = 'left')
df_mob_temp = df_mob_temp.drop(columns = 'is_relet_max')

temp1 = df_xz_concat.groupby(['order_number']).agg({'is_relet': 'max'}).reset_index()
temp1 = temp1.rename(columns = {'is_relet': 'is_relet_max'})
df_mob_temp = df_mob_temp.merge(temp1, on = 'order_number', how = 'left')
df_mob_temp['is_relet'] = np.where(df_mob_temp.is_relet.isna(), df_mob_temp.is_relet_max, df_mob_temp.is_relet)

temp2 = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数': 'max'}).reset_index()
temp2 = temp2.rename(columns = {'当前期数': '期数'})
df_mob_temp = df_mob_temp.merge(temp2, on = 'order_number', how = 'left')

# temp3 = df_mob_temp.groupby(['order_number']).agg({'买断价': 'max'}).reset_index()
# temp3 = temp3.rename(columns = {'买断价': '买断价_max'})
# df_mob_temp = df_mob_temp.merge(temp3, on = 'order_number', how = 'left')

In [ ]:
df_mob_temp.shape
df_mob_sz = df_mob_temp[df_mob_temp.is_relet == 0]
df_mob_xz = df_mob_temp[df_mob_temp.is_relet > 0]
df_mob_sz.shape
df_mob_xz.shape

# df_mob_sz_zs = df_mob_sz[df_mob_sz.租赁方案 == '租完即送']
# df_mob_sz_zg = df_mob_sz[df_mob_sz.租赁方案 == '租完归还']

# 推算后，采购成本占的权重 = 采购成本/买断价
# df_mob_sz_zs['当前累计实付采购成本'] = df_mob_sz_zs['当前累计实付租金']*(df_mob_sz_zs['purchase_amount']/df_mob_sz_zs['买断价'])
# df_mob_sz_zg['当前累计实付采购成本'] = df_mob_sz_zg['当前累计实付租金']*(df_mob_sz_zg['purchase_amount']/df_mob_sz_zg['买断价'])

# 买断价：以首租的买断价为准
df_mob_sz['当前累计实付采购成本'] = df_mob_sz['当前累计实付租金']*(df_mob_sz['purchase_amount']/df_mob_sz['买断价'])
df_mob_sz['采购成本分摊'] = df_mob_sz['purchase_amount']*(df_mob_sz['应付总租金']/df_mob_sz['买断价'])
df_mob_sz['采购成本分摊'] = np.where(df_mob_sz.租赁方案 == '租完即送', df_mob_sz.purchase_amount, df_mob_sz.采购成本分摊)
df_mob_sz['tmp1'] = df_mob_sz['purchase_amount'] - df_mob_sz['采购成本分摊']

temp4 = df_mob_sz.groupby(['order_number']).agg({'tmp1': 'max'}).reset_index()
temp4 = temp4.rename(columns = {'tmp1': '采购成本分摊'})

df_mob_xz = df_mob_xz.merge(temp4, on = 'order_number', how = 'left')
# df_mob_xz['当前累计实付采购成本'] = df_mob_xz['当前累计实付租金']*(df_mob_xz['采购成本分摊']/df_mob_xz['买断价'])
df_mob_xz['当前累计实付采购成本'] = df_mob_xz['当前累计实付租金']*(df_mob_xz['采购成本分摊']/df_mob_xz['应付总租金'])

df_mob_sz = df_mob_sz.drop(columns = ['tmp1'])

# view_month 为系统月的 overdue_days_mob 有误，不要使用
# 当前累计实付租金 等指标区分了首租和续租，要合并起来
temp5 = df_mob_sz.groupby(['order_number']).agg({'当前累计实付采购成本': 'max'}).reset_index()
temp5 = temp5.rename(columns = {'当前累计实付采购成本': '当前累计实付采购成本_首租'})
df_mob_xz = df_mob_xz.merge(temp5, on = 'order_number', how = 'left')

df_mob_xz['当前累计实付采购成本_首租'] = df_mob_xz['当前累计实付采购成本_首租'].fillna(0)
df_mob_xz['当前累计实付采购成本_all'] = df_mob_xz['当前累计实付采购成本_首租'] + df_mob_xz['当前累计实付采购成本']
df_mob_xz = df_mob_xz.drop(columns = ['当前累计实付采购成本_首租'])

df_mob_sz['当前累计实付采购成本_all'] = df_mob_sz['当前累计实付采购成本']


df_mob_temp = pd.concat([df_mob_sz, df_mob_xz])
# vintage--分母：租送-采购成本，租归且有续租的订单-采购成本，租归但未续租的订单-分摊后的采购成本
df_mob_temp['采购成本分摊_new'] = np.where(df_mob_temp.is_relet_max > 0, df_mob_temp.purchase_amount, df_mob_temp.采购成本分摊)
df_mob_temp['剩余采购成本_202508'] = df_mob_temp['采购成本分摊_new'] - df_mob_temp['当前累计实付采购成本_all']
# df_mob_temp['剩余采购成本_202508'] = df_mob_temp['采购成本分摊'] - df_mob_temp['当前累计实付采购成本']

df_mob_temp.shape
df_mob_temp[:1]

# var_1 = ['order_number','应付日期','当前期数','purchase_amount','当前累计实付租金','当前累计实付租金_all','剩余采购成本'
#          ,'采购成本分摊','采购成本分摊_new','当前累计实付采购成本','当前累计实付采购成本_all','剩余采购成本_202508','overdue_days_mob']

# df_mob_temp[df_mob_temp.order_number == 'A2023090814292948'][var_1]
# A202207050851373  --续租，提前买断
# A202208040916342  --归还逾期
# A202503020002175  --租归，应还日期为系统月的 overdue_days_mob 有误，不要使用
# A2024040309475173  --严重逾期
# A2024051707383726  --续租
# A2023090814292948   --租送

# df_mob_temp[df_mob_temp.租赁方案 == '租完即送'][['order_number']][30000:30005]

# df_mob_temp[(df_mob_temp.租赁方案 == '租完即送')&(df_mob_temp.purchase_amount != df_mob_temp.采购成本分摊_new)].shape
# df_mob_temp[(df_mob_temp.租赁方案 == '租完归还')&(df_mob_temp.is_relet_max > 0)&(df_mob_temp.purchase_amount != df_mob_temp.采购成本分摊_new)].shape
# df_mob_temp[(df_mob_temp.租赁方案 == '租完归还')&(df_mob_temp.is_relet_max == 0)&(df_mob_temp.purchase_amount == df_mob_temp.采购成本分摊_new)
#             &(df_mob_temp.purchase_amount >0)&(df_mob_temp.买断价 != df_mob_temp.应付总租金)].shape

(503343, 43)

(457014, 43)

(46329, 43)

(503343, 48)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1,是否拒量,price,residue_money,unrecovered_cost,剩余采购成本_new,is_relet,is_relet_max,期数,当前累计实付采购成本,采购成本分摊,当前累计实付采购成本_all,采购成本分摊_new,剩余采购成本_202508
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0,0,0.0,0.0,-3026.92,4792.05,0.0,2.0,12.0,244.813081,2937.756972,244.813081,5150.0,4905.186919


### 4_8_1.余额衰减

In [ ]:
# MOB：首期，用应付日期，不是实付日期，故月末下单的订单，没有MOB0，从MOB1开始（本次）；若首期按实际支付日期计算的话，月末下单的订单，其实是没有MOB1的
MOB_list = ['MOB0','MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
conditions_1 = (df_mob_temp.search_time <= '2025-07')
conditions_2 = (df_mob_temp.view_month <= '2025-07')

tmp1_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].groupby(['search_time']).agg({'purchase_amount': 'sum','采购成本分摊_new': 'sum'}).reset_index()
tmp1_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
tmp1_2 = tmp1_2[['search_time'] + MOB_list]
tmp1_1 = tmp1_1.merge(tmp1_2, on = 'search_time', how = 'left')
tmp1_1.to_excel('tmp1_1.xlsx', index = False)
# # 增加订单维度
# tmp1_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp1_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp1_2 = tmp1_2[['search_time'] + MOB_list]
# tmp1_1 = tmp1_1.merge(tmp1_2, on = 'search_time', how = 'left')
# tmp1_1.to_excel('tmp1_2.xlsx', index = False)

tmp2_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '12个月')].groupby(['search_time']).agg({'purchase_amount': 'sum','采购成本分摊_new': 'sum'}).reset_index()
tmp2_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '12个月')],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
tmp2_2 = tmp2_2[['search_time'] + MOB_list]
tmp2_1 = tmp2_1.merge(tmp2_2, on = 'search_time', how = 'left')
tmp2_1.to_excel('tmp2_1.xlsx', index = False)
# # 增加订单维度
# tmp2_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '12个月')].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp2_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '12个月')],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp2_2 = tmp2_2[['search_time'] + MOB_list]
# tmp2_1 = tmp2_1.merge(tmp2_2, on = 'search_time', how = 'left')
# tmp2_1.to_excel('tmp2_2.xlsx', index = False)


MOB_list = ['MOB0','MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
tmp3_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '18个月')].groupby(['search_time']).agg({'purchase_amount': 'sum','采购成本分摊_new': 'sum'}).reset_index()
tmp3_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '18个月')],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
tmp3_2 = tmp3_2[['search_time'] + MOB_list]
tmp3_1 = tmp3_1.merge(tmp3_2, on = 'search_time', how = 'left')
tmp3_1.to_excel('tmp3_1.xlsx', index = False)
# # 增加订单维度
# tmp3_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '18个月')].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp3_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '18个月')],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp3_2 = tmp3_2[['search_time'] + MOB_list]
# tmp3_1 = tmp3_1.merge(tmp3_2, on = 'search_time', how = 'left')
# tmp3_1.to_excel('tmp3_2.xlsx', index = False)

MOB_list = ['MOB0','MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
tmp4_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '24个月')].groupby(['search_time']).agg({'purchase_amount': 'sum','采购成本分摊_new': 'sum'}).reset_index()
tmp4_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '24个月')],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
tmp4_2 = tmp4_2[['search_time'] + MOB_list]
tmp4_1 = tmp4_1.merge(tmp4_2, on = 'search_time', how = 'left')
tmp4_1.to_excel('tmp4_1.xlsx', index = False)
# # 增加订单维度
# tmp4_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '24个月')].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp4_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '24个月')],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp4_2 = tmp4_2[['search_time'] + MOB_list]
# tmp4_1 = tmp4_1.merge(tmp4_2, on = 'search_time', how = 'left')
# tmp4_1.to_excel('tmp4_2.xlsx', index = False)

# 抖音的有首租是24期的
# df_mob_temp[(df_mob_temp.最大期数_Bin == '24个月')&(df_mob_temp.search_time == '2025-05')][var_1]

### 4_8_2.每月截面数据

In [ ]:
# 每月末采购价各逾期阶段的数据
df_mob_temp['逾期阶段'] = df_mob_temp.apply(lambda x: '1.正常' if x.overdue_days_mob == 0
                                        else '2.M1' if x.overdue_days_mob >= 1 and x.overdue_days_mob <= 30
                                        else '3.M2' if x.overdue_days_mob >= 31 and x.overdue_days_mob <= 60
                                        else '4.M3' if x.overdue_days_mob >= 61 and x.overdue_days_mob <= 90
                                        else '5.M4' if x.overdue_days_mob >= 91 and x.overdue_days_mob <= 120
                                        else '6.M5' if x.overdue_days_mob >= 121 and x.overdue_days_mob <= 150
                                        else '7.M6' if x.overdue_days_mob >= 151 and x.overdue_days_mob <= 180
                                        else '8.M6+' if x.overdue_days_mob > 180
                                        else '9.other', axis = 1)

In [ ]:
# 2023.10、2023.11、2023.12 有差异
tmp5_1 = pd.pivot_table(df_mob_temp,index='逾期阶段',columns='view_month',values='剩余采购成本_202508',aggfunc='sum').reset_index()
tmp5_1.to_excel('tmp5_1.xlsx', index = False)

In [ ]:
# # 增加订单维度
# tmp5_2 = pd.pivot_table(df_mob_temp,index='逾期阶段',columns='view_month',values='order_number',aggfunc='count').reset_index()
# tmp5_2.to_excel('tmp5_2.xlsx', index = False)

### 4_8_3.vintage（金额比例）

In [ ]:
# 分子统计
conditions_2 = (df_mob_temp.view_month <= '2025-07')
conditions_3 = (df_mob_temp.search_time <= '2025-05')
conditions_4 = (df_mob_temp.overdue_days_mob > 30)

tmp6_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_3 & conditions_4],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
MOB_list = ['MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
tmp6_1 = tmp6_1[['search_time'] + MOB_list]
tmp6_1.to_excel('tmp6_1.xlsx', index = False)

In [ ]:
conditions_5 = (df_mob_temp.search_time <= '2025-04')
conditions_6 = (df_mob_temp.overdue_days_mob > 60)

tmp7_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_5 & conditions_6],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
MOB_list = ['MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
tmp7_1 = tmp7_1[['search_time'] + MOB_list]
tmp7_1.to_excel('tmp7_1.xlsx', index = False)

In [ ]:
conditions_7 = (df_mob_temp.search_time <= '2025-03')
conditions_8 = (df_mob_temp.overdue_days_mob > 90)

tmp8_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_7 & conditions_8],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
MOB_list = ['MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
tmp8_1 = tmp8_1[['search_time'] + MOB_list]
tmp8_1.to_excel('tmp8_1.xlsx', index = False)

### 4_8_4.迁徙率

In [ ]:
# 正常客群比之前京享租统计rollrate的时候会多一点，因为之前分首租和续租，首租的第12期订单状态为 已完成，本次不分首租续租，且首租的第12期状态更新为还款中

# 2025.4.10
# df_mob_temp 中 应付日期为2025.4的，overdue_days_mob 皆为 30，加工口径需修改，本次不影响 rollrate
# var_1 = ['order_number','下单日期','当前期数','应付日期','实付日期new','当前应付租金','实付金额','overdue_days_mob','剩余总应收租金']
# df_mob_temp[(df_mob_temp.剩余总应收租金 == 0) & (df_mob_temp.overdue_days_mob > 0) & (df_mob_temp.应付日期 <= '2025-03-31')].shape
# df_mob_temp[(df_mob_temp.剩余总应收租金 == 0) & (df_mob_temp.overdue_days_mob > 0) & (df_mob_temp.应付日期 <= '2025-03-31')][var_1].tail(15)
# df_mob_temp[df_mob_temp.order_number == 'A20240328174912122'][var_1]

conditions_1 = (~df_mob_temp.订单状态_new.isin(['已买断','已完成','已结束']))
tmp9_1 = pd.DataFrame()
tmp9_1['滚动状态'] = ['1.正常_2.M1','2.M1_3.M2','3.M2_4.M3','4.M3_5.M4','5.M4_6.M5','6.M5_7.M6']

for var_1 in ['2022-07','2022-08','2022-09','2022-10','2022-11','2022-12'
                 ,'2023-01','2023-02','2023-03','2023-04','2023-05','2023-06','2023-07','2023-08','2023-09','2023-10','2023-11','2023-12'	
                 ,'2024-01','2024-02','2024-03','2024-04','2024-05','2024-06','2024-07','2024-08','2024-09','2024-10','2024-11','2024-12'
                 ,'2025-01','2025-02','2025-03','2025-04','2025-05','2025-06','2025-07']:
    var_2 = pd.to_datetime(var_1)
    # 往后推一个月
    var_3 = var_2 + pd.DateOffset(months=1)
    var_4 = pd.to_datetime(var_3).strftime('%Y-%m')

    temp1 = df_mob_temp[conditions_1 & (df_mob_temp.view_month == var_1)][['order_number','逾期阶段','剩余采购成本_202508']]
    temp2 = df_mob_temp[conditions_1 & (df_mob_temp.view_month == var_4)][['order_number','逾期阶段','剩余采购成本_202508']]

    temp1 = temp1.rename(columns = {'逾期阶段': '逾期阶段_c','剩余采购成本_202508': '剩余采购成本_c'})
    temp2 = temp2.rename(columns = {'逾期阶段': '逾期阶段_n','剩余采购成本_202508': '剩余采购成本_n'})

    temp3 = temp1.merge(temp2, on = 'order_number', how = 'inner')
    temp3['tmp'] = '_'
    temp3['滚动状态'] = temp3['逾期阶段_c'] + temp3['tmp'] + temp3['逾期阶段_n']
    temp3 = temp3.drop(columns = ['tmp'])
    temp4 = temp3[temp3.滚动状态.isin(['1.正常_2.M1','2.M1_3.M2','3.M2_4.M3','4.M3_5.M4','5.M4_6.M5','6.M5_7.M6'])].groupby(['滚动状态']).agg({'order_number':'count','剩余采购成本_n':'sum'}).reset_index()
    temp4 = temp4.rename(columns = {'order_number': '变坏的订单数'})
    temp4['逾期阶段_c'] = np.where(pd.notnull(temp4.滚动状态), temp4.滚动状态.str[:4], '-999')
    temp5 = temp3[temp3.逾期阶段_c.isin(['1.正常','2.M1','3.M2','4.M3','5.M4','6.M5'])].groupby(['逾期阶段_c']).agg({'order_number':'count','剩余采购成本_c':'sum'}).reset_index()
    temp6 = temp4.merge(temp5, on = '逾期阶段_c', how = 'left')

    temp6[var_1 + '_cnt'] = round(temp6['变坏的订单数'] / temp6['order_number'], 4)
    temp6[var_1 + '_amt'] = round(temp6['剩余采购成本_n'] / temp6['剩余采购成本_c'], 4)
    temp6 = temp6[['滚动状态',var_1 + '_cnt', var_1 + '_amt']]

    tmp9_1 = tmp9_1.merge(temp6, on = '滚动状态', how = 'left')

var_cnt = ['2022-07_cnt','2022-08_cnt','2022-09_cnt','2022-10_cnt','2022-11_cnt','2022-12_cnt'
                 ,'2023-01_cnt','2023-02_cnt','2023-03_cnt','2023-04_cnt','2023-05_cnt','2023-06_cnt','2023-07_cnt','2023-08_cnt','2023-09_cnt','2023-10_cnt','2023-11_cnt','2023-12_cnt'	
                 ,'2024-01_cnt','2024-02_cnt','2024-03_cnt','2024-04_cnt','2024-05_cnt','2024-06_cnt','2024-07_cnt','2024-08_cnt','2024-09_cnt','2024-10_cnt','2024-11_cnt','2024-12_cnt'
                 ,'2025-01_cnt','2025-02_cnt','2025-03_cnt','2025-04_cnt','2025-05_cnt','2025-06_cnt','2025-07_cnt']
var_amt = ['2022-07_amt','2022-08_amt','2022-09_amt','2022-10_amt','2022-11_amt','2022-12_amt'
                 ,'2023-01_amt','2023-02_amt','2023-03_amt','2023-04_amt','2023-05_amt','2023-06_amt','2023-07_amt','2023-08_amt','2023-09_amt','2023-10_amt','2023-11_amt','2023-12_amt'	
                 ,'2024-01_amt','2024-02_amt','2024-03_amt','2024-04_amt','2024-05_amt','2024-06_amt','2024-07_amt','2024-08_amt','2024-09_amt','2024-10_amt','2024-11_amt','2024-12_amt'
                 ,'2025-01_amt','2025-02_amt','2025-03_amt','2025-04_amt','2025-05_amt','2025-06_amt','2025-07_amt']
tmp9_1 = tmp9_1[['滚动状态'] + var_cnt + var_amt]
tmp9_1.to_excel('tmp9_1.xlsx', index = False)


### 4_8_5.首逾率

In [ ]:
# （1-1）按月维度统计，首逾发生在第2/3/4期的情况
# 2025.4 和 2025.5 的出库不含拒量

# 2025.8.22
# df_mob_temp 中为干净的宽表，不用做其他调整

df_xz_concat_1 = df_xz_concat.copy()
df_xz_concat_1 = df_xz_concat_1[~(df_xz_concat_1.search_time.isin(['2025-04','2025-05']) & (df_xz_concat_1.是否拒量 == 1))]

df_xz_concat_1['当前期数'] = df_xz_concat_1['当前期数'].astype(int)

df_yq_fq = df_xz_concat_1[["order_number","当前期数","逾期天数"]].pivot(index="order_number", columns="当前期数", values="逾期天数")
df_yq_fq = df_yq_fq.reset_index()
tmp1 = df_xz_concat_1[df_xz_concat_1.当前期数 == 1][['order_number','search_time','purchase_amount']]
tmp1 = tmp1.drop_duplicates(subset = 'order_number')
df_yq_fq = df_yq_fq.merge(tmp1, on = 'order_number', how = 'left')

df_yq_fq["FPD"] = df_yq_fq[2]
# note 20241222 修正SPD TPD QPD
df_yq_fq.loc[:,"SPD"] = np.where( df_yq_fq[2] == 0, df_yq_fq[3], 0)
df_yq_fq.loc[:,"TPD"] = np.where((df_yq_fq[2] == 0) & (df_yq_fq[3] == 0), df_yq_fq[4], 0)
df_yq_fq.shape

tmp = df_mob_temp.drop_duplicates(subset = 'order_number')[['order_number']]
tmp.shape

df_yq_fq = df_yq_fq.merge(tmp, on = 'order_number', how = 'inner')
df_yq_fq.shape
df_yq_fq[['order_number','search_time',2,3,4,'FPD','SPD','TPD']][:2]

(60087, 42)

(56678, 1)

(56678, 42)

,order_number,search_time,2,3,4,FPD,SPD,TPD
0,A202207050851373,2022-07,0.0,0.0,4.0,0.0,0.0,4.0
1,A202207131230485,2022-07,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# （1-2）订单维度-分子
tmp0 = df_yq_fq.groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp1 = df_yq_fq[df_yq_fq.FPD >= 1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp2 = df_yq_fq[df_yq_fq.FPD >= 7].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp3 = df_yq_fq[df_yq_fq.FPD >= 15].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp4 = df_yq_fq[df_yq_fq.FPD >= 30].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()

tmp5 = df_yq_fq[df_yq_fq.SPD >= 1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp6 = df_yq_fq[df_yq_fq.SPD >= 30].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp7 = df_yq_fq[df_yq_fq.TPD >= 1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp8 = df_yq_fq[df_yq_fq.TPD >= 30].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()

tmp1 = tmp1.rename(columns = {'order_number': 'FPD1'})
tmp2 = tmp2.rename(columns = {'order_number': 'FPD7'})
tmp3 = tmp3.rename(columns = {'order_number': 'FPD15'})
tmp4 = tmp4.rename(columns = {'order_number': 'FPD30'})
tmp5 = tmp5.rename(columns = {'order_number': 'SPD1'})
tmp6 = tmp6.rename(columns = {'order_number': 'SPD30'})
tmp7 = tmp7.rename(columns = {'order_number': 'TPD1'})
tmp8 = tmp8.rename(columns = {'order_number': 'TPD30'})

tmp10_1 = tmp0.merge(tmp1, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp2, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp3, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp4, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp5, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp6, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp7, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp8, on = 'search_time', how = 'left')

tmp10_1

,search_time,order_number,FPD1,FPD7,FPD15,FPD30,SPD1,SPD30,TPD1,TPD30
0,2022-07,21,4.0,3.0,2.0,2.0,3.0,2.0,1.0,NaN
1,2022-08,53,19.0,7.0,6.0,6.0,6.0,2.0,5.0,1.0
2,2022-09,40,9.0,6.0,3.0,3.0,6.0,1.0,1.0,NaN
3,2022-10,38,5.0,2.0,2.0,1.0,4.0,NaN,3.0,1.0
4,2022-11,41,10.0,3.0,3.0,3.0,3.0,NaN,2.0,NaN
5,2022-12,63,12.0,7.0,5.0,4.0,4.0,NaN,6.0,NaN
6,2023-01,212,50.0,16.0,14.0,11.0,27.0,6.0,12.0,2.0
7,2023-02,317,65.0,32.0,24.0,22.0,20.0,7.0,18.0,4.0
8,2023-03,302,31.0,23.0,18.0,15.0,34.0,10.0,29.0,5.0
9,2023-04,606,69.0,30.0,21.0,19.0,69.0,13.0,73.0,11.0


In [ ]:
# （2-1）每期的剩余采购成本取的是对应那期期末的剩余采购成本，不是那期逾期最终状态的剩余采购成本  

tmp1 = df_mob_temp[df_mob_temp.当前期数.isin([2,3,4])].pivot(index="order_number", columns="当前期数", values="剩余采购成本_202508")
tmp1 = tmp1.rename(columns = {2:'第2期剩余采购成本', 3:'第3期剩余采购成本', 4:'第4期剩余采购成本'})
df_yq_fq = df_yq_fq.merge(tmp1, on = 'order_number', how = 'left')

df_yq_fq['第2期剩余采购成本'] = df_yq_fq['第2期剩余采购成本'].fillna(0)
df_yq_fq['第3期剩余采购成本'] = df_yq_fq['第3期剩余采购成本'].fillna(0)
df_yq_fq['第4期剩余采购成本'] = df_yq_fq['第4期剩余采购成本'].fillna(0)
df_yq_fq[['order_number','search_time','FPD','SPD','TPD','第2期剩余采购成本','第3期剩余采购成本','第4期剩余采购成本']][:2]

# df_yq_fq[df_yq_fq.order_number == 'A2022073117434010'][['order_number','search_time','FPD','SPD','TPD','第2期剩余采购成本','第3期剩余采购成本','第4期剩余采购成本']]

,order_number,search_time,FPD,SPD,TPD,第2期剩余采购成本,第3期剩余采购成本,第4期剩余采购成本
0,A202207050851373,2022-07,0.0,0.0,4.0,4660.373838,4415.560757,4170.747676
1,A202207131230485,2022-07,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [ ]:
# （2-2）剩余采购成本维度-分子
tmp1 = df_yq_fq[df_yq_fq.FPD >= 1].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()
tmp2 = df_yq_fq[df_yq_fq.FPD >= 7].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()
tmp3 = df_yq_fq[df_yq_fq.FPD >= 15].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()
tmp4 = df_yq_fq[df_yq_fq.FPD >= 30].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()

tmp5 = df_yq_fq[df_yq_fq.SPD >= 1].groupby(['search_time']).agg({'第3期剩余采购成本': 'sum'}).reset_index()
tmp6 = df_yq_fq[df_yq_fq.SPD >= 30].groupby(['search_time']).agg({'第3期剩余采购成本': 'sum'}).reset_index()
tmp7 = df_yq_fq[df_yq_fq.TPD >= 1].groupby(['search_time']).agg({'第4期剩余采购成本': 'sum'}).reset_index()
tmp8 = df_yq_fq[df_yq_fq.TPD >= 30].groupby(['search_time']).agg({'第4期剩余采购成本': 'sum'}).reset_index()

tmp1 = tmp1.rename(columns = {'第2期剩余采购成本': 'FPD1'})
tmp2 = tmp2.rename(columns = {'第2期剩余采购成本': 'FPD7'})
tmp3 = tmp3.rename(columns = {'第2期剩余采购成本': 'FPD15'})
tmp4 = tmp4.rename(columns = {'第2期剩余采购成本': 'FPD30'})
tmp5 = tmp5.rename(columns = {'第3期剩余采购成本': 'SPD1'})
tmp6 = tmp6.rename(columns = {'第3期剩余采购成本': 'SPD30'})
tmp7 = tmp7.rename(columns = {'第4期剩余采购成本': 'TPD1'})
tmp8 = tmp8.rename(columns = {'第4期剩余采购成本': 'TPD30'})

tmp11_1 = tmp1.merge(tmp2, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp3, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp4, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp5, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp6, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp7, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp8, on = 'search_time', how = 'left')

tmp11_1

,search_time,FPD1,FPD7,FPD15,FPD30,SPD1,SPD30,TPD1,TPD30
0,2022-07,1.483385e+04,9129.077303,7572.850186,7572.850186,1.821877e+04,10687.238563,4.170748e+03,NaN
1,2022-08,5.237928e+04,19313.754995,17417.323324,17417.323324,1.950741e+04,10414.571854,1.339257e+04,2973.185022
2,2022-09,3.160795e+04,22094.532581,11612.181566,11612.181566,1.984061e+04,2340.862972,1.146300e+03,NaN
3,2022-10,1.473233e+04,6435.126610,6435.126610,5058.555181,7.469203e+03,NaN,8.890407e+03,2341.993806
4,2022-11,4.488237e+04,14817.002790,14817.002790,14817.002790,9.802156e+03,NaN,1.104112e+04,NaN
5,2022-12,5.234980e+04,34792.148911,23239.624464,18851.850979,1.286687e+04,NaN,2.521493e+04,NaN
6,2023-01,2.057737e+05,59949.095119,49336.693781,39141.451613,1.016840e+05,8941.752618,4.443421e+04,5573.418343
7,2023-02,2.065208e+05,102740.574915,77446.280716,72531.088008,5.722234e+04,27967.495637,5.399105e+04,12490.384165
8,2023-03,1.083885e+05,76539.142358,60754.318316,51964.531339,1.116855e+05,22426.743995,7.445283e+04,11633.063930
9,2023-04,2.608150e+05,114614.249394,79240.546447,75805.098930,2.512060e+05,50956.714559,2.512110e+05,32484.577723


### 4_8_6.资产分布

In [ ]:
# （1-1）宽表梳理
sql = '''-- 租后应收监控  
SELECT  tprm.order_number, tmu.id_card_num, tmu.province_name

from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
'''

df_pn = query(sql)
df_pn.shape

df_pn = df_pn.drop_duplicates(subset = ['order_number'])
df_pn.shape

df_mob_temp = df_mob_temp.merge(df_pn, on = 'order_number', how = 'left')

# A2024062722265551 的身份证为空，随机填充
df_mob_temp['id_card_num'] = np.where(df_mob_temp.id_card_num.isna(), '450422199705231266', df_mob_temp.id_card_num)

# 关联省、计算年龄、性别
# 定义一个函数来计算年龄
def calculate_age1(id_card, order_time):
    birth_year = int(id_card[6:10])
    birth_month = int(id_card[10:12])
    birth_day = int(id_card[12:14])
    birth_date = datetime(birth_year, birth_month, birth_day)
    age = order_time.year - birth_date.year - ((order_time.month, order_time.day) < (birth_date.month, birth_date.day))
    return age

df_mob_temp['age'] = df_mob_temp.apply(lambda row: calculate_age1(row['id_card_num'], row['下单日期']), axis=1)
df_mob_temp['性别'] = df_mob_temp['id_card_num'].apply(lambda x:clean.get_gender(x))

df_mob_temp.shape
df_mob_temp[:1]

(503343, 53)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1,是否拒量,price,residue_money,unrecovered_cost,剩余采购成本_new,is_relet,is_relet_max,期数,当前累计实付采购成本,采购成本分摊,当前累计实付采购成本_all,采购成本分摊_new,剩余采购成本_202508,逾期阶段,id_card_num,province_name,age,性别
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0,0,0.0,0.0,-3026.92,4792.05,0.0,2.0,12.0,244.813081,2937.756972,244.813081,5150.0,4905.186919,1.正常,362202199805282550,江西省,24,男


In [ ]:
# （2）画像字段统计
tmp1 = df_mob_temp[(df_mob_temp.当前期数 == 1)][['order_number','search_time','purchase_amount','最大期数_Bin','province_name','性别','age','采购成本分摊_new']]
tmp1 = tmp1.drop_duplicates(subset = 'order_number')
tmp1.shape
tmp1['purchase_amount_Bin'] = tmp1.apply(lambda x: '1.(0,3000)' if x.purchase_amount >= 0 and x.purchase_amount < 3000
                                         else '2.[3000,5000)' if x.purchase_amount >= 3000 and x.purchase_amount < 5000
                                         else '3.[5000,10000)' if x.purchase_amount >= 5000 and x.purchase_amount < 10000
                                         else '4.[10000,+)' if x.purchase_amount >= 10000
                                         else 'other', axis = 1)
tmp1['age_Bin'] = tmp1.apply(lambda x: '1.(18,22]' if x.age <= 22
                             else '2.(22,30]' if x.age > 22 and x.age <= 30
                             else '3.(30,40]' if x.age > 30 and x.age <= 40
                             else '4.(40,50]' if x.age > 40 and x.age <= 50
                             else '5.[50+)' if x.age > 50
                             else 'other',axis = 1)


tmp1[tmp1.search_time <= '2025-07'].groupby(['最大期数_Bin']).agg({'purchase_amount': 'sum', '采购成本分摊_new': 'sum','order_number': 'count'})
tmp1[tmp1.search_time <= '2025-07'].groupby(['purchase_amount_Bin']).agg({'order_number': 'count','purchase_amount': 'sum'})
tmp1[tmp1.search_time <= '2025-07'].groupby(['age_Bin']).agg({'order_number': 'count','purchase_amount': 'sum'})
tmp1[tmp1.search_time <= '2025-07'].groupby(['性别']).agg({'order_number': 'count','purchase_amount': 'sum'})
tmp1[tmp1.search_time <= '2025-07'].groupby(['province_name']).agg({'purchase_amount': 'sum', '采购成本分摊_new': 'sum','order_number': 'count'})

(56678, 8)

,purchase_amount,采购成本分摊_new,order_number
最大期数_Bin,,,
12个月,3.564166e+08,1.928757e+08,45844
18个月,8.503863e+06,8.503863e+06,1214
24个月,4.374483e+07,4.354219e+07,6026


,order_number,purchase_amount
purchase_amount_Bin,,
"1.(0,3000)",2702,1.934179e+06
"2.[3000,5000)",1751,7.419632e+06
"3.[5000,10000)",47232,3.846535e+08
"4.[10000,+)",1399,1.465798e+07


,order_number,purchase_amount
age_Bin,,
"1.(18,22]",12677,9.181752e+07
"2.(22,30]",19906,1.540051e+08
"3.(30,40]",15831,1.257392e+08
"4.(40,50]",4635,3.709074e+07
5.[50+),35,1.267100e+04


,order_number,purchase_amount
性别,,
女,17600,1.408057e+08
男,35484,2.678596e+08


,purchase_amount,采购成本分摊_new,order_number
province_name,,,
,806078.00,5.089470e+05,102
上海市,9362865.12,6.084529e+06,1224
中华人民共和国,24924.00,1.704721e+04,3
云南省,10367248.68,6.641903e+06,1368
内蒙古自治区,5824620.00,3.713054e+06,743
北京市,7724148.17,4.954015e+06,999
吉林省,6338111.00,3.982120e+06,809
四川省,17594180.00,1.137367e+07,2350
天津市,3587087.75,2.283997e+06,470


## 4_9.统计（8.28 new）

### 4_9_0.采购成本分摊口径梳理

<!-- 202508采购成本分摊逻辑：
1、租完即送--采购成本平均分摊到每期，即每期应还采购成本=采购成本/期数，每期应还采购成本=月租金*（采购成本/签约价）
2、租完归还
（1）归还（/买断）：分摊系数=总租金/签约价，租赁期间的每期应还采购成本=采购成本*分摊系数/期数，如签约价是13000、采购成本是9000、总租金是6500、买断尾款是6500、租期12期，则每期应还采购成本(375)=6500/13000*9000/12
（2）续租：首租的每期应还采购成本如（1），续租部分的每期应还采购成本=（总采购成本-首租采购成本）/续租期数 -->

In [ ]:
# df_mob_temp 表的字段解析
# 买断价：区分首租、续租（其中续租的包含首租的）
# 总应收：区分首租、续租
# 应付总租金：区分首租、续租
# 当前累计实付租金：区分首租、续租
# is_relet >= 1：为续租

# 增加/调整字段：
# （5）增加 采购成本分摊（区分首租续租）
# （6）增加 当前累计实付采购成本（区分首租续租）
# （7）增加 剩余采购成本_202508

# 新口径-vintage（续租跟首租的合并为一个订单，但分子不区分首续租） --本次使用：
# （1）分母：采购成本
# （2）分子：分摊后的剩余采购成本

# 新口径-vintage（续租跟首租的拆开为两个订单，分子区分首租和续租） --本次未使用：
# （1）分母：租送-采购成本，租归且有续租的订单-首租的为一个订单、续租的为新订单，租归但未续租的订单-分摊后的采购成本

In [ ]:
df_mob_temp.shape
df_mob_sz = df_mob_temp[df_mob_temp.is_relet == 0]
df_mob_xz = df_mob_temp[df_mob_temp.is_relet > 0]
df_mob_sz.shape
df_mob_xz.shape

# df_mob_sz_zs = df_mob_sz[df_mob_sz.租赁方案 == '租完即送']
# df_mob_sz_zg = df_mob_sz[df_mob_sz.租赁方案 == '租完归还']

# 推算后，采购成本占的权重 = 采购成本/买断价
# df_mob_sz_zs['当前累计实付采购成本'] = df_mob_sz_zs['当前累计实付租金']*(df_mob_sz_zs['purchase_amount']/df_mob_sz_zs['买断价'])
# df_mob_sz_zg['当前累计实付采购成本'] = df_mob_sz_zg['当前累计实付租金']*(df_mob_sz_zg['purchase_amount']/df_mob_sz_zg['买断价'])

# 买断价：以首租的买断价为准
df_mob_sz['当前累计实付采购成本'] = df_mob_sz['当前累计实付租金']*(df_mob_sz['purchase_amount']/df_mob_sz['买断价'])
df_mob_sz['采购成本分摊'] = df_mob_sz['purchase_amount']*(df_mob_sz['应付总租金']/df_mob_sz['买断价'])
df_mob_sz['采购成本分摊'] = np.where(df_mob_sz.租赁方案 == '租完即送', df_mob_sz.purchase_amount, df_mob_sz.采购成本分摊)
df_mob_sz['tmp1'] = df_mob_sz['purchase_amount'] - df_mob_sz['采购成本分摊']

temp4 = df_mob_sz.groupby(['order_number']).agg({'tmp1': 'max'}).reset_index()
temp4 = temp4.rename(columns = {'tmp1': '采购成本分摊'})

df_mob_xz = df_mob_xz.merge(temp4, on = 'order_number', how = 'left')
# df_mob_xz['当前累计实付采购成本'] = df_mob_xz['当前累计实付租金']*(df_mob_xz['采购成本分摊']/df_mob_xz['买断价'])
df_mob_xz['当前累计实付采购成本'] = df_mob_xz['当前累计实付租金']*(df_mob_xz['采购成本分摊']/df_mob_xz['应付总租金'])

df_mob_sz = df_mob_sz.drop(columns = ['tmp1'])

# view_month 为系统月的 overdue_days_mob 有误，不要使用
# 当前累计实付租金 等指标区分了首租和续租，要合并起来
temp5 = df_mob_sz.groupby(['order_number']).agg({'当前累计实付采购成本': 'max'}).reset_index()
temp5 = temp5.rename(columns = {'当前累计实付采购成本': '当前累计实付采购成本_首租'})
df_mob_xz = df_mob_xz.merge(temp5, on = 'order_number', how = 'left')

df_mob_xz['当前累计实付采购成本_首租'] = df_mob_xz['当前累计实付采购成本_首租'].fillna(0)
df_mob_xz['当前累计实付采购成本_all'] = df_mob_xz['当前累计实付采购成本_首租'] + df_mob_xz['当前累计实付采购成本']
df_mob_xz = df_mob_xz.drop(columns = ['当前累计实付采购成本_首租'])

df_mob_sz['当前累计实付采购成本_all'] = df_mob_sz['当前累计实付采购成本']


df_mob_temp = pd.concat([df_mob_sz, df_mob_xz])
# vintage--分母：租送-采购成本，租归且有续租的订单-采购成本，租归但未续租的订单-分摊后的采购成本
df_mob_temp['采购成本分摊_new'] = np.where(df_mob_temp.is_relet_max > 0, df_mob_temp.purchase_amount, df_mob_temp.采购成本分摊)
# df_mob_temp['剩余采购成本_202508'] = df_mob_temp['采购成本分摊_new'] - df_mob_temp['当前累计实付采购成本_all']
# df_mob_temp['剩余采购成本_202508'] = df_mob_temp['采购成本分摊'] - df_mob_temp['当前累计实付采购成本']
df_mob_temp['剩余采购成本_202508'] = df_mob_temp['purchase_amount'] - df_mob_temp['当前累计实付采购成本_all']

df_mob_temp.shape
df_mob_temp[:1]

# var_1 = ['order_number','应付日期','当前期数','purchase_amount','当前累计实付租金','当前累计实付租金_all','剩余采购成本'
#          ,'采购成本分摊','采购成本分摊_new','当前累计实付采购成本','当前累计实付采购成本_all','剩余采购成本_202508','overdue_days_mob']

# df_mob_temp[df_mob_temp.order_number == 'A2023090814292948'][var_1]
# A202207050851373  --续租，提前买断
# A202208040916342  --归还逾期
# A202503020002175  --租归，应还日期为系统月的 overdue_days_mob 有误，不要使用
# A2024040309475173  --严重逾期
# A2024051707383726  --续租
# A2023090814292948   --租送

# df_mob_temp[df_mob_temp.租赁方案 == '租完即送'][['order_number']][30000:30005]

# df_mob_temp[(df_mob_temp.租赁方案 == '租完即送')&(df_mob_temp.purchase_amount != df_mob_temp.采购成本分摊_new)].shape
# df_mob_temp[(df_mob_temp.租赁方案 == '租完归还')&(df_mob_temp.is_relet_max > 0)&(df_mob_temp.purchase_amount != df_mob_temp.采购成本分摊_new)].shape
# df_mob_temp[(df_mob_temp.租赁方案 == '租完归还')&(df_mob_temp.is_relet_max == 0)&(df_mob_temp.purchase_amount == df_mob_temp.采购成本分摊_new)
#             &(df_mob_temp.purchase_amount >0)&(df_mob_temp.买断价 != df_mob_temp.应付总租金)].shape

(552070, 49)

(502611, 49)

(49459, 49)

(552070, 54)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1,是否拒量,price,residue_money,unrecovered_cost,剩余采购成本_old,剩余采购成本_new,is_relet,is_relet_max,期数,买断价_max,id_card_num,province_name,age,性别,当前累计实付采购成本,采购成本分摊,当前累计实付采购成本_all,采购成本分摊_new,剩余采购成本_202508
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0,0,0.0,0.0,-3026.92,4792.05,4792.05,0.0,2.0,12.0,8176.92,362202199805282550,江西省,24,男,244.813081,2937.756972,244.813081,5150.0,4905.186919


### 4_9_1.余额衰减

In [ ]:
# MOB：首期，用应付日期，不是实付日期，故月末下单的订单，没有MOB0，从MOB1开始（本次）；若首期按实际支付日期计算的话，月末下单的订单，其实是没有MOB1的
MOB_list = ['MOB0','MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
conditions_1 = (df_mob_temp.search_time <= '2025-07')
conditions_2 = (df_mob_temp.view_month <= '2025-07')

tmp1_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].groupby(['search_time']).agg({'purchase_amount': 'sum'}).reset_index()
tmp1_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
tmp1_2 = tmp1_2[['search_time'] + MOB_list]
tmp1_1 = tmp1_1.merge(tmp1_2, on = 'search_time', how = 'left')
tmp1_1.to_excel('tmp1_1.xlsx', index = False)
# # 增加订单维度
# tmp1_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp1_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp1_2 = tmp1_2[['search_time'] + MOB_list]
# tmp1_1 = tmp1_1.merge(tmp1_2, on = 'search_time', how = 'left')
# tmp1_1.to_excel('tmp1_2.xlsx', index = False)

tmp2_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '12个月')].groupby(['search_time']).agg({'purchase_amount': 'sum'}).reset_index()
tmp2_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '12个月')],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
tmp2_2 = tmp2_2[['search_time'] + MOB_list]
tmp2_1 = tmp2_1.merge(tmp2_2, on = 'search_time', how = 'left')
tmp2_1.to_excel('tmp2_1.xlsx', index = False)
# # 增加订单维度
# tmp2_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '12个月')].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp2_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '12个月')],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp2_2 = tmp2_2[['search_time'] + MOB_list]
# tmp2_1 = tmp2_1.merge(tmp2_2, on = 'search_time', how = 'left')
# tmp2_1.to_excel('tmp2_2.xlsx', index = False)


MOB_list = ['MOB0','MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
tmp3_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '18个月')].groupby(['search_time']).agg({'purchase_amount': 'sum'}).reset_index()
tmp3_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '18个月')],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
tmp3_2 = tmp3_2[['search_time'] + MOB_list]
tmp3_1 = tmp3_1.merge(tmp3_2, on = 'search_time', how = 'left')
tmp3_1.to_excel('tmp3_1.xlsx', index = False)
# # 增加订单维度
# tmp3_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '18个月')].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp3_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '18个月')],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp3_2 = tmp3_2[['search_time'] + MOB_list]
# tmp3_1 = tmp3_1.merge(tmp3_2, on = 'search_time', how = 'left')
# tmp3_1.to_excel('tmp3_2.xlsx', index = False)

MOB_list = ['MOB0','MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
tmp4_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '24个月')].groupby(['search_time']).agg({'purchase_amount': 'sum'}).reset_index()
tmp4_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '24个月')],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
tmp4_2 = tmp4_2[['search_time'] + MOB_list]
tmp4_1 = tmp4_1.merge(tmp4_2, on = 'search_time', how = 'left')
tmp4_1.to_excel('tmp4_1.xlsx', index = False)
# # 增加订单维度
# tmp4_1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1 & (df_mob_temp.最大期数_Bin == '24个月')].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# tmp4_2 = pd.pivot_table(df_mob_temp[conditions_1 & conditions_2 & (df_mob_temp.最大期数_Bin == '24个月')],index='search_time',columns='MOB_new_1',values='order_number',aggfunc='count').reset_index()
# tmp4_2 = tmp4_2[['search_time'] + MOB_list]
# tmp4_1 = tmp4_1.merge(tmp4_2, on = 'search_time', how = 'left')
# tmp4_1.to_excel('tmp4_2.xlsx', index = False)

# 抖音的有首租是24期的
# df_mob_temp[(df_mob_temp.最大期数_Bin == '24个月')&(df_mob_temp.search_time == '2025-05')][var_1]

### 4_9_2.每月截面数据

In [ ]:
# 每月末采购价各逾期阶段的数据
df_mob_temp['逾期阶段'] = df_mob_temp.apply(lambda x: '1.正常' if x.overdue_days_mob == 0
                                        else '2.M1' if x.overdue_days_mob >= 1 and x.overdue_days_mob <= 30
                                        else '3.M2' if x.overdue_days_mob >= 31 and x.overdue_days_mob <= 60
                                        else '4.M3' if x.overdue_days_mob >= 61 and x.overdue_days_mob <= 90
                                        else '5.M4' if x.overdue_days_mob >= 91 and x.overdue_days_mob <= 120
                                        else '6.M5' if x.overdue_days_mob >= 121 and x.overdue_days_mob <= 150
                                        else '7.M6' if x.overdue_days_mob >= 151 and x.overdue_days_mob <= 180
                                        else '8.M6+' if x.overdue_days_mob > 180
                                        else '9.other', axis = 1)

In [ ]:
# 2023.10、2023.11、2023.12 有差异
tmp5_1 = pd.pivot_table(df_mob_temp,index='逾期阶段',columns='view_month',values='剩余采购成本_202508',aggfunc='sum').reset_index()
tmp5_1.to_excel('tmp5_1.xlsx', index = False)

### 4_9_3.vintage（金额比例）

In [ ]:
# 分子统计
conditions_2 = (df_mob_temp.view_month <= '2025-07')
conditions_3 = (df_mob_temp.search_time <= '2025-05')
conditions_4 = (df_mob_temp.overdue_days_mob > 30)

tmp6_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_3 & conditions_4],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
MOB_list = ['MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
tmp6_1 = tmp6_1[['search_time'] + MOB_list]
tmp6_1.to_excel('tmp6_1.xlsx', index = False)

In [ ]:
conditions_5 = (df_mob_temp.search_time <= '2025-04')
conditions_6 = (df_mob_temp.overdue_days_mob > 60)

tmp7_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_5 & conditions_6],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
MOB_list = ['MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
tmp7_1 = tmp7_1[['search_time'] + MOB_list]
tmp7_1.to_excel('tmp7_1.xlsx', index = False)

In [ ]:
conditions_7 = (df_mob_temp.search_time <= '2025-03')
conditions_8 = (df_mob_temp.overdue_days_mob > 90)

tmp8_1 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_7 & conditions_8],index='search_time',columns='MOB_new_1',values='剩余采购成本_202508',aggfunc='sum').reset_index()
MOB_list = ['MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24']
tmp8_1 = tmp8_1[['search_time'] + MOB_list]
tmp8_1.to_excel('tmp8_1.xlsx', index = False)

### 4_9_4.迁徙率

In [ ]:
# 正常客群比之前京享租统计rollrate的时候会多一点，因为之前分首租和续租，首租的第12期订单状态为 已完成，本次不分首租续租，且首租的第12期状态更新为还款中

# 2025.4.10
# df_mob_temp 中 应付日期为2025.4的，overdue_days_mob 皆为 30，加工口径需修改，本次不影响 rollrate
# var_1 = ['order_number','下单日期','当前期数','应付日期','实付日期new','当前应付租金','实付金额','overdue_days_mob','剩余总应收租金']
# df_mob_temp[(df_mob_temp.剩余总应收租金 == 0) & (df_mob_temp.overdue_days_mob > 0) & (df_mob_temp.应付日期 <= '2025-03-31')].shape
# df_mob_temp[(df_mob_temp.剩余总应收租金 == 0) & (df_mob_temp.overdue_days_mob > 0) & (df_mob_temp.应付日期 <= '2025-03-31')][var_1].tail(15)
# df_mob_temp[df_mob_temp.order_number == 'A20240328174912122'][var_1]

conditions_1 = (~df_mob_temp.订单状态_new.isin(['已买断','已完成','已结束']))
tmp9_1 = pd.DataFrame()
tmp9_1['滚动状态'] = ['1.正常_2.M1','2.M1_3.M2','3.M2_4.M3','4.M3_5.M4','5.M4_6.M5','6.M5_7.M6']

for var_1 in ['2022-07','2022-08','2022-09','2022-10','2022-11','2022-12'
                 ,'2023-01','2023-02','2023-03','2023-04','2023-05','2023-06','2023-07','2023-08','2023-09','2023-10','2023-11','2023-12'	
                 ,'2024-01','2024-02','2024-03','2024-04','2024-05','2024-06','2024-07','2024-08','2024-09','2024-10','2024-11','2024-12'
                 ,'2025-01','2025-02','2025-03','2025-04','2025-05','2025-06','2025-07']:
    var_2 = pd.to_datetime(var_1)
    # 往后推一个月
    var_3 = var_2 + pd.DateOffset(months=1)
    var_4 = pd.to_datetime(var_3).strftime('%Y-%m')

    temp1 = df_mob_temp[conditions_1 & (df_mob_temp.view_month == var_1)][['order_number','逾期阶段','剩余采购成本_202508']]
    temp2 = df_mob_temp[conditions_1 & (df_mob_temp.view_month == var_4)][['order_number','逾期阶段','剩余采购成本_202508']]

    temp1 = temp1.rename(columns = {'逾期阶段': '逾期阶段_c','剩余采购成本_202508': '剩余采购成本_c'})
    temp2 = temp2.rename(columns = {'逾期阶段': '逾期阶段_n','剩余采购成本_202508': '剩余采购成本_n'})

    temp3 = temp1.merge(temp2, on = 'order_number', how = 'inner')
    temp3['tmp'] = '_'
    temp3['滚动状态'] = temp3['逾期阶段_c'] + temp3['tmp'] + temp3['逾期阶段_n']
    temp3 = temp3.drop(columns = ['tmp'])
    temp4 = temp3[temp3.滚动状态.isin(['1.正常_2.M1','2.M1_3.M2','3.M2_4.M3','4.M3_5.M4','5.M4_6.M5','6.M5_7.M6'])].groupby(['滚动状态']).agg({'order_number':'count','剩余采购成本_n':'sum'}).reset_index()
    temp4 = temp4.rename(columns = {'order_number': '变坏的订单数'})
    temp4['逾期阶段_c'] = np.where(pd.notnull(temp4.滚动状态), temp4.滚动状态.str[:4], '-999')
    temp5 = temp3[temp3.逾期阶段_c.isin(['1.正常','2.M1','3.M2','4.M3','5.M4','6.M5'])].groupby(['逾期阶段_c']).agg({'order_number':'count','剩余采购成本_c':'sum'}).reset_index()
    temp6 = temp4.merge(temp5, on = '逾期阶段_c', how = 'left')

    temp6[var_1 + '_cnt'] = round(temp6['变坏的订单数'] / temp6['order_number'], 4)
    temp6[var_1 + '_amt'] = round(temp6['剩余采购成本_n'] / temp6['剩余采购成本_c'], 4)
    temp6 = temp6[['滚动状态',var_1 + '_cnt', var_1 + '_amt']]

    tmp9_1 = tmp9_1.merge(temp6, on = '滚动状态', how = 'left')

var_cnt = ['2022-07_cnt','2022-08_cnt','2022-09_cnt','2022-10_cnt','2022-11_cnt','2022-12_cnt'
                 ,'2023-01_cnt','2023-02_cnt','2023-03_cnt','2023-04_cnt','2023-05_cnt','2023-06_cnt','2023-07_cnt','2023-08_cnt','2023-09_cnt','2023-10_cnt','2023-11_cnt','2023-12_cnt'	
                 ,'2024-01_cnt','2024-02_cnt','2024-03_cnt','2024-04_cnt','2024-05_cnt','2024-06_cnt','2024-07_cnt','2024-08_cnt','2024-09_cnt','2024-10_cnt','2024-11_cnt','2024-12_cnt'
                 ,'2025-01_cnt','2025-02_cnt','2025-03_cnt','2025-04_cnt','2025-05_cnt','2025-06_cnt','2025-07_cnt']
var_amt = ['2022-07_amt','2022-08_amt','2022-09_amt','2022-10_amt','2022-11_amt','2022-12_amt'
                 ,'2023-01_amt','2023-02_amt','2023-03_amt','2023-04_amt','2023-05_amt','2023-06_amt','2023-07_amt','2023-08_amt','2023-09_amt','2023-10_amt','2023-11_amt','2023-12_amt'	
                 ,'2024-01_amt','2024-02_amt','2024-03_amt','2024-04_amt','2024-05_amt','2024-06_amt','2024-07_amt','2024-08_amt','2024-09_amt','2024-10_amt','2024-11_amt','2024-12_amt'
                 ,'2025-01_amt','2025-02_amt','2025-03_amt','2025-04_amt','2025-05_amt','2025-06_amt','2025-07_amt']
tmp9_1 = tmp9_1[['滚动状态'] + var_cnt + var_amt]
tmp9_1.to_excel('tmp9_1.xlsx', index = False)


### 4_9_5.首逾率

In [ ]:
# （1-1）按月维度统计，首逾发生在第2/3/4期的情况
# 2025.4 和 2025.5 的出库不含拒量

# 2025.8.22
# df_mob_temp 中为干净的宽表，不用做其他调整

df_xz_concat_1 = df_xz_concat.copy()
df_xz_concat_1 = df_xz_concat_1[~(df_xz_concat_1.search_time.isin(['2025-04','2025-05']) & (df_xz_concat_1.是否拒量 == 1))]

df_xz_concat_1['当前期数'] = df_xz_concat_1['当前期数'].astype(int)

df_yq_fq = df_xz_concat_1[["order_number","当前期数","逾期天数"]].pivot(index="order_number", columns="当前期数", values="逾期天数")
df_yq_fq = df_yq_fq.reset_index()
tmp1 = df_xz_concat_1[df_xz_concat_1.当前期数 == 1][['order_number','search_time','purchase_amount']]
tmp1 = tmp1.drop_duplicates(subset = 'order_number')
df_yq_fq = df_yq_fq.merge(tmp1, on = 'order_number', how = 'left')

df_yq_fq["FPD"] = df_yq_fq[2]
# note 20241222 修正SPD TPD QPD
df_yq_fq.loc[:,"SPD"] = np.where( df_yq_fq[2] == 0, df_yq_fq[3], 0)
df_yq_fq.loc[:,"TPD"] = np.where((df_yq_fq[2] == 0) & (df_yq_fq[3] == 0), df_yq_fq[4], 0)
df_yq_fq.shape

tmp = df_mob_temp.drop_duplicates(subset = 'order_number')[['order_number']]
tmp.shape

df_yq_fq = df_yq_fq.merge(tmp, on = 'order_number', how = 'inner')
df_yq_fq.shape
df_yq_fq[['order_number','search_time',2,3,4,'FPD','SPD','TPD']][:2]

(60964, 42)

(57591, 1)

(57591, 42)

,order_number,search_time,2,3,4,FPD,SPD,TPD
0,A202207050851373,2022-07,0.0,0.0,4.0,0.0,0.0,4.0
1,A202207131230485,2022-07,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# （1-2）订单维度-分子
tmp0 = df_yq_fq.groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp1 = df_yq_fq[df_yq_fq.FPD >= 1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp2 = df_yq_fq[df_yq_fq.FPD >= 7].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp3 = df_yq_fq[df_yq_fq.FPD >= 15].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp4 = df_yq_fq[df_yq_fq.FPD >= 30].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()

tmp5 = df_yq_fq[df_yq_fq.SPD >= 1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp6 = df_yq_fq[df_yq_fq.SPD >= 30].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp7 = df_yq_fq[df_yq_fq.TPD >= 1].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
tmp8 = df_yq_fq[df_yq_fq.TPD >= 30].groupby(['search_time']).agg({'order_number': 'count'}).reset_index()

tmp1 = tmp1.rename(columns = {'order_number': 'FPD1'})
tmp2 = tmp2.rename(columns = {'order_number': 'FPD7'})
tmp3 = tmp3.rename(columns = {'order_number': 'FPD15'})
tmp4 = tmp4.rename(columns = {'order_number': 'FPD30'})
tmp5 = tmp5.rename(columns = {'order_number': 'SPD1'})
tmp6 = tmp6.rename(columns = {'order_number': 'SPD30'})
tmp7 = tmp7.rename(columns = {'order_number': 'TPD1'})
tmp8 = tmp8.rename(columns = {'order_number': 'TPD30'})

tmp10_1 = tmp0.merge(tmp1, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp2, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp3, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp4, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp5, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp6, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp7, on = 'search_time', how = 'left')
tmp10_1 = tmp10_1.merge(tmp8, on = 'search_time', how = 'left')

tmp10_1

,search_time,order_number,FPD1,FPD7,FPD15,FPD30,SPD1,SPD30,TPD1,TPD30
0,2022-07,21,4.0,3.0,2.0,2.0,3.0,2.0,1.0,NaN
1,2022-08,53,19.0,7.0,6.0,6.0,6.0,2.0,5.0,1.0
2,2022-09,40,9.0,6.0,3.0,3.0,6.0,1.0,1.0,NaN
3,2022-10,38,5.0,2.0,2.0,1.0,4.0,NaN,3.0,1.0
4,2022-11,41,10.0,3.0,3.0,3.0,3.0,NaN,2.0,NaN
5,2022-12,63,12.0,7.0,5.0,4.0,4.0,NaN,6.0,NaN
6,2023-01,212,50.0,16.0,14.0,11.0,27.0,6.0,12.0,2.0
7,2023-02,317,65.0,32.0,24.0,22.0,20.0,7.0,18.0,4.0
8,2023-03,302,31.0,23.0,18.0,15.0,34.0,10.0,29.0,5.0
9,2023-04,606,69.0,30.0,21.0,19.0,69.0,13.0,73.0,11.0


In [ ]:
# （2-1）每期的剩余采购成本取的是对应那期期末的剩余采购成本，不是那期逾期最终状态的剩余采购成本  

tmp1 = df_mob_temp[df_mob_temp.当前期数.isin([2,3,4])].pivot(index="order_number", columns="当前期数", values="剩余采购成本_202508")
tmp1 = tmp1.rename(columns = {2:'第2期剩余采购成本', 3:'第3期剩余采购成本', 4:'第4期剩余采购成本'})
df_yq_fq = df_yq_fq.merge(tmp1, on = 'order_number', how = 'left')

df_yq_fq['第2期剩余采购成本'] = df_yq_fq['第2期剩余采购成本'].fillna(0)
df_yq_fq['第3期剩余采购成本'] = df_yq_fq['第3期剩余采购成本'].fillna(0)
df_yq_fq['第4期剩余采购成本'] = df_yq_fq['第4期剩余采购成本'].fillna(0)
df_yq_fq[['order_number','search_time','FPD','SPD','TPD','第2期剩余采购成本','第3期剩余采购成本','第4期剩余采购成本']][:2]

# df_yq_fq[df_yq_fq.order_number == 'A2022073117434010'][['order_number','search_time','FPD','SPD','TPD','第2期剩余采购成本','第3期剩余采购成本','第4期剩余采购成本']]

,order_number,search_time,FPD,SPD,TPD,第2期剩余采购成本,第3期剩余采购成本,第4期剩余采购成本
0,A202207050851373,2022-07,0.0,0.0,4.0,4660.373838,4415.560757,4170.747676
1,A202207131230485,2022-07,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [ ]:
# （2-2）剩余采购成本维度-分子
tmp1 = df_yq_fq[df_yq_fq.FPD >= 1].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()
tmp2 = df_yq_fq[df_yq_fq.FPD >= 7].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()
tmp3 = df_yq_fq[df_yq_fq.FPD >= 15].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()
tmp4 = df_yq_fq[df_yq_fq.FPD >= 30].groupby(['search_time']).agg({'第2期剩余采购成本': 'sum'}).reset_index()

tmp5 = df_yq_fq[df_yq_fq.SPD >= 1].groupby(['search_time']).agg({'第3期剩余采购成本': 'sum'}).reset_index()
tmp6 = df_yq_fq[df_yq_fq.SPD >= 30].groupby(['search_time']).agg({'第3期剩余采购成本': 'sum'}).reset_index()
tmp7 = df_yq_fq[df_yq_fq.TPD >= 1].groupby(['search_time']).agg({'第4期剩余采购成本': 'sum'}).reset_index()
tmp8 = df_yq_fq[df_yq_fq.TPD >= 30].groupby(['search_time']).agg({'第4期剩余采购成本': 'sum'}).reset_index()

tmp1 = tmp1.rename(columns = {'第2期剩余采购成本': 'FPD1'})
tmp2 = tmp2.rename(columns = {'第2期剩余采购成本': 'FPD7'})
tmp3 = tmp3.rename(columns = {'第2期剩余采购成本': 'FPD15'})
tmp4 = tmp4.rename(columns = {'第2期剩余采购成本': 'FPD30'})
tmp5 = tmp5.rename(columns = {'第3期剩余采购成本': 'SPD1'})
tmp6 = tmp6.rename(columns = {'第3期剩余采购成本': 'SPD30'})
tmp7 = tmp7.rename(columns = {'第4期剩余采购成本': 'TPD1'})
tmp8 = tmp8.rename(columns = {'第4期剩余采购成本': 'TPD30'})

tmp11_1 = tmp1.merge(tmp2, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp3, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp4, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp5, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp6, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp7, on = 'search_time', how = 'left')
tmp11_1 = tmp11_1.merge(tmp8, on = 'search_time', how = 'left')

tmp11_1

,search_time,FPD1,FPD7,FPD15,FPD30,SPD1,SPD30,TPD1,TPD30
0,2022-07,2.011999e+04,1.441522e+04,1.152142e+04,1.152142e+04,2.192677e+04,14395.243102,4.170748e+03,NaN
1,2022-08,1.006879e+05,4.439516e+04,3.970598e+04,3.970598e+04,3.556893e+04,14573.232518,2.471507e+04,7110.112568
2,2022-09,5.953628e+04,3.922856e+04,2.328755e+04,2.328755e+04,3.123586e+04,5923.954733,2.201306e+03,NaN
3,2022-10,2.770217e+04,1.123528e+04,1.123528e+04,8.503707e+03,1.103821e+04,NaN,1.613518e+04,4569.917899
4,2022-11,6.241504e+04,2.171324e+04,2.171324e+04,2.171324e+04,2.053163e+04,NaN,1.495681e+04,NaN
5,2022-12,8.348653e+04,5.472872e+04,3.934569e+04,3.096039e+04,2.413256e+04,NaN,3.313137e+04,NaN
6,2023-01,2.891509e+05,9.052284e+04,7.648995e+04,6.269557e+04,1.385061e+05,13695.975912,6.592379e+04,13341.913569
7,2023-02,3.858903e+05,1.983706e+05,1.556025e+05,1.491026e+05,1.130086e+05,50114.521449,9.309938e+04,22403.669671
8,2023-03,1.929962e+05,1.435504e+05,1.139971e+05,9.994415e+04,1.786738e+05,52330.857079,1.424050e+05,29595.600812
9,2023-04,4.513221e+05,2.051505e+05,1.398825e+05,1.302675e+05,4.073031e+05,86045.475352,4.133128e+05,68053.665189


### 4_9_6.资产分布

In [225]:
# （1-1）按照身份证提前省份
def province_func(xx):
    if xx == 11:
        return '北京'
    elif xx == 12:
        return '天津'
    elif xx == 13:
        return '河北'
    elif xx == 14:
        return '山西'
    elif xx == 15:
        return '内蒙古'
    elif xx == 21:
        return '辽宁'
    elif xx == 22:
        return '吉林'
    elif xx == 23:
        return '黑龙江'
    elif xx == 31:
        return '上海'
    elif xx == 32:
        return '江苏'
    elif xx == 33:
        return '浙江'
    elif xx == 34:
        return '安徽'
    elif xx == 35:
        return '福建'
    elif xx == 36:
        return '江西'
    elif xx == 37:
        return '山东'
    elif xx == 41:
        return '河南'
    elif xx == 42:
        return '湖北'
    elif xx == 43:
        return '湖南'
    elif xx == 44:
        return '广东'
    elif xx == 45:
        return '广西'
    elif xx == 46:
        return '海南'
    elif xx == 50:
        return '重庆'
    elif xx == 51:
        return '四川'
    elif xx == 52:
        return '贵州'
    elif xx == 53:
        return '云南'
    elif xx == 54:
        return '西藏'
    elif xx == 61:
        return '陕西'
    elif xx == 62:
        return '甘肃'
    elif xx == 63:
        return '青海'
    elif xx == 64:
        return '宁夏'
    elif xx == 65:
        return '新疆'
    else :
        return '其他'

In [ ]:
# （2）画像字段统计
tmp1 = df_mob_temp[(df_mob_temp.当前期数 == 1)][['order_number','search_time','id_card_num','purchase_amount','最大期数_Bin','province_name','性别','age','剩余采购成本_202508']]
tmp1['xx'] = tmp1.apply(lambda x: int(x.id_card_num[:2]), axis = 1)
tmp1['province_name_new'] = tmp1.apply(lambda row: province_func(row['xx']), axis=1)
tmp1 = tmp1.drop_duplicates(subset = 'order_number')
tmp1.shape
tmp1['purchase_amount_Bin'] = tmp1.apply(lambda x: '1.(0,3000)' if x.purchase_amount >= 0 and x.purchase_amount < 3000
                                         else '2.[3000,5000)' if x.purchase_amount >= 3000 and x.purchase_amount < 5000
                                         else '3.[5000,10000)' if x.purchase_amount >= 5000 and x.purchase_amount < 10000
                                         else '4.[10000,+)' if x.purchase_amount >= 10000
                                         else 'other', axis = 1)
tmp1['age_Bin'] = tmp1.apply(lambda x: '1.(18,22]' if x.age <= 22
                             else '2.(22,30]' if x.age > 22 and x.age <= 30
                             else '3.(30,40]' if x.age > 30 and x.age <= 40
                             else '4.(40,50]' if x.age > 40 and x.age <= 50
                             else '5.[50+)' if x.age > 50
                             else 'other',axis = 1)


tmp1[tmp1.search_time <= '2025-07'].groupby(['最大期数_Bin']).agg({'purchase_amount': 'sum','order_number': 'count'})
tmp1[tmp1.search_time <= '2025-07'].groupby(['purchase_amount_Bin']).agg({'order_number': 'count','purchase_amount': 'sum'})
tmp1[tmp1.search_time <= '2025-07'].groupby(['age_Bin']).agg({'order_number': 'count','purchase_amount': 'sum'})
tmp1[tmp1.search_time <= '2025-07'].groupby(['性别']).agg({'order_number': 'count','purchase_amount': 'sum'})
tmp1[tmp1.search_time <= '2025-07'].groupby(['province_name_new']).agg({'purchase_amount': 'sum','order_number': 'count'})

(57591, 11)

,purchase_amount,order_number
最大期数_Bin,,
12个月,3.549845e+08,45650
18个月,8.892394e+06,1270
24个月,4.477414e+07,6161


,order_number,purchase_amount
purchase_amount_Bin,,
"1.(0,3000)",2701,1.932530e+06
"2.[3000,5000)",1751,7.419632e+06
"3.[5000,10000)",47230,3.846409e+08
"4.[10000,+)",1399,1.465798e+07


,order_number,purchase_amount
age_Bin,,
"1.(18,22]",12675,9.180497e+07
"2.(22,30]",19906,1.540051e+08
"3.(30,40]",15830,1.257376e+08
"4.(40,50]",4635,3.709074e+07
5.[50+),35,1.267100e+04


,order_number,purchase_amount
性别,,
女,17599,1.407994e+08
男,35482,2.678516e+08


,purchase_amount,order_number
province_name_new,,
上海,2416565.00,289
云南,17002699.80,2239
内蒙古,8775108.00,1112
北京,2430646.00,293
吉林,10135569.00,1284
四川,29367092.00,3876
天津,2370624.00,298
宁夏,2003953.00,267
安徽,26510016.78,3412


,search_time,当前累计实付租金_all,当前累计已收买断金,当前累计实付,当前累计实付采购成本_all
0,2022-07,101212.98,21893.02,123106.00,66048.112459
1,2022-08,295200.32,15505.07,310705.39,198159.497552


### 4_9_7.补充

In [104]:
# （1）截止2025.7.31 各资方应收未收
# （1-1）已买断
# 截面数据
# 取买断的时间，订单表的 has_actual可以判断买断
sql1 = '''-- 买断信息   
SELECT too.order_id
,too.real_pay_money 
,too.pay_date
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断
and too.real_pay_money > 0 and too.pay_date is not null
'''
df_out_1 = query(sql1)
df_out_1.shape
df_out_1["pay_date"] = df_out_1['pay_date'].dt.date
df_out_1["pay_date"] = pd.to_datetime(df_out_1["pay_date"],errors="coerce")

temp1 = df_out_1.groupby(['order_id']).agg({'pay_date': 'max'}).reset_index()
temp1.shape

temp2 = df_xz_concat[['order_number','order_id']].drop_duplicates(subset = 'order_number')
temp3 = df_mob_temp[['order_number','rembursement_status','view_month','订单状态_new']].merge(temp2, on = 'order_number', how = 'left')
temp3 = temp3.merge(temp1, on = 'order_id', how = 'left')

temp3.shape
# A2022072701455912 order_id=11499 已买断，但买断表中信息为0，故用 订单状态_new 和 rembursement_status 综合判断是否已买断
# A2023021010432952 order_id=125472 已买断，解约时间是2025.4.14，买断最大日期是2025.9.17
# A20240419165429143 P站2个解约时间

# 买断逻辑变更
# 截止2025.9.21,9月买断266单，2029.9.12-2025.9.21买断259单，故这259单的订单状态不为已买断 ---8387 历史买断
# 2025.9.1后的买断不计算在内
# 续租的或租送的，买断的话，买断尾款是0
# rembursement_status 没有缺失或空
temp3[(temp3.rembursement_status == '已买断')&((temp3.pay_date < '2025-08-01')|(temp3.pay_date.isnull()))].drop_duplicates(subset = 'order_number').shape
df_md = temp3[(temp3.rembursement_status == '已买断')&((temp3.pay_date < '2025-08-01')|(temp3.pay_date.isnull()))&(temp3.view_month <= '2025-07')].drop_duplicates(subset = 'order_number')[['order_number']]
df_md['是否买断'] = 1
df_md.shape

# （1-2）已归还
sql = '''-- 租后应收监控  
SELECT  distinct tprm.order_number,tprm.relet_days,om.status,om.has_actual
from db_digua_business.t_postlease_receivables_monitoring   tprm  # 可能包含非自营的
left join db_digua_business.t_order om on tprm.order_id = om.id
'''

temp1 = query(sql)
temp1.shape

temp3 = df_xz_concat[['order_number','finish_date_new']].drop_duplicates(subset = 'order_number')   # df_xz_concat 自营的
temp4 = temp3.merge(temp1, on = 'order_number', how = 'left')

temp4.loc[:, '已归还'] = np.where((temp4.status.isin([8, 15, 16]))&(temp4.has_actual==0)&(temp4.relet_days==0), 1, 0)
temp4 = temp4.drop_duplicates(subset = 'order_number')
temp4['finish_date_new'] = pd.to_datetime(temp4['finish_date_new'])
df_gh = temp4[(temp4.已归还 == 1)&((temp4.finish_date_new < '2025-08-01')|(temp4.finish_date_new.isnull()))]
temp4[(temp4.已归还 == 1)].shape
df_gh = df_gh[['order_number','已归还']]
df_gh.shape

# （1-3）资方
sql = '''-- 租后应收监控  
SELECT  tprm.order_number
,ym.name -- 订单归属资方
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
;
'''

temp1 = query(sql)

# conditions_1 = (df_mob_temp.view_month == '2025-07')    2025.7 归还的
temp2 = df_mob_temp[conditions_1].merge(temp1, on = 'order_number', how = 'left')
temp2 = temp2.merge(df_md, on = 'order_number', how = 'left')
temp2 = temp2.merge(df_gh, on = 'order_number', how = 'left')

temp2['type'] = np.where(temp2.name == '维客自营账户', '1.维客自营账户', '5.代运营')
temp2['type'] = np.where(temp2.name == '海南鸿丰商业保理有限公司', '2.海南鸿丰商业保理有限公司', temp2.type)
temp2['type'] = np.where(temp2.name == '杭州金锤网络信息服务有限公司', '3.杭州金锤网络信息服务有限公司', temp2.type)
temp2['type'] = np.where(temp2.name == '成都越蓉金科科技有限公司', '4.成都越蓉金科科技有限公司', temp2.type)

# （4）统计
temp2.groupby(['type']).agg({'剩余总应收': 'sum'}).reset_index().to_excel('tmp.xlsx', index = False)
temp2[(temp2.是否买断 != 1)&(temp2.已归还 != 1)].groupby(['type']).agg({'剩余总应收': 'sum'}).reset_index().to_excel('tmp1.xlsx', index = False)


(8847, 3)

(8394, 2)

(671214, 6)

(7869, 6)

(7869, 2)

(75461, 4)

(2595, 6)

(2027, 2)

# 5.其他需求

## 5_1.月末业务数据

In [ ]:
# 备注
# 用 df_goal_temp，不跑 剔除样本逻辑，但跑 4_9_0.采购成本分摊口径梳理，得到 调整前 的实际数据
# 买断价、应付总租金、买断尾款 为下单时的值

tmp1 = df_mob_temp[(df_mob_temp.当前期数 == 1)]
tmp1.shape
temp1 = tmp1.groupby(['search_time']).agg({'order_number':'count'
                                           ,'purchase_amount':'sum'
                                           ,'买断价': 'sum'
                                           ,'应付总租金':'sum'
                                           ,'买断尾款':'sum'
                                           ,'price':'sum'
                                           ,'residue_money':'sum'}).reset_index()
temp1 = temp1.rename(columns = {'order_number':'出库数','purchase_amount':'采购金额','买断价':'签约价','price':'碎屏险','residue_money':'押金'})
conditions_1 = (df_mob_temp.view_month == '2025-08')
conditions_2 = (df_mob_temp.rembursement_status.isin(['已逾期','还款中','续租中','未到首期还款日','检测中']))
# conditions_2 = (df_mob_temp.剩余总应收 > 0) # df_mob_temp.rembursement_status 为 已买断/已完成 的， 剩余总应收 > 0，剩余总应收未加工
conditions_3 = (df_mob_temp.overdue_days_mob > 0)
conditions_4 = (df_mob_temp.overdue_days_mob > 30)
conditions_5 = (df_mob_temp.overdue_days_mob > 60)
conditions_6 = (df_mob_temp.overdue_days_mob > 90)

tmp2 = df_mob_temp.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp2 = tmp2.merge(df_mob_temp[['order_number','search_time','当前期数','当前累计实付租金_all','当前累计已收买断金','当前累计实付','当前累计实付采购成本_all']], on = ['order_number','当前期数'], how = 'left')

temp2 = df_mob_temp[conditions_1 & conditions_2].groupby(['search_time']).agg({'order_number':'count'
                                                                            #    ,'当前累计实付租金_all':'sum'
                                                                            #    ,'当前累计已收买断金':'sum'
                                                                            #    ,'当前累计实付':'sum'
                                                                            #    ,'当前累计实付采购成本_all':'sum'
                                                                               ,'剩余采购成本_old':'sum'
                                                                               ,'剩余采购成本':'sum'
                                                                               ,'剩余采购成本_202508':'sum'
                                                                               ,'剩余总应收':'sum'}).reset_index()
temp3 = df_mob_temp[conditions_1 & conditions_2 & conditions_3].groupby(['search_time']).agg({'剩余总应收':'sum'}).reset_index()
temp4 = df_mob_temp[conditions_1 & conditions_2 & conditions_4].groupby(['search_time']).agg({'剩余总应收':'sum'}).reset_index()
temp5 = df_mob_temp[conditions_1 & conditions_2 & conditions_5].groupby(['search_time']).agg({'剩余总应收':'sum'}).reset_index()
temp6 = df_mob_temp[conditions_1 & conditions_2 & conditions_6].groupby(['search_time']).agg({'剩余总应收':'sum'}).reset_index()
temp7 = tmp2.groupby(['search_time']).agg({'当前累计实付租金_all':'sum','当前累计已收买断金':'sum','当前累计实付':'sum','当前累计实付采购成本_all':'sum'}).reset_index()

temp3 = temp3.rename(columns = {'剩余总应收': 'm1'})
temp4 = temp4.rename(columns = {'剩余总应收': 'm2'})
temp5 = temp5.rename(columns = {'剩余总应收': 'm3'})
temp6 = temp6.rename(columns = {'剩余总应收': 'm3+'})

temp1 = temp1.merge(temp7, on = 'search_time', how = 'left')
temp1 = temp1.merge(temp2, on = 'search_time', how = 'left')
temp1 = temp1.merge(temp3, on = 'search_time', how = 'left')
temp1 = temp1.merge(temp4, on = 'search_time', how = 'left')
temp1 = temp1.merge(temp5, on = 'search_time', how = 'left')
temp1 = temp1.merge(temp6, on = 'search_time', how = 'left')

temp1 = temp1.rename(columns = {'order_number':'未结束订单数'
                                ,'当前累计实付采购成本_all': '当前累计实付采购成本_ks'
                                ,'剩余采购成本_old':'剩余采购成本1'
                                ,'剩余采购成本':'剩余采购成本_2'
                                ,'剩余采购成本_202508':'剩余采购成本_ks'})


path = 'C:/Users/hxq/Desktop/mine/3_临时分析需求/5.七月/'
# temp1.to_excel(path + '1_业务量及租后表现_20250902.xlsx', sheet_name = "调整后", index = False)
temp1.to_excel(path + 'tmp.xlsx', sheet_name = "调整前", index = False)
# temp1.to_excel(path + 'tmp.xlsx', sheet_name = "调整后", index = False)

# 当前累计实付采购成本_ks	客商的采购采购成本分摊口径
# 剩余采购成本_1	采购成本-实付租金
# 剩余采购成本_2	采购成本-实付租金，若当前逾期，则再减去碎屏险和押金
# 剩余采购成本_ks	采购成本-实付租金中分摊到成本的金额，客商口径


# df_mob_temp[(df_mob_temp.view_month == '2025-08')&(df_mob_temp.search_time == '2022-07')][['order_number','rembursement_status']]
# df_mob_temp[(df_mob_temp.view_month == '2022-07')&(df_mob_temp.search_time == '2022-07')][['order_number','rembursement_status']]

# df_mob_temp[df_mob_temp.order_number == 'A202207172051293'][['order_number','rembursement_status','当前期数','应付日期']]
# A202207190744402
# A202207172051293



(62033, 54)

In [ ]:
df_xz_concat[(df_xz_concat.实付日期new == '2025-09-01')&(df_xz_concat.归属渠道 == '芝麻租物')&(df_xz_concat.下单日期 >= '2024-07-01')&(df_xz_concat.下单日期 >= '2025-05-31')].shape


(297, 174)

## 5_2.租中预警

### （1）2025.9 复盘

In [ ]:
# note：
# 应还月 ，不是自然月，衍生 月中最大逾期天数
# 不要跑 4_6_1和4_6_2


# （1）宽表
# 逾期天数：当期逾期天数、overdue_days_mob：当期月末逾期天数
# df_mob_temp：已买断/已完成的对应期数已删去
df_hsl = df_xz_concat[['order_number','search_time','is_relet','当前期数','应付日期','当前应付租金','逾期天数','实付日期new','实付金额']]
df_hsl['实付金额'] = df_hsl['实付金额'].fillna(0)
df_hsl.shape

# df_hsl = df_hsl[~df_hsl.order_number.isin(goal_df_order_number)]
# df_hsl.shape

df_hsl = df_hsl.merge(df_mob_temp[['order_number','当前期数','overdue_days_mob']], on = ['order_number','当前期数'], how = 'left')
temp1 = df_mob_temp[['order_number','租赁方案']].drop_duplicates(subset = 'order_number')
df_hsl = df_hsl.merge(temp1, on = ['order_number'], how = 'left')
df_hsl['租赁方案'] = np.where(df_hsl.租赁方案.isna(),'租完归还',df_hsl.租赁方案)
df_hsl['overdue_days_mob'] = np.where(df_hsl.overdue_days_mob.isna(),0,df_hsl.overdue_days_mob)
df_hsl['应付年月'] = df_hsl['应付日期'].dt.strftime('%Y-%m')
df_hsl['实付年月'] = np.where(df_hsl.实付日期new.isna(), pd.NaT, pd.to_datetime(df_hsl.实付日期new.astype(str)).dt.strftime('%Y-%m'))
df_hsl['实付日'] = np.where(df_hsl.实付日期new.isna(), 0, pd.to_datetime(df_hsl.实付日期new.astype(str)).dt.day)
df_hsl['实付日'] = df_hsl['实付日'].fillna(0)

df_hsl['上一个当前期数'] = df_hsl['当前期数'] - 1
temp2 = df_hsl[['order_number','当前期数','overdue_days_mob','实付日','实付年月']]
temp2 = temp2.rename(columns = {'当前期数':'上一个当前期数', 'overdue_days_mob':'overdue_days_mob_pre','实付年月':'实付年月_pre'})
df_hsl = df_hsl.drop(columns = ['实付日'])
df_hsl = df_hsl.merge(temp2, on = ['order_number','上一个当前期数'], how = 'left')
df_hsl['overdue_days_mob_pre'] = df_hsl['overdue_days_mob_pre'].fillna(0)

# 当前月末逾期天数、月间发生过的最大逾期天数（上个月月末逾期、当月开始逾期），这两者比较大小，为当前月的最大逾期天数，当月最大逾期天数和当月月末逾期天数比较 求回收率
# 加工 月间发生过的最大逾期天数

# 月间发生过的最大逾期天数-当月开始逾期
df_hsl['overdue_days_1'] = np.where(df_hsl.应付年月 == df_hsl.实付年月, (df_hsl.实付日期new - df_hsl.应付日期).dt.days, 0)
df_hsl['overdue_days_1'] = np.where(df_hsl.overdue_days_1 < 0, 0, df_hsl.overdue_days_1)
# df_hsl['overdue_days_1'] = np.where((df_hsl.overdue_days_mob_pre > 0)&(df_hsl.应付年月 == df_hsl.实付年月), df_hsl.实付日期new.dt.day, df_hsl.overdue_days_1)
# 当月有还之前的，则为上一个月的月末逾期天数+实付日，否则为 当月逾期当月还的逾期天数
# df_hsl['overdue_days_2'] = np.where((df_hsl.overdue_days_mob_pre >= df_hsl.overdue_days_mob) & (df_hsl.overdue_days_mob_pre>0), df_hsl.overdue_days_mob_pre + df_hsl.实付日, df_hsl.overdue_days_mob) # df_hsl.overdue_days_mob_pre + df_hsl.overdue_days_1
# df_hsl['overdue_days_2'] = np.where(df_hsl.实付日期new_pre >= df_hsl.应付日期, df_hsl.overdue_days_mob_pre + df_hsl.实付日, df_hsl.overdue_days_2)
# 月间发生过的最大逾期天数-上个月月末逾期但当月还、月间发生过的最大逾期天数-当月开始逾期
df_hsl['overdue_days_2'] = np.where(df_hsl.实付年月_pre == df_hsl.应付年月, df_hsl.overdue_days_mob_pre + df_hsl.实付日, df_hsl.overdue_days_1)
# 当前月末逾期天数、月月间发生过的最大逾期天数（上个月月末逾期、当月开始逾期） 取最大值
df_hsl['overdue_days_mob_max'] = np.where(df_hsl.overdue_days_2 > df_hsl.overdue_days_mob, df_hsl.overdue_days_2, df_hsl.overdue_days_mob)
df_hsl['overdue_days_mob_max'] = np.where(df_hsl.当前期数 == 1, 0, df_hsl.overdue_days_mob_max)

df_hsl = df_hsl.drop(columns = ['上一个当前期数','实付日','实付年月_pre'])
df_hsl.shape
df_hsl = df_hsl[df_hsl.应付日期 <= '2025-08-31']
df_hsl.shape
df_hsl[:1]


# df_hsl[df_hsl.order_number == 'A202207201414474']
# A202207201414474
# A2022072220482114
# A2022072701455912  第26期的 overdue_days_mob 有误，超24期了，续租计算 overdue_days_mob 导致的

(824346, 9)

(824346, 17)

(505274, 17)

,order_number,search_time,is_relet,当前期数,应付日期,当前应付租金,逾期天数,实付日期new,实付金额,overdue_days_mob,租赁方案,应付年月,实付年月,overdue_days_mob_pre,overdue_days_1,overdue_days_2,overdue_days_mob_max
0,A202207050851373,2022-07,0.0,1.0,2022-07-05,357.95,0,2022-07-05,357.95,0.0,租完归还,2022-07,2022-07,0.0,0.0,0.0,0.0


In [ ]:
# （2-1）客群标签指标
# 预警对象：上一个月末无逾期的且观察月未还的订单
# 近3个月最大逾期天数>=1
# 历史最大逾期天数>30
# 综合风险等级为'2' 且 履约保护>15
# 自有模型2501>15 且 数控风险等级为'996'
# 评分等级为'C'/'D' 且 蚁盾分3.0>70

# 下单时的指标值
# 综合风险等级 rentOnlineRiskLevelOverall
sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT *
from db_credit.risk_alipay_auth_query
;
'''
zm_fk_df = query(sql1)
zm_fk_df = zm_fk_df[['trace_id', 'rentOnlineRiskLevelOverall']]
zm_fk_df.shape


# 蚂蚁链-履约保护  nsf_rental_v1_3_hundred
sql1='''
SELECT trace_id, nsf_rental_v1_3_hundred	
from db_credit.risk_antchain_risk_general
;
'''
my_zl_general_df=query(sql1)
my_zl_general_df['nsf_rental_v1_3_hundred'] = my_zl_general_df['nsf_rental_v1_3_hundred'].astype(float)
my_zl_general_df.shape

# 自有模型-2501
sql1='''-- 风控-自有模型-2501
SELECT * 
from db_credit.risk_private_model_2501_base
;
'''
mx_2501_df =query(sql1)
mx_2501_df['model_score_2501'] = pd.to_numeric(mx_2501_df['score'], errors='coerce')
mx_2501_df = mx_2501_df[['trace_id', 'model_score_2501']]
mx_2501_df.shape

# zm_jh_t0_level 蚂蚁数控风险等级 
sql1='''-- 芝麻数控-交互安全风控
select
order_id, 
replace(case when JSON_VALID(risk_t0) THEN JSON_EXTRACT(risk_t0, '$.level') end,'"','') as zm_jh_t0_level 
from db_credit.risk_alipay_interactive_prod
;
'''
zm_jh_df=query(sql1)
zm_jh_df.shape

# 评分等级
sql1='''
SELECT * 
from db_credit.risk
;
'''
risk_df =query(sql1)
risk_df.shape


sql1='''-- 蚁盾分2.0
SELECT trace_id, score as my_zl_score2
from db_credit.risk_antchain_ato
;
'''
my_zl_df_2 = query(sql1)
my_zl_df_2['my_zl_score2'] = my_zl_df_2['my_zl_score2'].astype(float)
my_zl_df_2.shape

# 中间表
sql1='''-- trace关联表 
SELECT * 
from db_credit.order_association
;
'''

trace_df =query(sql1)
trace_df.shape

sql1='''-- risk关联表 
SELECT trace_id, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.is_model') end,'"','') as is_model
from db_credit.risk
;
'''
risk_df_model = query(sql1)
risk_df_model.shape

# 关联
df_out_data = trace_df.merge(zm_fk_df, on = 'trace_id', how = 'left')
df_out_data = df_out_data.merge(my_zl_general_df, on = 'trace_id', how = 'left')

mx_2501_df = mx_2501_df.merge(risk_df_model, on='trace_id', how='left')
df_out_data = df_out_data.merge(mx_2501_df, left_on='order_trace_id', right_on='trace_id', how='left')
df_out_data.drop('trace_id_y', axis=1, inplace=True)
df_out_data.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)

df_out_data = df_out_data.merge(zm_jh_df, on = 'order_id', how = 'left')
df_out_data = df_out_data.merge(risk_df[['trace_id','grade']], left_on='order_trace_id', right_on='trace_id', how='left')
df_out_data.drop('trace_id_y', axis=1, inplace=True)
df_out_data.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)

df_out_data = df_out_data.merge(my_zl_df_2, on = 'trace_id', how = 'left')
df_out_data.shape

(393655, 2)

(122329, 2)

(210942, 2)

(152506, 2)

(554671, 12)

(235062, 2)

(3179183, 5)

(554676, 2)

In [ ]:
# （2-2）客群标签指标
# 历史逾期
var_1 = ['order_number','rentOnlineRiskLevelOverall','nsf_rental_v1_3_hundred','model_score_2501','zm_jh_t0_level','grade','my_zl_score2']
df_hsl = df_hsl.merge(df_out_data[var_1], on = 'order_number', how = 'left')

# note：逾期天数 为截止2025.9.2的最大逾期天数
temp1 = df_hsl[df_hsl.应付年月.isin(['2025-02','2025-03','2025-04'])].groupby(['order_number']).agg({'overdue_days_mob_max': 'max'}).reset_index()
temp2 = df_hsl[df_hsl.应付年月 < '2025-04'].groupby(['order_number']).agg({'逾期天数': 'max'}).reset_index() # 应付日期为4月的，到5月初还没满足 > 30天，故取的是4月之前的
temp2_1 = df_hsl[(df_hsl.应付年月 <= '2025-04')&(df_hsl.当前期数.isin([2,3,4]))].groupby(['order_number']).agg({'逾期天数': 'max'}).reset_index() 
temp1 = temp1.rename(columns = {'overdue_days_mob_max':'overdue_days_m3'})
temp2 = temp2.rename(columns = {'逾期天数':'overdue_days_h'})
temp2_1 = temp2_1.rename(columns = {'逾期天数':'max_mob3dpd1'})
temp3 = df_hsl[df_hsl.应付年月 == '2025-05']
temp3 = temp3.merge(temp1, on = 'order_number', how = 'left')
temp3 = temp3.merge(temp2, on = 'order_number', how = 'left')
temp3 = temp3.merge(temp2_1, on = 'order_number', how = 'left')
temp3.shape

temp1 = df_hsl[df_hsl.应付年月.isin(['2025-03','2025-04','2025-05'])].groupby(['order_number']).agg({'overdue_days_mob_max': 'max'}).reset_index()
temp2 = df_hsl[df_hsl.应付年月 < '2025-05'].groupby(['order_number']).agg({'逾期天数': 'max'}).reset_index() # 应付日期为4月的，到5月初还没满足 > 30天，故取的是4月之前的
temp2_1 = df_hsl[(df_hsl.应付年月 <= '2025-05')&(df_hsl.当前期数.isin([2,3,4]))].groupby(['order_number']).agg({'逾期天数': 'max'}).reset_index() 
temp1 = temp1.rename(columns = {'overdue_days_mob_max':'overdue_days_m3'})
temp2 = temp2.rename(columns = {'逾期天数':'overdue_days_h'})
temp2_1 = temp2_1.rename(columns = {'逾期天数':'max_mob3dpd1'})
temp4 = df_hsl[df_hsl.应付年月 == '2025-06']
temp4 = temp4.merge(temp1, on = 'order_number', how = 'left')
temp4 = temp4.merge(temp2, on = 'order_number', how = 'left')
temp4 = temp4.merge(temp2_1, on = 'order_number', how = 'left')
temp4.shape

temp1 = df_hsl[df_hsl.应付年月.isin(['2025-04','2025-05','2025-06'])].groupby(['order_number']).agg({'overdue_days_mob_max': 'max'}).reset_index()
temp2 = df_hsl[df_hsl.应付年月 < '2025-06'].groupby(['order_number']).agg({'逾期天数': 'max'}).reset_index() # 应付日期为4月的，到5月初还没满足 > 30天，故取的是4月之前的
temp2_1 = df_hsl[(df_hsl.应付年月 <= '2025-06')&(df_hsl.当前期数.isin([2,3,4]))].groupby(['order_number']).agg({'逾期天数': 'max'}).reset_index() 
temp1 = temp1.rename(columns = {'overdue_days_mob_max':'overdue_days_m3'})
temp2 = temp2.rename(columns = {'逾期天数':'overdue_days_h'})
temp2_1 = temp2_1.rename(columns = {'逾期天数':'max_mob3dpd1'})
temp5 = df_hsl[df_hsl.应付年月 == '2025-07']
temp5 = temp5.merge(temp1, on = 'order_number', how = 'left')
temp5 = temp5.merge(temp2, on = 'order_number', how = 'left')
temp5 = temp5.merge(temp2_1, on = 'order_number', how = 'left')
temp5.shape

temp1 = df_hsl[df_hsl.应付年月.isin(['2025-05','2025-06','2025-07'])].groupby(['order_number']).agg({'overdue_days_mob_max': 'max'}).reset_index()
temp2 = df_hsl[df_hsl.应付年月 < '2025-07'].groupby(['order_number']).agg({'逾期天数': 'max'}).reset_index() # 应付日期为4月的，到5月初还没满足 > 30天，故取的是4月之前的
temp2_1 = df_hsl[(df_hsl.应付年月 <= '2025-07')&(df_hsl.当前期数.isin([2,3,4]))].groupby(['order_number']).agg({'逾期天数': 'max'}).reset_index() 
temp1 = temp1.rename(columns = {'overdue_days_mob_max':'overdue_days_m3'})
temp2 = temp2.rename(columns = {'逾期天数':'overdue_days_h'})
temp2_1 = temp2_1.rename(columns = {'逾期天数':'max_mob3dpd1'})
temp6 = df_hsl[df_hsl.应付年月 == '2025-08']
temp6 = temp6.merge(temp1, on = 'order_number', how = 'left')
temp6 = temp6.merge(temp2, on = 'order_number', how = 'left')
temp6 = temp6.merge(temp2_1, on = 'order_number', how = 'left')
temp6.shape

df_hsl_202505_08 = pd.concat([temp3, temp4, temp5, temp6])

# path = 'C:/Users/hxq/Desktop/mine/4_专题分析/21_租中预警/'
# # df_analy_data 的 mob3dpd_max >= 1 ， max_mob3dpd1 为 1， max_mob3dpd1 ：mob3的最大逾期天数 是否>=1
# temp = pd.read_excel(path + 'order_max_mob3dpd1.xlsx')
# df_hsl_202505_08 = df_hsl_202505_08.merge(temp, on = 'order_number', how = 'left')

df_hsl_202505_08.shape
df_hsl_202505_08[:1]

(35389, 26)

(39556, 26)

(44356, 26)

(47942, 26)

(167243, 26)

,order_number,search_time,is_relet,当前期数,应付日期,当前应付租金,逾期天数,实付日期new,实付金额,overdue_days_mob,租赁方案,应付年月,实付年月,overdue_days_mob_pre,overdue_days_1,overdue_days_2,overdue_days_mob_max,rentOnlineRiskLevelOverall,nsf_rental_v1_3_hundred,model_score_2501,zm_jh_t0_level,grade,my_zl_score2,overdue_days_m3,overdue_days_h,max_mob3dpd1
0,A2023020820382546,2023-02,1.0,28.0,2025-05-03,266.4,122,NaT,0.0,206.0,租完归还,2025-05,NaT,176.0,0.0,0.0,206.0,NaN,NaN,NaN,NaN,NaN,NaN,176.0,302.0,1.0


In [ ]:
# （3）客群打标签
# 预警对象：上一个月末无逾期的且观察月未还的订单
# 近3个月最大逾期天数>=1
# 历史最大逾期天数>30
# 综合风险等级为'2' 且 履约保护>15
# 自有模型2501>15 且 数控风险等级为'996'
# 评分等级为'C'/'D' 且 蚁盾分3.0>70 -- 88月及之前的用蚁盾分2.0

# overdue_days_m3：近3个月最大逾期天数
# max_mob3dpd1：mob3的最大逾期天数

# 预警对象：上一个月月末无逾期的且实付年月在应还年月及之后的(精确的话：实付日期-应付日期 >= -3 )
df_hsl_202505_08['overdue_days_tmp'] = np.where(df_hsl_202505_08.实付日期new.notna(), (df_hsl_202505_08.实付日期new - df_hsl_202505_08.应付日期).dt.days, 0)
df_hsl_202505_08['nsf_rental_v1_3_hundred'] = df_hsl_202505_08['nsf_rental_v1_3_hundred'].astype(float)

# 月初批量预警，用的是 overdue_days_mob_pre；实际应用中，清单按天更新，取的是当前无逾期，不是 上个月月末的逾期状态
conditions_1 = (df_hsl_202505_08.overdue_days_mob_pre == 0)
conditions_2 = (df_hsl_202505_08.overdue_days_tmp >= -3)

# conditions_3 = (df_hsl_202505_08.overdue_days_m3 > 1)  # 或
conditions_3 = (df_hsl_202505_08.max_mob3dpd1 >= 1)  # 或    为了跟2025.8月初筛选样本的逻辑一样
conditions_4 = (df_hsl_202505_08.overdue_days_h > 30)

conditions_5 = (df_hsl_202505_08.rentOnlineRiskLevelOverall == '2') & (df_hsl_202505_08.nsf_rental_v1_3_hundred > 15)
conditions_6 = (df_hsl_202505_08.model_score_2501 > 15) & (df_hsl_202505_08.zm_jh_t0_level == '996')
conditions_7 = (df_hsl_202505_08.grade.isin(['C','D'])) & (df_hsl_202505_08.my_zl_score2 > 62.5)

conditions_8 = (df_hsl_202505_08.search_time >= '2024-11') & (df_hsl_202505_08.应付年月 == '2025-05')  # 线下放到 近6个月，上线后为3个月
conditions_9 = (df_hsl_202505_08.search_time >= '2024-12') &(df_hsl_202505_08.应付年月 == '2025-06')  # 线下放到 近6个月，上线后为3个月
conditions_10 = (df_hsl_202505_08.search_time >= '2025-01') & (df_hsl_202505_08.应付年月 == '2025-07')  # 线下放到 近6个月，上线后为3个月
conditions_11 = (df_hsl_202505_08.search_time >= '2025-02') & (df_hsl_202505_08.应付年月 == '2025-08')  # 线下放到 近6个月，上线后为3个月

df_hsl_202505_08['label'] = np.where(conditions_1 & conditions_2 & (conditions_3|conditions_4), 1, 0)
df_hsl_202505_08['label'] = np.where(conditions_1 & conditions_2 & conditions_5 & (conditions_8 | conditions_9 | conditions_10 | conditions_11), 1, df_hsl_202505_08.label)
df_hsl_202505_08['label'] = np.where(conditions_1 & conditions_2 & conditions_6 & (conditions_8 | conditions_9 | conditions_10 | conditions_11), 1, df_hsl_202505_08.label)
df_hsl_202505_08['label'] = np.where(conditions_1 & conditions_2 & conditions_7 & (conditions_8 | conditions_9 | conditions_10 | conditions_11), 1, df_hsl_202505_08.label)

df_hsl_202505_08['label'] = np.where(df_hsl_202505_08.当前期数 == 1 , 0, df_hsl_202505_08.label)
df_hsl_202505_08['label'] = np.where(df_hsl_202505_08.当前期数 == 13 , 0, df_hsl_202505_08.label)

pd.pivot_table(df_hsl_202505_08,index='应付年月',columns='label',values='order_number',aggfunc='count')
# df_hsl_202505_08[(df_hsl_202505_08.应付年月 == '2025-08')&(df_hsl_202505_08.label == 1)][['order_number','当前期数','overdue_days_tmp','应付日期']].to_excel('tmp.xlsx')  #导出去跟8月初的核对
# 数据保留
# df_hsl_202505_08_new[['order_number','search_time','应付年月','label','label_cs','overdue_days_mob_max','overdue_days_mob_pre','overdue_days_tmp']].to_excel('zzyj_fp_20250903.xlsx', index = False)

label,0,1
应付年月,,
2025-05,33151,2238
2025-06,37111,2445
2025-07,41001,3355
2025-08,43673,4269


In [ ]:
# （4）数据核对
# 2025.8 的清单与以上口径筛选出来的差异不大，2025.8 的用实际的预警清单，即 2025.7.29 导出来的每天更新的，实际预警过的
path = 'C:/Users/hxq/Desktop/mine/4_专题分析/21_租中预警/'
temp = pd.read_excel(path + 'zzyj_list_20250729.xlsx')
temp = temp[['order_number']]
temp['label'] = 1
temp.shape

df_hsl_202505_07 = df_hsl_202505_08[df_hsl_202505_08.应付年月 <= '2025-07']
df_hsl_202508 = df_hsl_202505_08[df_hsl_202505_08.应付年月 == '2025-08']
df_hsl_202508 = df_hsl_202508.drop(columns = ['label'])
df_hsl_202508 = df_hsl_202508.merge(temp, on = 'order_number', how = 'outer')
df_hsl_202508['label'] = df_hsl_202508['label'].fillna(0)
df_hsl_202508['label'] = np.where(df_hsl_202508.overdue_days_tmp < -3, 0 , df_hsl_202508.label)  # 还款日前三天还款的，不用预警，559 条

df_hsl_202505_08_new = pd.concat([df_hsl_202505_07, df_hsl_202508])
df_hsl_202505_08_new = df_hsl_202505_08_new.drop_duplicates(subset = ['order_number','应付年月'])
pd.pivot_table(df_hsl_202505_08_new,index='应付年月',columns='label',values='order_number',aggfunc='count')

(4614, 2)

label,0.0,1.0
应付年月,,
2025-05,33151,2238
2025-06,37111,2445
2025-07,41001,3355
2025-08,43884,4055


In [ ]:
# （5）关联催收表
# 出现过在催收表，表示有入催
sql = ''' -- 代理任务表
select * from db_rent.ya_agent_task
'''
df_cs=query(sql)
df_cs.shape

df_cs["委派日期"]=df_cs["add_time"].dt.date
df_cs["委派日期"]=pd.to_datetime(df_cs["委派日期"],errors="coerce")
df_cs['委派月份'] = df_cs["委派日期"].dt.strftime('%Y-%m')

df_cs_1 = df_cs.drop_duplicates(subset = ['order_number','委派月份'])[['order_number','委派月份']]  # 只看新增逾期的委派
df_cs_1.shape
df_cs_1['label_cs'] = 1
df_cs_1 = df_cs_1.rename(columns = {'委派月份': '应付年月'})

# 合并预警表
df_hsl_202505_08_new = df_hsl_202505_08_new.merge(df_cs_1, on = ['order_number','应付年月'], how = 'left')
df_hsl_202505_08_new['label_cs'] = df_hsl_202505_08_new['label_cs'].fillna(0)
df_hsl_202505_08_new.shape
df_hsl_202505_08_new[:1]

(120530, 38)

(91566, 2)

(167240, 30)

,order_number,search_time,is_relet,当前期数,应付日期,当前应付租金,逾期天数,实付日期new,实付金额,overdue_days_mob,租赁方案,应付年月,实付年月,overdue_days_mob_pre,overdue_days_1,overdue_days_2,overdue_days_mob_max,rentOnlineRiskLevelOverall,nsf_rental_v1_3_hundred,model_score_2501,zm_jh_t0_level,grade,my_zl_score2,overdue_days_m3,overdue_days_h,max_mob3dpd1,overdue_days_tmp,label,label_tmp,label_cs
0,A2023020820382546,2023-02,1.0,28.0,2025-05-03,266.4,122,NaT,0.0,206.0,租完归还,2025-05,NaT,176.0,0.0,0.0,206.0,NaN,NaN,NaN,NaN,NaN,NaN,176.0,302.0,1.0,0.0,0.0,0,1.0


In [ ]:
# （6-1）统计-高风险客群
conditions_1 = (df_hsl_202505_08_new.label == 1)
# conditions_2 = (df_hsl_202505_08_new.overdue_days_mob_pre == 0)  # 当时是7.29导的数据，还款日是29、30、31 的，月底逾期没有去除，所以不用这个条件了
conditions_3 = (df_hsl_202505_08_new.overdue_days_mob_max > 0)
conditions_4 = (df_hsl_202505_08_new.label_cs >= 1)

temp1 = df_hsl_202505_08_new[conditions_1].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
# temp2 = df_hsl_202505_08_new[conditions_1 & (df_hsl_202505_08_new.overdue_days_mob_pre > 0)].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
# temp3 = df_hsl_202505_08_new[conditions_1 & conditions_2 & conditions_3].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
# temp4 = df_hsl_202505_08_new[conditions_1 & conditions_2 & conditions_3 & conditions_4].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
temp3 = df_hsl_202505_08_new[conditions_1 & conditions_3].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
temp4 = df_hsl_202505_08_new[conditions_1 & conditions_3 & conditions_4].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()

temp1 = temp1.rename(columns = {'order_number': '订单数'})
# temp2 = temp2.rename(columns = {'order_number': '存量逾期订单数'})
temp3 = temp3.rename(columns = {'order_number': '新增逾期订单数'})
temp4 = temp4.rename(columns = {'order_number': '新增入催订单数'})

# temp = temp1.merge(temp2, on = '应付年月', how = 'left')
temp = temp1.merge(temp3, on = '应付年月', how = 'left')
# temp = temp.merge(temp3, on = '应付年月', how = 'left')
temp = temp.merge(temp4, on = '应付年月', how = 'left')

# temp['存量逾期订单数'] = temp['存量逾期订单数'].fillna(0)
temp['新增逾期订单数'] = temp['新增逾期订单数'].fillna(0)
temp['新增入催订单数'] = temp['新增入催订单数'].fillna(0)
temp

,应付年月,订单数,新增逾期订单数,新增入催订单数
0,2025-05,2238,1259,1129
1,2025-06,2445,1350,1212
2,2025-07,3355,1778,1622
3,2025-08,4055,2000,1782


In [ ]:
# （6-2）统计-其他客群
conditions_1 = (df_hsl_202505_08_new.label == 0)
conditions_2 = (df_hsl_202505_08_new.overdue_days_mob_pre == 0)
conditions_3 = (df_hsl_202505_08_new.overdue_days_mob_max > 0)
conditions_4 = (df_hsl_202505_08_new.label_cs >= 1)

temp1 = df_hsl_202505_08_new[conditions_1].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
temp2 = df_hsl_202505_08_new[conditions_1 & (df_hsl_202505_08_new.overdue_days_mob_pre > 0)].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
temp3 = df_hsl_202505_08_new[conditions_1 & conditions_2 & conditions_3].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
temp4 = df_hsl_202505_08_new[conditions_1 & conditions_2 & conditions_3 & conditions_4].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()

temp1 = temp1.rename(columns = {'order_number': '订单数'})
temp2 = temp2.rename(columns = {'order_number': '存量逾期订单数'})
temp3 = temp3.rename(columns = {'order_number': '新增逾期订单数'})
temp4 = temp4.rename(columns = {'order_number': '新增入催订单数'})

temp = temp1.merge(temp2, on = '应付年月', how = 'left')
temp = temp.merge(temp3, on = '应付年月', how = 'left')
temp = temp.merge(temp4, on = '应付年月', how = 'left')

temp['存量逾期订单数'] = temp['存量逾期订单数'].fillna(0)
temp['新增逾期订单数'] = temp['新增逾期订单数'].fillna(0)
temp['新增入催订单数'] = temp['新增入催订单数'].fillna(0)
temp

,应付年月,订单数,存量逾期订单数,新增逾期订单数,新增入催订单数
0,2025-05,33151,4616,2488,2265
1,2025-06,37111,5125,2715,2433
2,2025-07,41001,5539,3660,3358
3,2025-08,43884,6196,4157,3644


In [ ]:
# （6-3）统计-all
conditions_2 = (df_hsl_202505_08_new.overdue_days_mob_pre == 0)
conditions_3 = (df_hsl_202505_08_new.overdue_days_mob_max > 0)
conditions_4 = (df_hsl_202505_08_new.label_cs >= 1)

temp1 = df_hsl_202505_08_new.groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
temp2 = df_hsl_202505_08_new[df_hsl_202505_08_new.overdue_days_mob_pre > 0].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
temp3 = df_hsl_202505_08_new[conditions_2 & conditions_3].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()
temp4 = df_hsl_202505_08_new[conditions_2 & conditions_3 & conditions_4].groupby(['应付年月']).agg({'order_number':'count'}).reset_index()

temp1 = temp1.rename(columns = {'order_number': '订单数'})
temp2 = temp2.rename(columns = {'order_number': '存量逾期订单数'})
temp3 = temp3.rename(columns = {'order_number': '新增逾期订单数'})
temp4 = temp4.rename(columns = {'order_number': '新增入催订单数'})

temp = temp1.merge(temp2, on = '应付年月', how = 'left')
temp = temp.merge(temp3, on = '应付年月', how = 'left')
temp = temp.merge(temp4, on = '应付年月', how = 'left')

temp['存量逾期订单数'] = temp['存量逾期订单数'].fillna(0)
temp['新增逾期订单数'] = temp['新增逾期订单数'].fillna(0)
temp['新增入催订单数'] = temp['新增入催订单数'].fillna(0)
temp

,应付年月,订单数,存量逾期订单数,新增逾期订单数,新增入催订单数
0,2025-05,35389,4616,3747,3394
1,2025-06,39556,5125,4065,3645
2,2025-07,44356,5539,5438,4980
3,2025-08,47939,6308,6045,5341


## 5_3.山东融越

In [ ]:
# 出库订单数	采购价	签约价	应收未收	应收未收_正常	应收未收_逾期

temp1 = df_xz_concat[['order_number','merchant_name']]
temp1 = temp1.drop_duplicates(subset = 'order_number')
temp2 = df_mob_temp.merge(temp1, on = 'order_number', how = 'left')

sql = '''-- 租后应收监控  
SELECT  tprm.order_number
,ym.name -- 订单归属资方
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
;
'''

temp3 = query(sql)
temp2 = temp2.merge(temp3, on = 'order_number', how = 'left')

conditions_1 = (temp2.view_month == '2025-08')
# temp2[conditions_1].剩余总应收.sum()

# name 维客自营账户   --  merchant_name 迪瓜优选
# 海南鸿丰商业保理有限公司
# 杭州金锤网络信息服务有限公司
# 成都越蓉金科科技有限公司

# temp2[conditions_1 & (temp2.merchant_name == '成都越蓉金科科技有限公司')].shape
# temp2[conditions_1 & (temp2.name == '成都越蓉金科科技有限公司')].shape

# conditions_3 = (temp2.merchant_name.isin(['迪瓜优选','海南鸿丰商业保理有限公司','杭州金锤网络信息服务有限公司','成都越蓉金科科技有限公司']))
# temp2[conditions_1 & conditions_3].剩余总应收.sum()

# conditions_2 = (temp2.name.isin(['维客自营账户','海南鸿丰商业保理有限公司','杭州金锤网络信息服务有限公司','成都越蓉金科科技有限公司']))
# temp2[conditions_1 & conditions_2].剩余总应收.sum()

temp4 = temp2[temp2.当前期数 == 1][['order_number','name','purchase_amount','买断价','search_time']]
temp4['type'] = np.where(temp4.name == '维客自营账户', '1.维客自营账户', '5.代运营')
temp4['type'] = np.where(temp4.name == '海南鸿丰商业保理有限公司', '2.海南鸿丰商业保理有限公司', temp4.type)
temp4['type'] = np.where(temp4.name == '杭州金锤网络信息服务有限公司', '3.杭州金锤网络信息服务有限公司', temp4.type)
temp4['type'] = np.where(temp4.name == '成都越蓉金科科技有限公司', '4.成都越蓉金科科技有限公司', temp4.type)

temp5 = temp2[temp2.view_month == '2025-08'][['order_number','剩余总应收','overdue_days_mob']]
temp6 = temp4.merge(temp5, on = 'order_number', how = 'left')
temp6 = temp6.drop_duplicates(subset = 'order_number')

temp4.shape
temp5.shape
temp6.shape
temp6[temp6.search_time <= '2025-08'].shape

temp7 = temp6[temp6.search_time <= '2025-08'].groupby(['type']).agg({'order_number':'count','purchase_amount':'sum','买断价':'sum','剩余总应收':'sum'}).reset_index()
temp8 = temp6[(temp6.search_time <= '2025-08')&(temp6.overdue_days_mob > 0)].groupby(['type']).agg({'剩余总应收':'sum'}).reset_index()
temp8 = temp8.rename(columns = {'剩余总应收': '应收未收_逾期'})

temp9 = temp7.merge(temp8, on = 'type', how = 'left')

temp9.to_excel('tmp.xlsx', index = False)
temp9

# # 2025.9.23
# temp10 = temp6[(temp6.search_time <= '2025-08')&(temp6.type == '2.海南鸿丰商业保理有限公司')].groupby(['search_time']).agg({'order_number':'count','purchase_amount':'sum','买断价':'sum','剩余总应收':'sum'}).reset_index()
# temp11 = temp6[(temp6.search_time <= '2025-08')&(temp6.type == '2.海南鸿丰商业保理有限公司')&(temp6.overdue_days_mob > 0)].groupby(['search_time']).agg({'剩余总应收':'sum'}).reset_index()
# temp11 = temp11.rename(columns = {'剩余总应收': '应收未收_逾期'})
# temp12 = temp10.merge(temp11, on = 'search_time', how = 'left')

# temp12.to_excel('tmp1.xlsx', index = False)


# temp10 = temp6[(temp6.search_time <= '2025-08')&(temp6.type == '3.杭州金锤网络信息服务有限公司')].groupby(['search_time']).agg({'order_number':'count','purchase_amount':'sum','买断价':'sum','剩余总应收':'sum'}).reset_index()
# temp11 = temp6[(temp6.search_time <= '2025-08')&(temp6.type == '3.杭州金锤网络信息服务有限公司')&(temp6.overdue_days_mob > 0)].groupby(['search_time']).agg({'剩余总应收':'sum'}).reset_index()
# temp11 = temp11.rename(columns = {'剩余总应收': '应收未收_逾期'})
# temp12 = temp10.merge(temp11, on = 'search_time', how = 'left')

# temp12.to_excel('tmp2.xlsx', index = False)

(59359, 6)

(49787, 3)

(59343, 8)

(58250, 8)

,type,order_number,purchase_amount,买断价,剩余总应收,应收未收_逾期
0,1.维客自营账户,9830,6.616019e+07,9.945139e+07,2.479461e+07,23225936.46
1,2.海南鸿丰商业保理有限公司,4582,3.801300e+07,5.873323e+07,4.910221e+07,3589225.57
2,3.杭州金锤网络信息服务有限公司,1736,1.453330e+07,2.147762e+07,1.033874e+07,3911212.68
3,4.成都越蓉金科科技有限公司,937,7.001558e+06,1.098882e+07,3.923544e+06,2522623.22
4,5.代运营,41165,3.254066e+08,5.019582e+08,3.295637e+08,65833579.72


## 5_5.山东高速

### （1）2025.9

#### 1-数据概览

In [ ]:
# 剔除样本后的 df_mob_temp

# 笔数总计
# 截止当前总出款额（采购额）
# 下单时，支付的押金
# 下单时，签约总租金
# 碎屏保修服务费 -- 碎屏险
# 已收租金
# 逾期租金
# 待收租金余额（不含逾期）

df_mob_temp = df_mob_temp.drop_duplicates(['order_number','当前期数'])

conditions_1 = (df_mob_temp.search_time <= '2025-08')
conditions_2 = (df_mob_temp.rembursement_status.isin(['已逾期','还款中','续租中','未到首期还款日','检测中']))
# conditoins_3 = (df_mob_temp.purchase_amount != 0)

df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].shape
df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].purchase_amount.sum()
df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].residue_money.sum() 
df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].应付总租金.sum()  # 首租
# df_mob_temp[df_mob_temp.is_relet >= 1].drop_duplicates(subset = ['order_number']).应付总租金.sum()  # 续租
df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].price.sum()
# df_mob_temp[(df_mob_temp.view_month == '2025-08')].当前累计实付租金_all.sum()
tmp1 = df_mob_temp.groupby(['order_number']).agg({'当前累计实付租金_all': 'max'}).reset_index()
tmp1.当前累计实付租金_all.sum()  # 用 "2-所有历史数据-月统计" 的 ， 已收租金 用 总-剩余
# df_mob_temp[(df_mob_temp.view_month == '2025-08') & conditions_2 & (df_mob_temp.overdue_days_mob > 0)].剩余总应收租金.sum()
# df_mob_temp[(df_mob_temp.view_month == '2025-08') & conditions_2 & (df_mob_temp.overdue_days_mob == 0)].剩余总应收租金.sum() #

df_mob_temp[(df_mob_temp.view_month == '2025-08') & (df_mob_temp.overdue_days_mob > 0)].剩余总应收租金.sum()
df_mob_temp[(df_mob_temp.view_month == '2025-08') & (df_mob_temp.overdue_days_mob == 0)].剩余总应收租金.sum() #

(58249, 51)

450903332.96999997

292287.56

370289036.16999996

14800565.0

236918065.17000002

41733860.15

129913676.99000001

#### 2-所有历史数据-月统计

In [ ]:
# （1）下单时数据
# 月份 笔数	采购成本	首次签约价	总租金（含首租、续租）	总租金（首租）	买断尾款	首期租金	预付租金	碎屏险	押金
# 应付总租金：包含首租续租
# 买断尾款 = 首次签约价 - 总租金（首租）

df_mob_temp['dif_days'] = df_mob_temp.apply(lambda x: (pd.to_datetime(x['下单日期']) - pd.to_datetime(x['实付日期new'])).days, axis = 1)
df_mob_temp['是否预付租金'] = np.where((df_mob_temp.dif_days >= -5) & (df_mob_temp.当前期数 > 1), 1, 0)

df_xz_concat['dif_days'] = df_xz_concat.apply(lambda x: (pd.to_datetime(x['下单日期']) - pd.to_datetime(x['实付日期new'])).days, axis = 1)
df_xz_concat['是否预付租金'] = np.where((df_xz_concat.dif_days >= -5) & (df_xz_concat.当前期数 > 1), 1, 0)

conditions_1 = (df_mob_temp.search_time <= '2025-08')
conditions_2 = (df_mob_temp.是否预付租金 == 1)
temp1 = df_mob_temp[(df_mob_temp.当前期数 == 1) & conditions_1].groupby(['search_time']).agg({'order_number':'count'
                                                                                          ,'买断价':'sum','purchase_amount':'sum'
                                                                                          ,'应付总租金':'sum'  # 下单时的应付总租金
                                                                                          ,'实付金额':'sum'    #首期实付金额
                                                                                          ,'price':'sum','residue_money':'sum'}).reset_index()
temp1 = temp1.rename(columns = {'实付金额':'首期实付租金','应付总租金': '下单时应付总租金'})
# temp2 = df_mob_temp[conditions_1 & conditions_2].groupby(['search_time']).agg({'实付金额':'sum'}).reset_index()  # 不全，无 应收日期为 2025.8 之后的
temp2 = df_xz_concat[(df_xz_concat.search_time <= '2025-08') & (df_xz_concat.是否预付租金 == 1)].groupby(['search_time']).agg({'实付金额':'sum'}).reset_index()
# '实付金额','当前应付租金'

# 总租金
# tmp1 = df_mob_temp[df_mob_temp.is_relet >= 1].drop_duplicates(subset = 'order_number')[['order_number','search_time','应付总租金']]
# 限制续租的观察点 &(df_mob_temp.view_month <= '2025-08')
tmp1 = df_mob_temp[(df_mob_temp.is_relet >= 1)&(df_mob_temp.view_month <= '2025-08')].drop_duplicates(subset = 'order_number')[['order_number','search_time','应付总租金']]
tmp2 = df_mob_temp[(df_mob_temp.当前期数 == 1)][['order_number','search_time','应付总租金']]
tmp3 = pd.concat([tmp1, tmp2])
tmp4 = tmp3[tmp3.search_time <= '2025-08'].groupby(['search_time']).agg({'应付总租金': 'sum'}).reset_index()


temp3 = temp1.merge(temp2, on = 'search_time', how = 'outer')
temp3 = temp3.merge(tmp4, on = 'search_time', how = 'left')
temp3 = temp3[['search_time','order_number','purchase_amount','买断价','应付总租金','下单时应付总租金','首期实付租金','实付金额','price','residue_money']]
temp3 = temp3.rename(columns = {'实付金额': '预付金额'})
temp3

,search_time,order_number,purchase_amount,买断价,应付总租金,下单时应付总租金,首期实付租金,预付金额,price,residue_money
0,2022-07,21,117025.00,183994.00,145294.48,121115.43,7034.65,15674.15,0.0,0.00
1,2022-08,53,338531.00,499270.04,370778.91,298639.26,15267.70,58519.48,0.0,0.00
2,2022-09,40,251122.75,363918.00,266786.09,211174.80,14267.85,42288.77,0.0,0.00
3,2022-10,38,224119.52,301447.00,216247.35,171699.21,13516.70,19184.68,0.0,0.00
4,2022-11,41,327485.22,418219.00,334870.28,246703.00,18242.90,34854.30,0.0,0.00
5,2022-12,63,538752.59,704216.20,549762.80,434546.20,27856.90,56459.90,0.0,474.50
6,2023-01,212,1470270.00,2084284.82,1639112.20,1333193.20,71898.85,162243.10,0.0,1506.35
7,2023-02,317,2068328.82,3053671.44,2127919.71,1706203.48,57127.56,243634.09,0.0,1278.00
8,2023-03,302,1985535.00,2920068.23,2137431.17,1653750.27,54550.36,199534.83,0.0,27301.70
9,2023-04,606,4105496.50,6194304.37,4332564.34,3351800.63,101256.01,250433.13,0.0,30428.70


In [79]:
# （2）2025.9.22 增加 
# 截面数据
# 买断订单数	买断已收租金	买断|尾款
# 取买断的时间，订单表的 has_actual可以判断买断
sql1 = '''-- 买断信息   
SELECT too.order_id
,too.real_pay_money 
,too.pay_date
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断
and too.real_pay_money > 0 and too.pay_date is not null
'''
df_out_1 = query(sql1)
df_out_1.shape
df_out_1["pay_date"] = df_out_1['pay_date'].dt.date
df_out_1["pay_date"] = pd.to_datetime(df_out_1["pay_date"],errors="coerce")

temp1 = df_out_1.groupby(['order_id']).agg({'pay_date': 'max'}).reset_index()
temp1.shape

temp2 = df_xz_concat[['order_number','order_id']].drop_duplicates(subset = 'order_number')
temp3 = df_mob_temp.merge(temp2, on = 'order_number', how = 'left')
temp3 = temp3.merge(temp1, on = 'order_id', how = 'left')

temp3.shape
# A2022072701455912 order_id=11499 已买断，但买断表中信息为0，故用 订单状态_new 和 rembursement_status 综合判断是否已买断
# A2023021010432952 order_id=125472 已买断，解约时间是2025.4.14，买断最大日期是2025.9.17
# A20240419165429143 P站2个解约时间

# 买断逻辑变更
# 截止2025.9.21,9月买断266单，2029.9.12-2025.9.21买断259单，故这259单的订单状态不为已买断 ---8387 历史买断
# 2025.9.1后的买断不计算在内
# 续租的或租送的，买断的话，买断尾款是0
# rembursement_status 没有缺失或空
# temp3[(temp3.rembursement_status == '已买断')&((temp3.pay_date < '2025-09-12')|(temp3.pay_date.isnull()))].drop_duplicates(subset = 'order_number').shape
temp3[(temp3.rembursement_status == '已买断')&((temp3.pay_date < '2025-09-01')|(temp3.pay_date.isnull()))].drop_duplicates(subset = 'order_number').shape
temp4 = temp3[(temp3.rembursement_status == '已买断')&((temp3.pay_date < '2025-09-01')|(temp3.pay_date.isnull()))&(temp3.view_month <= '2025-08')].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
temp5 = temp4.merge(temp3[['order_number','search_time','当前期数','当前累计实付租金_all','当前累计已收买断金']], on = ['order_number','当前期数'], how = 'left')
temp5.shape

# df_mob_temp[df_mob_temp.order_number == 'A202207050851373'][['order_number','search_time','rembursement_status','当前期数','当前累计实付租金_all','当前累计已收买断金','买断价','买断价_max']]
temp5[:2]
temp5.groupby(['search_time']).agg({'order_number': 'count', '当前累计实付租金_all': 'sum', '当前累计已收买断金': 'sum'})

df_md = temp5[['order_number']]
df_md['是否买断'] = 1

(9419, 3)

(8923, 2)

(713494, 51)

(8759, 51)

(8759, 5)

,order_number,当前期数,search_time,当前累计实付租金_all,当前累计已收买断金
0,A202207050851373,22.0,2022-07,7847.20,0.00
1,A202207151707153,12.0,2022-07,4799.16,2598.91


,order_number,当前累计实付租金_all,当前累计已收买断金
search_time,,,
2022-07,12,71542.97,21893.02
2022-08,21,164836.72,14475.07
2022-09,18,138375.63,26831.80
2022-10,22,141487.30,30661.00
2022-11,23,179255.96,43972.90
2022-12,39,349123.29,92733.40
2023-01,110,929671.95,197504.80
2023-02,169,1228707.63,433504.09
2023-03,153,1191170.19,333180.02


In [105]:
# （3）2025.9.22 增加 
# 12期时			
# 归还订单	归还订单签约价(尾款+应收租金)	归还订单已收租金
path = 'C:/Users/hxq/Desktop/mine/3_临时分析需求/2.数据需求/'
# 预估残值1：系统的
# 预估残值2：测算的
# 是否归还：1 表示归还
temp1 = pd.read_excel(path + '归还残值_20250909.xlsx')
temp1 = temp1.drop_duplicates(subset = 'order_number')
temp1.shape

temp2 = df_mob_temp[df_mob_temp.view_month <= '2025-08'].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
var_1 = ['order_number','search_time','当前期数','买断价','应付总租金','当前累计实付租金_all','当前累计已收买断金','overdue_days_mob','剩余总应收租金','剩余总应收买断金','rembursement_status']
temp2 = temp2.merge(df_mob_temp[var_1], on = ['order_number','当前期数'], how = 'left')
temp2 = temp2.merge(temp1, on = 'order_number', how = 'left')
temp2['是否归还'] = temp2['是否归还'].fillna(0)
temp2.shape
temp2.是否归还.value_counts()

temp2[temp2.是否归还 == 1].groupby(['search_time']).agg({'order_number':'count','买断价':'sum','当前累计实付租金_all':'sum','预估残值1':'sum'})

df_gh = temp2[temp2.是否归还 == 1][['order_number','是否归还','预估残值1','预估残值2']]

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/hxq/Desktop/mine/3_临时分析需求/2.数据需求/归还残值_20250909.xlsx'

In [ ]:
# （4）每个订单最大的期数对应的观察点及其他信息
var_1 = ['order_number','search_time','view_month','当前期数','买断价','应付总租金','当前累计实付租金_all','当前累计已收买断金'
         ,'overdue_days_mob','剩余总应收租金','剩余总应收买断金','rembursement_status','is_relet','当前累计实付租金']
temp1 = df_mob_temp[df_mob_temp.view_month <= '2025-08'].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
temp1 = temp1.merge(df_mob_temp[var_1], on = ['order_number','当前期数'], how = 'left')
temp1.shape

# 应付总租金：首租的为首租的，续租的为续租的，因为取的是最大期数

temp1 = temp1.merge(df_md, on = 'order_number', how = 'left')
temp1 = temp1.merge(df_gh, on = 'order_number', how = 'left')
temp1.shape

temp2 = df_mob_temp[df_mob_temp.当前期数 == 1][['order_number','应付总租金']]
temp3 = df_mob_temp[df_mob_temp.is_relet > 0][['order_number','应付总租金']]
temp2 = temp2.drop_duplicates(subset = 'order_number')
temp3 = temp3.drop_duplicates(subset = 'order_number')
temp2 = temp2.rename(columns = {'应付总租金':'首租应付总租金'})
temp3 = temp3.rename(columns = {'应付总租金':'续租应付总租金'})

temp1 = temp1.merge(temp2, on = 'order_number', how = 'left')
temp1 = temp1.merge(temp3, on = 'order_number', how = 'left')

temp1['续租应付总租金'] = temp1['续租应付总租金'].fillna(0)
temp1[:1]

# temp1[temp1.search_time == '2022-07'][var_1]

(57990, 14)

(57990, 18)

,order_number,当前期数,search_time,view_month,买断价,应付总租金,当前累计实付租金_all,当前累计已收买断金,overdue_days_mob,剩余总应收租金,剩余总应收买断金,rembursement_status,is_relet,当前累计实付租金,是否买断,是否归还,预估残值1,预估残值2,首租应付总租金,续租应付总租金
0,A202207050851373,22.0,2022-07,2024-04,8176.92,3953.0,7847.2,0.0,0.0,0.0,0.0,已买断,1.0,3551.8,1.0,NaN,NaN,NaN,4295.4,3953.0


In [ ]:
# （5）已收租金	逾期笔数	逾期租金	待收租金（不含逾期）
# 已收租金 用 总-剩余
# 续租笔数：并没有限制观察点

# 旧口径
# conditions_1 = (df_mob_temp.search_time <= '2025-08')
# conditions_2 = (df_mob_temp.rembursement_status.isin(['已逾期','还款中','续租中','未到首期还款日','检测中']))

# temp4 = df_mob_temp.groupby(['order_number']).agg({'当前累计实付租金_all': 'max'}).reset_index()
# temp5 = df_mob_temp[df_mob_temp.当前期数 == 1][['order_number','search_time']].merge(temp4, on = 'order_number', how = 'left')
# temp5.shape

# temp6 = temp5.groupby(['search_time']).agg({'当前累计实付租金_all': 'sum'}).reset_index()  # 已收租金
# temp7 = temp3.merge(temp6, on = 'search_time', how = 'left')

# # temp8 = df_mob_temp[(df_mob_temp.view_month == '2025-08') & conditions_2 & (df_mob_temp.overdue_days_mob > 0)].groupby(['search_time']).agg({'order_number':'count', '剩余总应收租金':'sum'}).reset_index()
# temp8 = df_mob_temp[(df_mob_temp.view_month == '2025-08') & (df_mob_temp.overdue_days_mob > 0)].groupby(['search_time']).agg({'order_number':'count', '剩余总应收租金':'sum'}).reset_index()
# temp8 = temp8.rename(columns = {'order_number':'逾期笔数','剩余总应收租金':'逾期租金'})

# # temp9 = df_mob_temp[(df_mob_temp.view_month == '2025-08') & conditions_2 & (df_mob_temp.overdue_days_mob == 0)].groupby(['search_time']).agg({'剩余总应收租金':'sum'}).reset_index()
# temp9 = df_mob_temp[(df_mob_temp.view_month == '2025-08') & (df_mob_temp.overdue_days_mob == 0)].groupby(['search_time']).agg({'剩余总应收租金':'sum'}).reset_index()
# temp10 = df_mob_temp[df_mob_temp.is_relet >= 1].drop_duplicates(subset = ['order_number'])[['order_number','search_time']]
# temp11 = temp10.groupby(['search_time']).agg({'order_number': 'count'}).reset_index()
# temp11 = temp11.rename(columns = {'order_number':'续租笔数'})

# path = 'C:/Users/hxq/Desktop/mine/3_临时分析需求/2.数据需求/'
# # 预估残值1：系统的
# # 预估残值2：测算的
# # 是否归还：1 表示归还
# temp12 = pd.read_excel(path + '归还残值_20250909.xlsx')
# temp12 = temp12.drop_duplicates(subset = 'order_number')
# temp12.shape
# temp13 = df_mob_temp[df_mob_temp.当前期数 == 1][['order_number','search_time']].merge(temp12, on = 'order_number', how = 'left')
# temp13['是否归还'] = temp13['是否归还'].fillna(0)
# temp14 = temp13[temp13.是否归还 == 1].groupby(['search_time']).agg({'预估残值1': 'sum'}).reset_index()

# temp7 = temp7.merge(temp8, on = 'search_time', how = 'left')
# temp7 = temp7.merge(temp9, on = 'search_time', how = 'left')
# temp7 = temp7.merge(temp11, on = 'search_time', how = 'left')
# temp7 = temp7.merge(temp14, on = 'search_time', how = 'left')

# temp7 = temp7.fillna(0)
# temp7['已收租金_new'] = temp7['应付总租金'] - temp7['逾期租金'] - temp7['剩余总应收租金']
# var_1 = ['search_time','order_number','purchase_amount','买断价','应付总租金','下单时应付总租金','预付金额','price','residue_money','已收租金_new','逾期笔数','逾期租金','剩余总应收租金','续租笔数','预估残值1','当前累计实付租金_all']
# temp7 = temp7[var_1]
# temp7

# # 笔数	购机款-签约价	采购成本	总租金	预付租金	碎屏险	押金	已收租金	逾期笔数	逾期租金	待收租金（不含逾期）	续租笔数	已归还残值

# # 当前累计实付租金_all 不用，用 已收租金_new


,search_time,order_number,purchase_amount,买断价,应付总租金,下单时应付总租金,预付金额,price,residue_money,已收租金_new,逾期笔数,逾期租金,剩余总应收租金,续租笔数,预估残值1,当前累计实付租金_all
0,2022-07,21,117025.00,183994.00,145294.48,121115.43,15674.15,0.0,0.00,104970.04,9.0,40324.44,0.00,5.0,1529.58,97667.93
1,2022-08,53,338531.00,499270.04,370778.91,298639.26,58519.48,0.0,0.00,291164.06,29.0,79614.85,0.00,14.0,8318.80,282051.26
2,2022-09,40,251122.75,363918.00,266786.09,211174.80,42288.77,0.0,0.00,231171.29,19.0,35614.80,0.00,12.0,11958.40,214254.00
3,2022-10,38,224119.52,301447.00,216247.35,171699.21,19184.68,0.0,0.00,190455.02,14.0,25792.33,0.00,13.0,13158.40,183315.54
4,2022-11,41,327485.22,418219.00,334870.28,246703.00,34854.30,0.0,0.00,295601.22,16.0,39269.06,0.00,17.0,19997.60,276697.96
5,2022-12,63,538752.59,704216.20,549762.80,434546.20,56459.90,0.0,474.50,487913.40,20.0,61849.40,0.00,20.0,11778.80,478402.70
6,2023-01,212,1470270.00,2084284.82,1639112.20,1333193.20,162243.10,0.0,1506.35,1416685.20,79.0,222427.00,0.00,61.0,60661.95,1374568.30
7,2023-02,317,2068328.82,3053671.44,2127919.71,1706203.48,243634.09,0.0,1278.00,1880647.40,101.0,247272.31,0.00,69.0,135981.42,1824840.17
8,2023-03,302,1985535.00,2920068.23,2137431.17,1653750.27,199534.83,0.0,27301.70,1858471.91,107.0,278959.26,0.00,68.0,124636.16,1749349.15
9,2023-04,606,4105496.50,6194304.37,4332564.34,3351800.63,250433.13,0.0,30428.70,3754058.01,203.0,578506.33,0.00,133.0,337840.41,3571180.28


In [ ]:
# （6）已收租金	逾期笔数	逾期租金	待收租金（不含逾期）
# 已收租金 用 总应收租金（含首租续租）-剩余总应收租金
# 新口径：买断、归还的订单不包含在内
# 不要用 rembursement_status 来判断订单2025.8.31的状态

conditions_1 = (temp1.view_month == '2025-08') 
conditions_2 = (temp1.overdue_days_mob > 0)
conditions_3 = (temp1.overdue_days_mob == 0)
conditions_4 = (temp1.是否买断 != 1)
conditions_5 = (temp1.是否归还 != 1)

tmp1 = temp1[conditions_1 & conditions_2 & conditions_4 & conditions_5].groupby(['search_time']).agg({'order_number':'count','剩余总应收租金':'sum'}).reset_index()
tmp2 = temp1[conditions_1 & conditions_3 & conditions_4 & conditions_5].groupby(['search_time']).agg({'剩余总应收租金':'sum'}).reset_index()

tmp1 = tmp1.rename(columns = {'order_number': '逾期笔数', '剩余总应收租金': '逾期租金'})
tmp2 = tmp2.rename(columns = {'剩余总应收租金': '待收租金（不含逾期）'})
tmp3 = tmp1.merge(tmp2, on = 'search_time', how = 'outer')
tmp3

,search_time,逾期笔数,逾期租金,待收租金（不含逾期）
0,2022-07,7.0,39756.49,NaN
1,2022-08,28.0,79614.85,NaN
2,2022-09,17.0,35614.80,NaN
3,2022-10,13.0,25792.33,NaN
4,2022-11,11.0,39269.06,NaN
5,2022-12,20.0,61849.40,NaN
6,2023-01,79.0,222427.00,NaN
7,2023-02,100.0,246869.31,NaN
8,2023-03,104.0,274289.50,NaN
9,2023-04,200.0,576443.49,NaN


In [ ]:
# （7）2025.9.22 增加 
# 12期内
# 逾期笔数	逾期订单已收租金	逾期租金	逾期尾款
# 逾期数据限制观察点为 2025.8.31，订单状态优先级 买断/归还 → 续租
conditions_1 = (temp1.view_month == '2025-08') 
conditions_2 = (temp1.overdue_days_mob > 0)
conditions_4 = (temp1.是否买断 != 1)
conditions_5 = (temp1.是否归还 != 1)
conditions_6 = (temp1.is_relet == 0)

tmp4 = temp1[conditions_1 & conditions_2 & conditions_4 & conditions_5 & conditions_6].groupby(['search_time']).agg({'order_number':'count'
                                                                                                                     ,'应付总租金':'sum'
                                                                                                                     ,'剩余总应收租金':'sum'
                                                                                                                     ,'剩余总应收买断金':'sum'}).reset_index()
tmp4['逾期订单已收租金'] = tmp4['应付总租金'] - tmp4['剩余总应收租金']  # 未续租订单的 应付总租金 为首租的

temp1[conditions_1 & conditions_2 & conditions_4 & conditions_5 & conditions_6].当前累计实付租金_all.sum()
tmp4.逾期订单已收租金.sum()
tmp4[['search_time','order_number','逾期订单已收租金','剩余总应收租金','剩余总应收买断金']]

37358086.55

37347951.06

,search_time,order_number,逾期订单已收租金,剩余总应收租金,剩余总应收买断金
0,2022-07,7,20874.67,39756.49,16580.61
1,2022-08,25,84285.47,72011.06,110835.15
2,2022-09,15,48645.12,29769.30,61593.58
3,2022-10,11,36461.50,18938.71,41660.79
4,2022-11,7,35747.10,25520.90,18913.60
5,2022-12,17,85604.20,50409.30,70768.50
6,2023-01,66,262075.95,179082.60,204406.55
7,2023-02,85,320812.26,187124.11,397469.85
8,2023-03,89,287987.94,212967.05,400495.86
9,2023-04,160,565766.79,382277.06,806667.58


In [ ]:
# 2025.9.22 增加 
# 12期时|+6期或者12期							
# 续租笔数	续租前收租金	续租时增加租金	续租未逾期订单	续租正常订单实收租金	续租逾期订单	续租逾期已收租金	续租逾期租金
# 续租笔数限制最大观察点  temp1 已经限制了最大观察点为 2025.8.31
# 逾期数据限制观察点
conditions_1 = (temp1.view_month == '2025-08') 
conditions_2 = (temp1.overdue_days_mob > 0)
conditions_3 = (temp1.overdue_days_mob == 0)
conditions_4 = (temp1.是否买断 != 1)
conditions_5 = (temp1.是否归还 != 1)
conditions_7 = (temp1.is_relet > 0)

tmp5 = temp1[conditions_7].groupby(['search_time']).agg({'order_number':'count'
                                                         ,'首租应付总租金': 'sum'
                                                         ,'续租应付总租金': 'sum'}).reset_index()
# 续租逾期订单
tmp6 = temp1[conditions_1 & conditions_2 & conditions_4 & conditions_5 & conditions_7].groupby(['search_time']).agg({'order_number':'count'
                                                                                                                     ,'续租应付总租金':'sum'
                                                                                                                     ,'当前累计实付租金':'sum'
                                                                                                                     ,'剩余总应收租金':'sum'}).reset_index()
tmp6['续租逾期已收租金'] = tmp6['续租应付总租金'] - tmp6['剩余总应收租金']
# 续租逾期已收租金 和 当前累计实付租金 结果一样
# tmp6.当前累计实付租金.sum()
# tmp6.续租逾期已收租金.sum()
tmp6 = tmp6[['search_time','order_number','当前累计实付租金','剩余总应收租金']]
tmp6 = tmp6.rename(columns = {'order_number': '续租逾期订单','当前累计实付租金':'续租逾期已收租金','剩余总应收租金':'续租逾期租金'})

# 未逾期订单包含完成订单，故不用加时间点
order_list = temp1[conditions_1 & conditions_2 & conditions_4 & conditions_5 & conditions_7].order_number.to_list()
conditions_8 = (~temp1.order_number.isin(order_list))
tmp7 = temp1[conditions_7 & conditions_8].groupby(['search_time']).agg({'order_number':'count'
                                                                        ,'续租应付总租金': 'sum'
                                                                        ,'剩余总应收租金': 'sum'
                                                                        ,'当前累计实付租金': 'sum'}).reset_index()
tmp7['续租正常已收租金'] = tmp7['续租应付总租金'] - tmp7['剩余总应收租金']
# 以下两者稍有差异，用 续租正常已收租金
# tmp7.续租正常已收租金.sum()
# tmp7.当前累计实付租金.sum()
tmp7 = tmp7[['search_time','order_number','续租正常已收租金']]
tmp7 = tmp7.rename(columns = {'order_number': '续租未逾期订单'})

tmp5 = tmp5.merge(tmp7, on = 'search_time', how = 'left')
tmp5 = tmp5.merge(tmp6, on = 'search_time', how = 'left')
tmp5

# A202207131230485 P站显示归还
# A202208231129327 P站显示待赔偿

,search_time,order_number,首租应付总租金,续租应付总租金,续租未逾期订单,续租正常已收租金,续租逾期订单,续租逾期已收租金,续租逾期租金
0,2022-07,5,22561.42,24179.05,5,23611.10,NaN,NaN,NaN
1,2022-08,14,68288.36,72139.65,11,60876.16,3.0,3659.70,7603.79
2,2022-09,12,63208.91,55611.29,10,46737.59,2.0,3028.20,5845.50
3,2022-10,13,55982.50,44548.14,11,34660.92,2.0,3033.60,6853.62
4,2022-11,17,92272.70,88167.28,13,66850.72,4.0,7568.40,13748.16
5,2022-12,20,125693.60,115216.60,17,98358.50,3.0,5418.00,11440.10
6,2023-01,61,304912.80,305919.00,48,232713.00,13.0,29861.60,43344.40
7,2023-02,69,337207.33,421716.23,54,333928.91,15.0,28042.12,59745.20
8,2023-03,68,332134.91,483680.90,53,367068.60,15.0,55289.85,61322.45
9,2023-04,133,639482.81,980763.71,93,674682.04,40.0,111915.24,194166.43


In [ ]:

temp1[temp1.search_time == '2022-08']

,order_number,当前期数,search_time,view_month,买断价,应付总租金,当前累计实付租金_all,当前累计已收买断金,overdue_days_mob,剩余总应收租金,剩余总应收买断金,rembursement_status,is_relet,当前累计实付租金,是否买断,是否归还,预估残值1,预估残值2,首租应付总租金,续租应付总租金
21,A202208011619263,23.0,2022-08,2024-05,12508.80,6698.00,12718.04,0.00,0.0,0.00,0.00,已买断,1.0,6698.00,1.0,NaN,NaN,NaN,6020.04,6698.00
22,A202208021352414,24.0,2022-08,2024-06,8646.00,5089.40,8646.00,0.00,0.0,0.00,0.00,已买断,1.0,5082.00,1.0,NaN,NaN,NaN,3564.00,5089.40
23,A202208031654408,26.0,2022-08,2024-08,11469.60,6530.60,11469.60,0.00,0.0,0.00,0.00,已买断,1.0,6529.56,1.0,NaN,NaN,NaN,4940.04,6530.60
24,A202208032116483,16.0,2022-08,2023-11,10919.00,6150.20,8209.80,0.00,0.0,0.00,0.00,已买断,1.0,2415.00,1.0,NaN,NaN,NaN,5794.80,6150.20
25,A202208040916342,37.0,2022-08,2025-08,7799.00,3564.00,2943.00,0.00,838.0,621.00,4235.00,已逾期,0.0,2943.00,NaN,NaN,NaN,NaN,3564.00,0.00
26,A202208051612571,37.0,2022-08,2025-08,7798.00,7798.68,5080.96,0.00,926.0,2717.72,0.00,已逾期,0.0,5080.96,NaN,NaN,NaN,NaN,7798.68,0.00
27,A202208052103231,37.0,2022-08,2025-08,8839.00,4323.63,4323.63,0.00,750.0,0.00,4515.37,已逾期,0.0,4323.63,NaN,NaN,NaN,NaN,4323.63,0.00
28,A202208061909342,37.0,2022-08,2025-08,7799.00,3564.00,3564.00,0.00,750.0,0.00,4235.00,续租中,0.0,3564.00,NaN,NaN,NaN,NaN,3564.00,0.00
29,A202208071350446,37.0,2022-08,2025-08,11438.00,5895.58,3841.06,0.00,924.0,2054.52,5542.42,已逾期,0.0,3841.06,NaN,NaN,NaN,NaN,5895.58,0.00
30,A202208081149107,37.0,2022-08,2025-08,12739.00,6859.18,2078.54,0.00,1046.0,4780.64,5879.82,已逾期,0.0,2078.54,NaN,NaN,NaN,NaN,6859.18,0.00


In [ ]:
df_mob_temp.columns
df_mob_temp.rembursement_status.value_counts()

Index(['order_number', '下单日期', '租赁方案', '当前期数', '应付日期', '当前应付租金', '实付金额',
       '实付日期new', '买断价', '应付总租金', '买断尾款', '订单状态_new', '当月已收买断金', '当前累计已收买断金',
       '剩余总应收买断金', '提前状态', 'MOB', 'overdue_days_mob', '当前累计应付租金', '当前累计实付租金',
       'search_time', '剩余总应收租金', '剩余总应收', '当前累计实付', '总应收', '当前累计实付租金_all',
       '续租总期数', 'view_month', 'purchase_amount', 'rembursement_status',
       '剩余采购成本', '最大期数', '最大期数_Bin', 'MOB_new', 'MOB_new_1', '是否拒量', 'price',
       'residue_money', 'unrecovered_cost', '剩余采购成本_old', '剩余采购成本_new',
       'is_relet', 'is_relet_max', '期数', '买断价_max', 'id_card_num',
       'province_name', 'age', '性别', 'dif_days', '是否预付租金'],
      dtype='object')

rembursement_status
已逾期        203258
还款中        147261
已买断        119216
续租中         58268
已完成         16070
未到首期还款日      9250
检测中          1656
Name: count, dtype: int64

#### 3-vintage

#### 4-订单统计

In [ ]:
# （1）宽表生成
temp1 = df_xz_concat[['order_number','order_id']].drop_duplicates(subset = 'order_number')
df_mob_temp = df_mob_temp.merge(temp1, on = 'order_number', how = 'left')

df_mob_temp.shape

sql = '''-- 租后应收监控  
SELECT  tprm.order_number, om.begin_date, om.end_date
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
;
'''

df_zhys_tmp = query(sql)
df_zhys_tmp.shape

df_zhys_tmp = df_zhys_tmp.drop_duplicates(subset = 'order_number')
df_zhys_tmp.shape
df_mob_temp = df_mob_temp.merge(df_zhys_tmp, on = 'order_number', how = 'left')
df_mob_temp.shape


df_mob_temp['begin_date'] = pd.to_datetime(df_mob_temp['begin_date'])  # 确保列是datetime类型
df_mob_temp['begin_date_str'] = df_mob_temp['begin_date'].dt.strftime('%Y-%m-%d')
df_mob_temp['end_date'] = pd.to_datetime(df_mob_temp['end_date'])  # 确保列是datetime类型
df_mob_temp['end_date_str'] = df_mob_temp['end_date'].dt.strftime('%Y-%m-%d')

df_mob_temp['租赁时间'] = df_mob_temp['begin_date_str'].str.cat(df_mob_temp['end_date_str'], sep='~')
df_mob_temp = df_mob_temp.drop(columns = ['begin_date', 'begin_date_str', 'end_date', 'end_date_str'])

# 总租金
# 应付总租金_all：含首租和续租的
tmp1 = df_mob_temp[df_mob_temp.is_relet >= 1].drop_duplicates(subset = 'order_number')[['order_number','search_time','应付总租金']]
tmp2 = df_mob_temp[(df_mob_temp.当前期数 == 1)][['order_number','search_time','应付总租金']]
tmp3 = pd.concat([tmp1, tmp2])
tmp4 = tmp3.groupby(['order_number']).agg({'应付总租金': 'sum'}).reset_index()
tmp4 = tmp4.rename(columns = {'应付总租金':'应付总租金_all'})
df_mob_temp = df_mob_temp.merge(tmp4, on = 'order_number', how = 'left')
df_mob_temp.shape


temp1 = df_xz_concat[['order_number','当前期数','逾期天数']]
df_mob_temp = df_mob_temp.merge(temp1, on = ['order_number','当前期数'], how = 'left')
df_mob_temp['当前期数'] = df_mob_temp['当前期数'].astype(int)
df_mob_temp['应付日期'] = pd.to_datetime(df_mob_temp['应付日期'])  # 确保列是datetime类型
df_mob_temp['应付日期_str'] = df_mob_temp['应付日期'].dt.strftime('%Y-%m-%d')

df_mob_temp['实付日期new'] = pd.to_datetime(df_mob_temp['实付日期new'])  # 确保列是datetime类型
df_mob_temp['实付日期new_str'] = df_mob_temp['实付日期new'].dt.strftime('%Y-%m-%d')
df_mob_temp.shape

# df_mob_temp[df_mob_temp.order_number == 'A2024040309475173'] # 严重逾期
# df_mob_temp[df_mob_temp.order_number == 'A2024051707383726']  # 续租


(535500, 52)

(67782, 3)

(67753, 3)

(535500, 54)

(535500, 54)

In [ ]:
# （2）统计1
# 订单id	订单号	下单时间	租赁时间	总期数	总应还租金	已还总租金	剩余应还总租金
temp1 = df_mob_temp[(df_mob_temp.view_month == '2025-08')][['order_number','剩余总应收租金']]
var_1 = ['order_id','order_number','下单日期','search_time','租赁时间','最大期数','应付总租金_all','买断尾款']
temp2 = df_mob_temp[df_mob_temp.当前期数 == 1][var_1].merge(temp1, on = 'order_number', how = 'left')
temp2['剩余总应收租金'] = temp2['剩余总应收租金'].fillna(0)
temp2['已收租金_new'] = temp2['应付总租金_all'] - temp2['剩余总应收租金']
temp2.shape
temp2 = temp2[['order_id','order_number','下单日期','租赁时间','最大期数','应付总租金_all','已收租金_new','剩余总应收租金','买断尾款']]
temp2 = temp2.drop_duplicates(subset = 'order_number')
temp2.shape
temp2[:1]
# search_time = 2024.8的数据 有2W左右对不上

(59346, 10)

(59343, 9)

,order_id,order_number,下单日期,租赁时间,最大期数,应付总租金_all,已收租金_new,剩余总应收租金,买断尾款
0,8924,A202207050851373,2022-07-05,2022-07-05~2024-10-02,24.0,8248.4,8248.4,0.0,3234.6


In [ ]:
# （3）统计2
# 第一期应还时间	第一期实际还款时间	第一期应还金额	第一期实际还款金额	第一期逾期时间(天)

df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期'])  # 确保列是datetime类型
df_xz_concat['应付日期_str'] = df_xz_concat['应付日期'].dt.strftime('%Y-%m-%d')

df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'])  # 确保列是datetime类型
df_xz_concat['实付日期new_str'] = df_xz_concat['实付日期new'].dt.strftime('%Y-%m-%d')


# 应还时间
temp3 = df_xz_concat[["order_number","当前期数","应付日期_str"]].pivot(index="order_number", columns="当前期数", values="应付日期_str").reset_index()
temp3 = temp3.rename(columns = {1:'第一期应还日期', 2:'第二期应还日期', 3:'第三期应还日期', 4:'第四期应还日期', 5:'第五期应还日期', 6:'第六期应还日期'
                                ,7:'第七期应还日期', 8:'第八期应还日期', 9:'第九期应还日期', 10:'第十期应还日期', 11:'第十一期应还日期', 12:'第十二期应还日期'})
var_1 = ['order_number','第一期应还日期','第二期应还日期','第三期应还日期','第四期应还日期','第五期应还日期','第六期应还日期','第七期应还日期','第八期应还日期'
         ,'第九期应还日期','第十期应还日期','第十一期应还日期','第十二期应还日期']
temp3 = temp3[var_1]
# temp3[:1]

# 实还时间
temp4 = df_xz_concat[["order_number","当前期数","实付日期new_str"]].pivot(index="order_number", columns="当前期数", values="实付日期new_str").reset_index()
temp4 = temp4.rename(columns = {1:'第一期实付日期', 2:'第二期实付日期', 3:'第三期实付日期', 4:'第四期实付日期', 5:'第五期实付日期', 6:'第六期实付日期'
                                ,7:'第七期实付日期', 8:'第八期实付日期', 9:'第九期实付日期', 10:'第十期实付日期', 11:'第十一期实付日期', 12:'第十二期实付日期'})
var_1 = ['order_number','第一期实付日期','第二期实付日期','第三期实付日期','第四期实付日期','第五期实付日期','第六期实付日期','第七期实付日期','第八期实付日期'
         ,'第九期实付日期','第十期实付日期','第十一期实付日期','第十二期实付日期']
temp4 = temp4[var_1]
# temp4[:1]

# 应还金额
temp5 = df_xz_concat[["order_number","当前期数","当前应付租金"]].pivot(index="order_number", columns="当前期数", values="当前应付租金").reset_index()
temp5 = temp5.rename(columns = {1:'第一期应还金额', 2:'第二期应还金额', 3:'第三期应还金额', 4:'第四期应还金额', 5:'第五期应还金额', 6:'第六期应还金额'
                                ,7:'第七期应还金额', 8:'第八期应还金额', 9:'第九期应还金额', 10:'第十期应还金额', 11:'第十一期应还金额', 12:'第十二期应还金额'})
var_1 = ['order_number','第一期应还金额','第二期应还金额','第三期应还金额','第四期应还金额','第五期应还金额','第六期应还金额','第七期应还金额','第八期应还金额'
         ,'第九期应还金额','第十期应还金额','第十一期应还金额','第十二期应还金额']
temp5 = temp5[var_1]
# temp5[:1]

# 实付金额
temp6 = df_xz_concat[["order_number","当前期数","实付金额"]].pivot(index="order_number", columns="当前期数", values="实付金额").reset_index()
temp6 = temp6.rename(columns = {1:'第一期实付金额', 2:'第二期实付金额', 3:'第三期实付金额', 4:'第四期实付金额', 5:'第五期实付金额', 6:'第六期实付金额'
                                ,7:'第七期实付金额', 8:'第八期实付金额', 9:'第九期实付金额', 10:'第十期实付金额', 11:'第十一期实付金额', 12:'第十二期实付金额'})
var_1 = ['order_number','第一期实付金额','第二期实付金额','第三期实付金额','第四期实付金额','第五期实付金额','第六期实付金额','第七期实付金额','第八期实付金额'
         ,'第九期实付金额','第十期实付金额','第十一期实付金额','第十二期实付金额']
temp6 = temp6[var_1]
# temp6[:1]

# 逾期
temp7 = df_xz_concat[["order_number","当前期数","逾期天数"]].pivot(index="order_number", columns="当前期数", values="逾期天数").reset_index()
temp7 = temp7.rename(columns = {1:'第一期逾期时间', 2:'第二期逾期时间', 3:'第三期逾期时间', 4:'第四期逾期时间', 5:'第五期逾期时间', 6:'第六期逾期时间'
                                ,7:'第七期逾期时间', 8:'第八期逾期时间', 9:'第九期逾期时间', 10:'第十期逾期时间', 11:'第十一期逾期时间', 12:'第十二期逾期时间'})
var_1 = ['order_number','第一期逾期时间','第二期逾期时间','第三期逾期时间','第四期逾期时间','第五期逾期时间','第六期逾期时间','第七期逾期时间','第八期逾期时间'
         ,'第九期逾期时间','第十期逾期时间','第十一期逾期时间','第十二期逾期时间']
temp7 = temp7[var_1]
# temp7[:1]

var_1 = ['order_id','order_number','下单日期','租赁时间','最大期数','应付总租金_all','已收租金_new','剩余总应收租金'
         ,'第一期应还日期','第一期实付日期','第一期应还金额','第一期实付金额','第一期逾期时间'
         ,'第二期应还日期','第二期实付日期','第二期应还金额','第二期实付金额','第二期逾期时间'
         ,'第三期应还日期','第三期实付日期','第三期应还金额','第三期实付金额','第三期逾期时间'
         ,'第四期应还日期','第四期实付日期','第四期应还金额','第四期实付金额','第四期逾期时间'
         ,'第五期应还日期','第五期实付日期','第五期应还金额','第五期实付金额','第五期逾期时间'
         ,'第六期应还日期','第六期实付日期','第六期应还金额','第六期实付金额','第六期逾期时间'
         ,'第七期应还日期','第七期实付日期','第七期应还金额','第七期实付金额','第七期逾期时间'
         ,'第八期应还日期','第八期实付日期','第八期应还金额','第八期实付金额','第八期逾期时间'
         ,'第九期应还日期','第九期实付日期','第九期应还金额','第九期实付金额','第九期逾期时间'
         ,'第十期应还日期','第十期实付日期','第十期应还金额','第十期实付金额','第十期逾期时间'
         ,'第十一期应还日期','第十一期实付日期','第十一期应还金额','第十一期实付金额','第十一期逾期时间'
         ,'第十二期应还日期','第十二期实付日期','第十二期应还金额','第十二期实付金额','第十二期逾期时间'
         ,'买断尾款'
         ]


temp8 = temp2.merge(temp3, on = 'order_number', how = 'left')
temp8 = temp8.merge(temp4, on = 'order_number', how = 'left')
temp8 = temp8.merge(temp5, on = 'order_number', how = 'left')
temp8 = temp8.merge(temp6, on = 'order_number', how = 'left')
temp8 = temp8.merge(temp7, on = 'order_number', how = 'left')

temp8 = temp8[var_1]
temp8.shape
temp8[:1]

# "2024年4月
# 2024年6月
# 2024年10月
# 2024年12月
# 2025年3月，提供以上5各月份订单数据"							

conditions_1 = (temp8.下单日期 >= '2024-04-01')&(temp8.下单日期 <= '2024-04-30')
conditions_2 = (temp8.下单日期 >= '2024-06-01')&(temp8.下单日期 <= '2024-06-30')
conditions_3 = (temp8.下单日期 >= '2024-10-01')&(temp8.下单日期 <= '2024-10-31')
conditions_4 = (temp8.下单日期 >= '2024-12-01')&(temp8.下单日期 <= '2024-12-31')
conditions_5 = (temp8.下单日期 >= '2025-03-01')&(temp8.下单日期 <= '2025-03-31')

temp9 = temp8[conditions_1 | conditions_2 | conditions_3 | conditions_4 | conditions_5]
temp9['下单日期'] = temp9['下单日期'].astype('str')
temp9.shape


# var_1 = ['order_number','当前期数','应付日期','当前应付租金','实付金额','实付日期new','订单状态_new','应付总租金_all','剩余总应收租金','overdue_days_mob','逾期天数']
# df_mob_temp[df_mob_temp.order_number == 'A2024040309475173'][var_1] # 严重逾期
# df_mob_temp[df_mob_temp.order_number == 'A2024051707383726'][var_1]  # 续租

(59343, 69)

,order_id,order_number,下单日期,租赁时间,最大期数,应付总租金_all,已收租金_new,剩余总应收租金,第一期应还日期,第一期实付日期,第一期应还金额,第一期实付金额,第一期逾期时间,第二期应还日期,第二期实付日期,第二期应还金额,第二期实付金额,第二期逾期时间,第三期应还日期,第三期实付日期,第三期应还金额,第三期实付金额,第三期逾期时间,第四期应还日期,第四期实付日期,第四期应还金额,第四期实付金额,第四期逾期时间,第五期应还日期,第五期实付日期,第五期应还金额,第五期实付金额,第五期逾期时间,第六期应还日期,第六期实付日期,第六期应还金额,第六期实付金额,第六期逾期时间,第七期应还日期,第七期实付日期,第七期应还金额,第七期实付金额,第七期逾期时间,第八期应还日期,第八期实付日期,第八期应还金额,第八期实付金额,第八期逾期时间,第九期应还日期,第九期实付日期,第九期应还金额,第九期实付金额,第九期逾期时间,第十期应还日期,第十期实付日期,第十期应还金额,第十期实付金额,第十期逾期时间,第十一期应还日期,第十一期实付日期,第十一期应还金额,第十一期实付金额,第十一期逾期时间,第十二期应还日期,第十二期实付日期,第十二期应还金额,第十二期实付金额,第十二期逾期时间,买断尾款
0,8924,A202207050851373,2022-07-05,2022-07-05~2024-10-02,24.0,8248.4,8248.4,0.0,2022-07-05,2022-07-05,357.95,357.95,0.0,2022-08-05,2022-08-05,357.95,357.95,0.0,2022-09-05,2022-09-05,357.95,357.95,0.0,2022-10-05,2022-10-09,357.95,357.95,4.0,2022-11-05,2022-11-08,357.95,357.95,3.0,2022-12-05,2022-12-08,357.95,357.95,3.0,2023-01-05,2023-01-07,357.95,357.95,2.0,2023-02-05,2023-02-06,357.95,357.95,1.0,2023-03-05,2023-03-07,357.95,357.95,2.0,2023-04-05,2023-04-01,357.95,357.95,0.0,2023-05-05,2023-05-01,357.95,357.95,0.0,2023-06-05,2023-06-01,357.95,357.95,0.0,3234.6


(10282, 69)

In [ ]:
# （4）统计3
# 第十三期买断应还时间	第十三期买断实际还款时间	第十三期买断应还金额	第十三期买断实际还款金额	第十三期买断逾期时间(天)
# 应付买断尾款  实付买断尾款 实际买断日期
# 以下衍生 实付买断尾款 实际买断日期
# 买断数据 
sql1 = '''-- 买断信息   
SELECT too.order_id
,too.real_pay_money 
,too.pay_date
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断
and too.real_pay_money > 0 and too.pay_date is not null
'''
df_out_1 = query(sql1)
df_out_1.shape
df_out_1["pay_date"] = df_out_1['pay_date'].dt.date
df_out_1["pay_date"] = pd.to_datetime(df_out_1["pay_date"],errors="coerce")

df_out_2 = df_out_1.groupby(['order_id']).agg({'real_pay_money':'sum', 'pay_date': 'max'}).reset_index()
df_out_2.shape

temp10 = temp9.merge(df_out_2, on = 'order_id', how = 'left')
temp10['pay_date'] = temp10['pay_date'].astype('str')
temp10['pay_date'] = pd.to_datetime(temp10['pay_date'])  # 确保列是datetime类型
temp10['pay_date'] = temp10['pay_date'].dt.strftime('%Y-%m-%d')



(7281, 3)

(6917, 3)

In [ ]:
# （5）统计4
# 商品名称	商品规格

# note 2025.4.30
sql = '''-- 租后应收监控  
SELECT  tprm.order_number,tod.product_name,tprm.specification
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
;
'''

temp11 = query(sql)
# (27787, 62)
temp11.shape
temp11 = temp11.drop_duplicates(subset = 'order_number')
temp11.shape

temp12 = temp10.merge(temp11, on = 'order_number', how = 'left')
temp12 = temp12.rename(columns = {'real_pay_money': '实付买断尾款','pay_date': '实际买断日期','product_name': '商品名称','specification': '商品规格'})
temp12.shape
# temp12.to_excel('山东高速_订单明细_0911.xlsx', index = False)

(67782, 3)

(67753, 3)

(10282, 73)

## 5_6.国创资本

### （1）2025.9

#### 1-运营统计

In [ ]:
# 客商样本口径
# 当月申请客户数（发起租赁业务申请客户数） -- 进件数
# 当月审批通过客户数（通过风控审批客户数） -- 机审通过件

# 当月放款客户数（实际发生业务客户数）	
# 当月采购价	
# 当月放款总应收（合计）	
# 当月放款总应收（总租金）	
# 当月放款总应收（尾款）	
# 首逾客户数

# temp1 = df_mob_temp[df_mob_temp.当前期数 == 1].groupby(['search_time']).agg({'order_number':'count','purchase_amount':'sum','买断价':'sum','应付总租金':'sum','买断尾款':'sum'}).reset_index()


temp2 = df_mob_temp[["order_number","当前期数","逾期天数"]].pivot(index="order_number", columns="当前期数", values="逾期天数").reset_index()
temp2 = temp2.rename(columns = {2:'第二期逾期天数'})
temp2 = temp2[['order_number','第二期逾期天数']]
temp2['是否首逾'] = np.where(temp2.第二期逾期天数 >= 1, 1, 0)

var_1 = ['order_number','search_time','purchase_amount','买断价','应付总租金','买断尾款']
temp3 = df_mob_temp[df_mob_temp.当前期数 == 1][var_1].merge(temp2, on = 'order_number', how = 'left')

temp4 = temp3.groupby(['search_time']).agg({'order_number':'count','purchase_amount':'sum','买断价':'sum','应付总租金':'sum','买断尾款':'sum','是否首逾':'sum'}).reset_index()

# 截至统计月末：剩余应收租金的客户数	
# 截至统计月末：剩余总应收租金	
# 截至统计月末：逾期客户数	
# 截至统计月：逾期应收总金额

conditions_2 = (df_mob_temp.rembursement_status.isin(['已逾期','还款中','续租中','未到首期还款日','检测中']))
temp5 = df_mob_temp[(df_mob_temp.view_month == '2025-08')][['order_number','search_time','rembursement_status','剩余总应收租金','剩余总应收','overdue_days_mob']]
# 剩余总应收租金 ： 用的是 应收租金-实收租金，但到期当期的订单，其 剩余总应收租金 为0
# temp5['剩余应收租金的客户数'] = np.where(temp5.剩余总应收租金 > 0, 1 , 0)
temp5['剩余应收租金的客户数'] = np.where(temp5.rembursement_status.isin(['已逾期','还款中','续租中','未到首期还款日','检测中']), 1 , 0)
temp5['逾期客户数'] = np.where((temp5.overdue_days_mob > 0)&(temp5.rembursement_status.isin(['已逾期','还款中','续租中','未到首期还款日','检测中'])), 1 , 0)
temp5['逾期应收总金额'] = np.where((temp5.overdue_days_mob > 0)&(temp5.rembursement_status.isin(['已逾期','还款中','续租中','未到首期还款日','检测中'])), temp5.剩余总应收, 0)

temp6 = temp5.groupby(['search_time']).agg({'剩余应收租金的客户数':'sum','剩余总应收租金':'sum','逾期客户数':'sum','逾期应收总金额':'sum'}).reset_index()

temp7 = temp4.merge(temp6, on = 'search_time', how = 'left')
temp7

# 最近一个月的剩余应收租金的客户数和剩余总应收租金用以下口径（因为8月末下单的，首期应还是9月，用 df_mob_temp.view_month == '2025-08' 取不出来，如 A2025082901172035
df_mob_temp[(df_mob_temp.search_time == '2025-08')&(df_mob_temp.当前期数 == 1)&conditions_2].剩余总应收租金.sum()
df_mob_temp[(df_mob_temp.search_time == '2025-08')&(df_mob_temp.当前期数 == 1)&conditions_2].shape
df_mob_temp[(df_mob_temp.search_time == '2025-08')&(df_mob_temp.当前期数 == 1)&conditions_2].view_month.value_counts()


#  11630 、 A202207282047042 已买断，但买断表没有，故分期表识别不出来，也没有订单完成时间，但 订单表的rembursement_status状态为已买断，故时点的数据可用 rembursement_status 来限制一下

# sql = '''-- 租后应收监控  
# SELECT  tprm.order_number, tprm.order_finish_date
# ,om.order_finish_date as 订单完成时间
# ,too.real_pay_money 
# ,too.pay_date

# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# left join db_digua_business.t_order om on tprm.order_id = om.id
# left join  (select distinct order_id, real_pay_money, pay_date from db_digua_business.t_order_out) too on too.order_id = tprm.order_id

# where tprm.order_number = 'A202207282047042'
# ;
# '''
# query(sql)

,search_time,order_number,purchase_amount,买断价,应付总租金,买断尾款,是否首逾,剩余应收租金的客户数,剩余总应收租金,逾期客户数,逾期应收总金额
0,2022-07,21,117025.00,183994.00,121115.43,62877.02,4,7.0,40324.44,7.0,56337.10
1,2022-08,53,338531.00,499270.04,298639.26,200632.82,19,26.0,79614.85,26.0,175473.85
2,2022-09,40,251122.75,363918.00,211174.80,152743.20,9,16.0,35614.80,16.0,93090.88
3,2022-10,38,224119.52,301447.00,171699.21,129747.79,5,11.0,25792.33,11.0,59826.12
4,2022-11,41,327485.22,418219.00,246703.00,171516.00,10,11.0,39269.06,11.0,58182.66
5,2022-12,63,538752.59,704216.20,434546.20,269670.00,12,20.0,61849.40,20.0,132617.90
6,2023-01,212,1470270.00,2084284.82,1333193.20,739899.62,50,79.0,222427.00,79.0,426833.55
7,2023-02,317,2068328.82,3053671.44,1706203.48,1336709.21,65,100.0,247165.35,100.0,644232.20
8,2023-03,302,1985535.00,2920068.23,1653750.27,1263619.11,31,102.0,278959.26,102.0,655581.57
9,2023-04,606,4105496.50,6194304.37,3351800.63,2842503.74,69,200.0,578506.33,200.0,1383111.07


#### 2-vintage

In [ ]:
# 30+ 用现成的（系统版本）
# 0+ 如下：

# 分子统计
conditions_2 = (df_mob_temp.view_month <= '2025-08')
conditions_3 = (df_mob_temp.search_time <= '2025-07')
conditions_4 = (df_mob_temp.overdue_days_mob > 0)

tmp6_3 = pd.pivot_table(df_mob_temp[conditions_2 & conditions_3 & conditions_4],index='search_time',columns='MOB_new_1',values='剩余采购成本',aggfunc='sum').reset_index()
MOB_list = ['MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
           ,'MOB13','MOB14','MOB15']
# MOB_list = ['MOB1','MOB2','MOB3','MOB4','MOB5','MOB6','MOB7','MOB8','MOB9','MOB10','MOB11','MOB12'
#            ,'MOB13','MOB14','MOB15','MOB16','MOB17','MOB18','MOB19','MOB20','MOB21','MOB22','MOB23','MOB24'
#            ,'MOB25','MOB26','MOB27','MOB28','MOB29','MOB30','MOB31','MOB32','MOB33','MOB34','MOB35','MOB36']
tmp6_3 = tmp6_3[['search_time'] + MOB_list]
tmp6_3.to_excel('tmp6_3.xlsx', index = False)

## 5_7.宁波银行

### （1）2025.9

In [ ]:
# 20个代运营主体的历史出库、在租订单数、签约价、采购价
path = 'C:/Users/hxq/Desktop/mine/12_资方/13_宁波银行/1_材料/'
df_dyy = pd.read_excel(path + '20家主体.xlsx')
df_dyy = df_dyy.rename(columns = {'（账号）名称': 'name'})

df_dyy_zt = df_dyy.name.to_list()
len(df_dyy_zt)

need_zt = ['广州一小时','数字湖科技','智特兴科技','影贝网络科技','广东易代租机','华南通信科技','华南通信二号店'
           ,'乾宝通1号店','乾宝通二号店','乾宝通三号店','乾宝通四号店'
           ,'和鸿润科技','博威世纪科技','星穹智创科技','深圳华一规划','沧渊智创科技'
           ,'瑞和嘉信科技','瑞和嘉信科技二店','瑞和嘉信科技三店','瑞和嘉信科技六店','瑞和嘉信科技五店','瑞和嘉信科技八店','瑞和嘉信科技七店','瑞和嘉信科技九店'
           ,'大龙发一号店','大龙发二号店','大龙发三号店','大龙发六号店','大龙发五号店','大龙发七号店','大龙发八号店','大龙发九号店','大龙发十号店','大龙发十一号店','大龙发十六号店'
           ,'大龙发十八号店','大龙发十九号店','大龙发十三号店','大龙发十五号店','大龙发二十号店','大龙发十七号店','大龙发二十二号店','大龙发二十三号店','大龙发二十五号店','大龙发二十六号店'
           ,'永冠兴六号店','永冠兴三号店','永冠兴九号店','永冠兴八号店','永冠兴十号店','永冠兴五号店','永冠兴二号店','永冠兴七号店','永冠兴十一号店','永冠兴十二号店','永冠兴十三号店','永冠兴十五号店'
           ,'百力一号店','百力科技二号店','百力科技三号店','百力科技五号店','百力科技六号店','百力科技七号店','百力科技八号店'
           ,'中迪晟科技','欧加贸易店','深圳益涛科技','深圳众源店']

# # 取资方名称
# sql = '''-- 租后应收监控  
# SELECT  tprm.order_number
# ,ym.name -- 订单归属资方
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# left join db_digua_business.t_order om on tprm.order_id = om.id
# left join db_digua_business.t_order_risk tor on om.id = tor.order_id
# -- 渠道名称
# left join db_digua_business.t_channel cc on om.channel = cc.scene 
# left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
# ;
# '''

# temp3 = query(sql)
# df_mob_temp_1 = df_mob_temp.merge(temp3, on = 'order_number', how = 'left')

# 统计
conditions_1 = (df_mob_temp_1.name.isin(need_zt))
conditions_2 = (df_mob_temp_1.当前期数 == 1)
conditions_3 = (df_mob_temp_1.view_month == '2025-08')
conditions_4 = (df_mob_temp_1.rembursement_status.isin(['已逾期','还款中','续租中','未到首期还款日','检测中']))

df_mob_temp_1[conditions_1 & conditions_2].shape
df_mob_temp_1[conditions_1 & conditions_2].买断价.sum()
df_mob_temp_1[conditions_1 & conditions_2].purchase_amount.sum()

df_mob_temp_1[conditions_1 & conditions_3 & conditions_4].shape

temp1 = df_mob_temp_1[conditions_1 & conditions_2].groupby(['name']).agg({'order_number': 'count', '买断价':'sum', 'purchase_amount':'sum'}).reset_index()
temp2 = df_mob_temp_1[conditions_1 & conditions_3 & conditions_4].groupby(['name']).agg({'order_number': 'count'}).reset_index()
temp3 = temp1.merge(temp2, on = 'name', how = 'outer')
temp3.to_excel('tmp.xlsx', index = False)

20

(8950, 58)

103972355.17

67408765.75

(6561, 58)

In [ ]:
df_mob_temp.columns

Index(['order_number', '下单日期', '租赁方案', '当前期数', '应付日期', '当前应付租金', '实付金额',
       '实付日期new', '买断价', '应付总租金', '买断尾款', '订单状态_new', '当月已收买断金', '当前累计已收买断金',
       '剩余总应收买断金', '提前状态', 'MOB', 'overdue_days_mob', '当前累计应付租金', '当前累计实付租金',
       'search_time', '剩余总应收租金', '剩余总应收', '当前累计实付', '总应收', '当前累计实付租金_all',
       '续租总期数', 'view_month', 'purchase_amount', 'rembursement_status',
       '剩余采购成本', '最大期数', '最大期数_Bin', 'MOB_new', 'MOB_new_1', '是否拒量', 'price',
       'residue_money', 'unrecovered_cost', '剩余采购成本_old', '剩余采购成本_new',
       'is_relet', 'is_relet_max', '期数', '买断价_max', 'id_card_num',
       'province_name', 'age', '性别', 'dif_days', '是否预付租金', 'order_id', '租赁时间',
       '应付总租金_all', '逾期天数', '应付日期_str', '实付日期new_str'],
      dtype='object')

## 5_8.买断

In [ ]:
# # temp1 = df_goal_temp[df_goal_temp.当前期数 == 1][['order_number', 'search_time']]
# # path = 'C:/Users/hxq/Desktop/mine/3_临时分析需求/2.数据需求/'
# # 预估残值1：系统的
# # 预估残值2：测算的
# # 是否归还：1 表示归还
# temp2 = pd.read_excel(path + '归还残值_20250909.xlsx')
# temp1 = temp1.merge(temp2, on = 'order_number', how = 'left')
# temp1.shape
# temp1 = temp1.drop_duplicates(subset = 'order_number')
# temp1.shape
# temp1.groupby(['search_time']).agg({'是否归还': 'sum'})

In [ ]:
# （1）宽表整合
# 12期时的订单及逾期天数
temp1 = df_goal_temp[df_goal_temp.当前期数 == 12][['order_number','search_time','overdue_days_mob']]
temp1 = temp1.drop_duplicates(subset = 'order_number')
temp1.shape
df_order = temp1.order_number.to_list()

temp2 = df_goal_temp[df_goal_temp.当前期数 < 11][['order_number','search_time']]
temp2 = temp2.drop_duplicates(subset = 'order_number')
temp2['overdue_days_mob'] = 0
temp2.shape
temp2 = temp2[~temp2.order_number.isin(df_order)]
temp2.shape

temp3 = pd.concat([temp1, temp2])
temp3.shape

# 订单表取买断
sql = '''-- 租后应收监控  
SELECT  tprm.order_number
,om.has_actual

from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
;
'''
temp4 = query(sql)
temp4.shape
temp4 = temp4.drop_duplicates(subset = 'order_number')
temp4.shape

# 分期表取已续租
temp5 = df_xz_concat[df_xz_concat.is_relet >= 1][['order_number']]
temp5 = temp5.drop_duplicates(subset = 'order_number')
temp5['是否续租'] = 1
temp5.shape

# 打归还标签
path = 'C:/Users/hxq/Desktop/mine/3_临时分析需求/2.数据需求/'
# 预估残值1：系统的
# 预估残值2：测算的
# 是否归还：1 表示归还
temp6 = pd.read_excel(path + '归还残值_20250909.xlsx')
temp6.shape
temp6 = temp6.drop_duplicates(subset = 'order_number')
temp6 = temp6[['order_number','是否归还']]
temp6.shape

# 表合并
temp7 = temp3.merge(temp4, on = 'order_number', how = 'left')
temp7 = temp7.merge(temp5, on = 'order_number', how = 'left')
temp7 = temp7.merge(temp6, on = 'order_number', how = 'left')
temp7.shape
temp7[:1]

(20577, 3)

(63211, 3)

(42634, 3)

(63211, 3)

(67909, 2)

...

(7793, 2)

(60895, 4)

(60880, 2)

(63211, 6)

,order_number,search_time,overdue_days_mob,has_actual,是否续租,是否归还
0,A202207050851373,2022-07,0.0,1,1.0,0.0


In [ ]:
# （2）统计

temp7.has_actual.value_counts()
temp7.是否续租.value_counts()
temp7.是否归还.value_counts()

# temp7['是否续租'] = np.where(temp7.has_actual == 1, 0, temp7.是否续租)
# temp7['是否续租'] = np.where(temp7.是否归还 == 1, 0, temp7.是否续租)

temp7[temp7.overdue_days_mob == 0].groupby(['search_time']).agg({'order_number':'count','是否归还':'sum','has_actual':'sum','是否续租':'sum'})

# temp7[(temp7.overdue_days_mob == 0)&(temp7.search_time == '2022-07')]

has_actual
0    54794
1     8417
Name: count, dtype: int64

是否续租
0.0    9685
1.0    5294
Name: count, dtype: int64

是否归还
0.0    59597
1.0     1268
Name: count, dtype: int64

,order_number,是否归还,has_actual,是否续租
search_time,,,,
2022-07,14,1.0,11,0.0
2022-08,35,3.0,20,4.0
2022-09,34,4.0,18,4.0
2022-10,33,4.0,22,2.0
2022-11,35,6.0,23,5.0
2022-12,51,4.0,39,4.0
2023-01,156,23.0,108,13.0
2023-02,255,49.0,162,17.0
2023-03,228,47.0,148,16.0


## 5_9.和运

### （1）2025.9

In [ ]:
# （1）剔除样本后的 df_mob_temp 对应的订单的台账发送

In [ ]:
# （2-1）保理额度评估 --数据不发送，用 df_goal_temp
# all
# 下单日期	出库订单数	采购成本	签约价	总租金	买断尾款	首期实付租金	预付租金	逾期金额_成本	逾期金额_应收
# 由于大订单的问题，单独更新 2025.8 的预付金额
df_xz_concat[(df_xz_concat.search_time == '2025-08')&(df_xz_concat.当前期数 >= 10)&(df_xz_concat.实付日期new <= '2025-09-15')].实付金额.sum()

df_xz_concat['dif_days'] = df_xz_concat.apply(lambda x: (pd.to_datetime(x['下单日期']) - pd.to_datetime(x['实付日期new'])).days, axis = 1)
df_xz_concat['是否预付租金'] = np.where((df_xz_concat.dif_days >= -5) & (df_xz_concat.当前期数 > 1), 1, 0)
temp1 = df_xz_concat[(df_xz_concat.search_time <= '2025-08') & (df_xz_concat.是否预付租金 == 1)].groupby(['search_time']).agg({'实付金额':'sum'}).reset_index()
temp2 = df_goal_temp[(df_goal_temp.search_time <= '2025-08') & (df_goal_temp.当前期数 == 1)].groupby(['search_time']).agg({'order_number':'count'
                                                                                                                       ,'purchase_amount':'sum'
                                                                                                                       ,'买断价':'sum'
                                                                                                                       ,'应付总租金':'sum'
                                                                                                                       ,'买断尾款':'sum'
                                                                                                                       ,'实付金额':'sum'}).reset_index()

temp1 = temp1.rename(columns = {'实付金额':'预付租金'})
temp2 = temp2.rename(columns = {'实付金额':'首期实付租金'})

temp3 = temp2.merge(temp1, on = 'search_time', how = 'left')

# 逾期统计
# 每个订单最大的期数对应的观察点及其他信息
var_1 = ['order_number','search_time','view_month','当前期数','overdue_days_mob','剩余总应收租金','剩余总应收','剩余采购成本_old','剩余总应收买断金']
temp4 = df_goal_temp[df_goal_temp.view_month <= '2025-08'].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
temp4 = temp4.merge(df_goal_temp[var_1], on = ['order_number','当前期数'], how = 'left')
temp4.shape

temp4 = temp4.merge(df_md, on = 'order_number', how = 'left')
temp4 = temp4.merge(df_gh, on = 'order_number', how = 'left')
temp4.shape

conditions_1 = (temp4.view_month == '2025-08') 
conditions_2 = (temp4.overdue_days_mob > 0)
conditions_3 = (temp4.overdue_days_mob == 0)
conditions_4 = (temp4.是否买断 != 1)
conditions_5 = (temp4.是否归还 != 1)

temp5 = temp4[conditions_1 & conditions_2 & conditions_4 & conditions_5].groupby(['search_time']).agg({'剩余采购成本_old':'sum','剩余总应收':'sum'}).reset_index()

temp6 = temp3.merge(temp5, on = 'search_time', how = 'left')
temp6
# df_xz_concat[(df_xz_concat.search_time == '2025-08') & (df_xz_concat.当前期数 == 1)][['order_number','实付金额']].to_excel('tmp.xlsx', index = False)
# df_xz_concat[(df_xz_concat.search_time == '2025-08') & (df_xz_concat.当前期数 == 12)][['order_number','下单日期','实付日期new','dif_days','实付金额']].to_excel('tmp.xlsx', index = False)

(61858, 9)

(61858, 13)

,search_time,order_number,purchase_amount,买断价,应付总租金,买断尾款,首期实付租金,预付租金,剩余采购成本_old,剩余总应收
0,2022-07,22,125405.00,195693.00,132814.08,62877.02,7522.10,15674.15,37664.28,64490.70
1,2022-08,56,358451.00,527606.04,319267.29,208449.79,16393.32,58519.48,102786.70,206828.93
2,2022-09,44,277982.75,402654.00,234001.22,168652.78,15825.45,42288.77,51555.91,114735.37
3,2022-10,41,249543.09,337968.00,192968.21,144999.79,15264.50,19184.68,43475.05,74985.52
4,2022-11,43,344258.81,440057.00,258335.00,181722.00,19198.70,34854.30,47859.38,76380.96
5,2022-12,66,566276.59,740353.20,460878.20,279475.00,30020.80,56459.90,88372.60,154567.70
6,2023-01,221,1533459.00,2173455.52,1393924.10,766642.52,73875.75,162243.10,225107.30,485115.85
7,2023-02,332,2181552.82,3218625.44,1798841.81,1408651.08,59740.11,243634.09,339779.40,744362.69
8,2023-03,316,2083021.00,3063400.23,1734257.67,1326443.71,57112.51,199534.83,358944.87,772240.54
9,2023-04,634,4317651.50,6511461.37,3517788.91,2993672.46,105603.44,250433.13,701522.47,1584826.50


## 5_10.财务需求

### （1）2025.9

In [ ]:
# 维客自营账户+鸿丰
# 2025.9.23
# 出库订单数、逾期订单数、逾期金额（应收）、签约价、逾期率（金额口径）、逾期率（订单口径），按月统计
# 2025.9.25
# df_mob_temp.to_parquet('F:\需求\总需求\df_mob_temp.parquet', engine='pyarrow')
# 1、按年统计，进件数、订单出库数、采购金额、签约价-自营
# 2、25年按月统计，进件数、订单出库数、采购金额、签约价-自营
# 3、应收未收：区分纯自营、代运营，纯自营 指维客账户、鸿丰、越榕、金锤
# 4、维客账户、鸿丰、越榕、金锤 按月统计 订单出库数、采购金额、签约价
temp1 = df_xz_concat[['order_number','merchant_name']]
temp1 = temp1.drop_duplicates(subset = 'order_number')
temp2 = df_goal_temp.merge(temp1, on = 'order_number', how = 'left')

sql = '''-- 租后应收监控  
SELECT  tprm.order_number
,ym.name -- 订单归属资方
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
;
'''

temp3 = query(sql)
temp2 = temp2.merge(temp3, on = 'order_number', how = 'left')

conditions_1 = (temp2.view_month == '2025-10')
# temp2[conditions_1].剩余总应收.sum()

# name 维客自营账户   --  merchant_name 迪瓜优选
# 海南鸿丰商业保理有限公司
# 杭州金锤网络信息服务有限公司
# 成都越蓉金科科技有限公司

# temp2[conditions_1 & (temp2.merchant_name == '成都越蓉金科科技有限公司')].shape
# temp2[conditions_1 & (temp2.name == '成都越蓉金科科技有限公司')].shape

# conditions_3 = (temp2.merchant_name.isin(['迪瓜优选','海南鸿丰商业保理有限公司','杭州金锤网络信息服务有限公司','成都越蓉金科科技有限公司']))
# temp2[conditions_1 & conditions_3].剩余总应收.sum()

# conditions_2 = (temp2.name.isin(['维客自营账户','海南鸿丰商业保理有限公司','杭州金锤网络信息服务有限公司','成都越蓉金科科技有限公司']))
# temp2[conditions_1 & conditions_2].剩余总应收.sum()

temp4 = temp2[temp2.当前期数 == 1][['order_number','name','purchase_amount','买断价','search_time']]
temp4['type'] = np.where(temp4.name == '维客自营账户', '1.维客自营账户', '5.代运营')
temp4['type'] = np.where(temp4.name == '海南鸿丰商业保理有限公司', '2.海南鸿丰商业保理有限公司', temp4.type)
temp4['type'] = np.where(temp4.name == '杭州金锤网络信息服务有限公司', '3.杭州金锤网络信息服务有限公司', temp4.type)
temp4['type'] = np.where(temp4.name == '成都越蓉金科科技有限公司', '4.成都越蓉金科科技有限公司', temp4.type)

temp5 = temp2[temp2.view_month == '2025-10'][['order_number','剩余总应收','overdue_days_mob']]  # 固定了观察点
temp6 = temp4.merge(temp5, on = 'order_number', how = 'left')
temp6 = temp6.drop_duplicates(subset = 'order_number')
temp6 = temp6.merge(df_md, on = 'order_number', how = 'left')
temp6 = temp6.merge(df_gh, on = 'order_number', how = 'left')

temp4.shape
temp5.shape
temp6.shape
temp6[temp6.search_time <= '2025-10'].shape


# # 资方
conditions_4 = (temp6.是否买断 != 1)
conditions_5 = (temp6.是否归还 != 1)
# temp7 = temp6[temp6.search_time <= '2025-08'].groupby(['type']).agg({'order_number':'count','purchase_amount':'sum','买断价':'sum','剩余总应收':'sum'}).reset_index()
# temp8 = temp6[(temp6.search_time <= '2025-08')&(temp6.overdue_days_mob > 0)&conditions_4&conditions_5].groupby(['type']).agg({'剩余总应收':'sum'}).reset_index()
# temp8 = temp8.rename(columns = {'剩余总应收': '应收未收_逾期'})

# temp9 = temp7.merge(temp8, on = 'type', how = 'left')

# temp9.to_excel('tmp.xlsx', index = False)
# temp9

# 2025.9.23
# 资方*下单月
temp10 = temp6[(temp6.search_time <= '2025-08')&(temp6.type == '2.海南鸿丰商业保理有限公司')].groupby(['search_time']).agg({'order_number':'count','买断价':'sum'}).reset_index()
temp11 = temp6[(temp6.search_time <= '2025-08')&(temp6.type == '2.海南鸿丰商业保理有限公司')&(temp6.overdue_days_mob > 0)&conditions_4&conditions_5].groupby(['search_time']).agg({'order_number':'count','剩余总应收':'sum'}).reset_index()
temp11 = temp11.rename(columns = {'order_number':'逾期订单数','剩余总应收': '应收未收_逾期'})
temp12 = temp10.merge(temp11, on = 'search_time', how = 'left')
temp12.to_excel('tmp1.xlsx', index = False)

# 
temp10 = temp6[(temp6.search_time <= '2025-08')&(temp6.type == '1.维客自营账户')].groupby(['search_time']).agg({'order_number':'count','买断价':'sum'}).reset_index()
temp11 = temp6[(temp6.search_time <= '2025-08')&(temp6.type == '1.维客自营账户')&(temp6.overdue_days_mob > 0)].groupby(['search_time']).agg({'order_number':'count','剩余总应收':'sum'}).reset_index()
temp11 = temp11.rename(columns = {'order_number':'逾期订单数','剩余总应收': '应收未收_逾期'})
temp12 = temp10.merge(temp11, on = 'search_time', how = 'left')

temp12.to_excel('tmp2.xlsx', index = False)

### （2）2025.10



In [99]:

# df_mob_temp.to_parquet('F:\需求\总需求\df_mob_temp.parquet', engine='pyarrow')
# 1、按年统计，进件数、订单出库数、采购金额、签约价-自营
# 2、25年按月统计，进件数、订单出库数、采购金额、签约价-自营
# 3、应收未收：区分纯自营、代运营，纯自营 指维客账户、鸿丰、越榕、金锤
# 4、维客账户、鸿丰、越榕、金锤 按月统计 订单出库数、采购金额、签约价
temp1 = df_xz_concat[['order_number','merchant_name']]
temp1 = temp1.drop_duplicates(subset = 'order_number')
temp2 = df_goal_temp.merge(temp1, on = 'order_number', how = 'left')

sql = '''-- 租后应收监控  
SELECT  tprm.order_number
,ym.name -- 订单归属资方
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
;
'''

temp3 = query(sql)
temp2 = temp2.merge(temp3, on = 'order_number', how = 'left')




temp4 = temp2[temp2.当前期数 == 1][['order_number','name','purchase_amount','买断价','search_time']]
temp4['type'] = np.where(temp4['name'].str.contains('维客自营账户|海南鸿丰商业保理有限公司|杭州金锤网络信息服务有限公司|成都越蓉金科科技有限公司'),'纯自营','代运营')
# temp4 = temp4[temp4.type == '纯自营']
temp4 = temp4.drop_duplicates(subset = 'order_number')
temp4.loc[:, '年份'] = pd.to_datetime(temp4.search_time).dt.year
# 自营按年分组，不存在进件列，需要另外统计，进件可以通过日报表中获取
# tempg = temp4.groupby('年份').agg({'order_number': 'count', 'purchase_amount': 'sum', '买断价': 'sum'}).rename(columns={'order_number': '出库数', 'purchase_amount': '采购金额', '买断价': '签约价'})
# tempg#.to_excel('F:\需求\总需求\自营业务量_2025.11.24.xlsx')
# 纯自营按月分组
tempg = temp4.groupby('search_time').agg({'order_number': 'count', 'purchase_amount': 'sum', '买断价': 'sum'}).rename(columns={'order_number': '出库数', 'purchase_amount': '采购金额', '买断价': '签约价'})
tempg
# .to_excel('F:\需求\总需求\自营业务量_2025.11.24.xlsx')


,出库数,采购金额,签约价
search_time,,,
2022-07,22,125405.00,195693.00
2022-08,56,358451.00,527606.04
2022-09,44,277982.75,402654.00
2022-10,41,249543.09,337968.00
2022-11,43,344258.81,440057.00
2022-12,20,165907.59,214203.00
2023-01,204,1409394.00,2003642.10
2023-02,180,1143310.82,1705241.44
2023-03,87,532338.00,795607.00


In [97]:
temp5 = temp2[temp2.view_month == '2025-11'][['order_number','剩余总应收','overdue_days_mob']]  # 固定了观察点
temp6 = temp4.merge(temp5, on = 'order_number', how = 'left')
temp6 = temp6.drop_duplicates(subset = 'order_number', keep = 'last')
temp6 = temp6.merge(df_md, on = 'order_number', how = 'left')
# temp6 = temp6.merge(df_gh, on = 'order_number', how = 'left')
temp6[temp6.search_time <= '2025-11'].shape

# # 资方
conditions_4 = (temp6.是否买断 != 1)
# conditions_5 = (temp6.是否归还 != 1)
# temp7 = temp6[temp6.search_time <= '2025-08'].groupby(['type']).agg({'order_number':'count','purchase_amount':'sum','买断价':'sum','剩余总应收':'sum'}).reset_index()
# temp8 = temp6[(temp6.search_time <= '2025-08')&(temp6.overdue_days_mob > 0)&conditions_4&conditions_5].groupby(['type']).agg({'剩余总应收':'sum'}).reset_index()
# temp8 = temp8.rename(columns = {'剩余总应收': '应收未收_逾期'})

# temp9 = temp7.merge(temp8, on = 'type', how = 'left')

# temp9.to_excel('tmp.xlsx', index = False)
# temp9

# 2025.9.23
# 资方*下单月&conditions_4&conditions_5
# temp10 = temp6[(temp6.search_time <= '2025-11')].groupby(['search_time']).agg({'order_number':'count','买断价':'sum'}).reset_index()
# 分别统计纯自营、代运营应收未收金额
temp11 = temp6[(temp6.search_time <= '2025-11')].groupby(['type']).agg({'order_number':'count','剩余总应收':'sum'}).reset_index()
# temp11
# temp11 = temp11.rename(columns = {'order_number':'逾期订单数','剩余总应收': '应收未收_逾期'})
temp11
# temp12 = temp10.merge(temp11, on = 'search_time', how = 'left')
# temp12.to_excel('tmp1.xlsx', index = False)


# temp10 = temp6[(temp6.search_time <= '2025-08')&(temp6.type == '1.维客自营账户')].groupby(['search_time']).agg({'order_number':'count','买断价':'sum'}).reset_index()
# temp11 = temp6[(temp6.search_time <= '2025-08')&(temp6.type == '1.维客自营账户')&(temp6.overdue_days_mob > 0)&conditions_4&conditions_5].groupby(['search_time']).agg({'order_number':'count','剩余总应收':'sum'}).reset_index()
# temp11 = temp11.rename(columns = {'order_number':'逾期订单数','剩余总应收': '应收未收_逾期'})
# temp12 = temp10.merge(temp11, on = 'search_time', how = 'left')

(18960, 10)

,type,order_number,剩余总应收
0,纯自营,18960,80905395.82


In [94]:
temp11.to_excel('F:\需求\总需求/tmp2.xlsx', index = False)

In [96]:
temp2[temp2.order_number=='A202207050851373'][['view_month', '剩余总应收', '剩余总应收租金']]

,view_month,剩余总应收,剩余总应收租金
0,2022-07,7172.05,3937.45
1,2022-08,6814.10,3579.50
2,2022-09,6456.15,3221.55
3,2022-10,6098.20,2863.60
4,2022-11,5740.25,2505.65
5,2022-12,5382.30,2147.70
6,2023-01,5024.35,1789.75
7,2023-02,4666.40,1431.80
8,2023-03,4308.45,1073.85
9,2023-04,3950.50,715.90


In [229]:
# df_mob_temp['age_bins'] = pd.cut(df_mob_temp['age'],[0,22,30,40,55,np.inf])
# df_mob_temp[(df_mob_temp['overdue_days_mob']>30)&(df_mob_temp['view_month']=='2025-09')].drop_duplicates(subset='order_number').groupby('age_bins').agg({'order_number':'count'})
# df_mob_temp[(df_mob_temp['overdue_days_mob']>60)&(df_mob_temp['view_month']=='2025-09')].drop_duplicates(subset='order_number').groupby('age_bins').agg({'order_number':'count'})
# df_mob_temp[(df_mob_temp['overdue_days_mob']>90)&(df_mob_temp['view_month']=='2025-09')].drop_duplicates(subset='order_number').groupby('age_bins').agg({'order_number':'count'})

# df_mob_temp[(df_mob_temp['overdue_days_mob']>30)&(df_mob_temp['view_month']=='2025-09')].drop_duplicates(subset='order_number').groupby('性别').agg({'order_number':'count'})
# df_mob_temp[(df_mob_temp['overdue_days_mob']>60)&(df_mob_temp['view_month']=='2025-09')].drop_duplicates(subset='order_number').groupby('性别').agg({'order_number':'count'})
# df_mob_temp[(df_mob_temp['overdue_days_mob']>90)&(df_mob_temp['view_month']=='2025-09')].drop_duplicates(subset='order_number').groupby('性别').agg({'order_number':'count'})

# df_mob_temp['xx'] = df_mob_temp.apply(lambda x: int(x.id_card_num[:2]), axis = 1)
# df_mob_temp['province_name_new'] = df_mob_temp.apply(lambda row: province_func(row['xx']), axis=1)
# df_mob_temp[(df_mob_temp['overdue_days_mob']>30)&(df_mob_temp['view_month']=='2025-09')].drop_duplicates(subset='order_number').groupby('province_name_new').agg({'order_number':'count'}).to_clipboard()
# df_mob_temp[(df_mob_temp['overdue_days_mob']>60)&(df_mob_temp['view_month']=='2025-09')].drop_duplicates(subset='order_number').groupby('province_name_new').agg({'order_number':'count'}).to_clipboard()
# df_mob_temp[(df_mob_temp['overdue_days_mob']>90)&(df_mob_temp['view_month']=='2025-09')].drop_duplicates(subset='order_number').groupby('province_name_new').agg({'order_number':'count'}).to_clipboard()

In [ ]:
df_md.loc[:, '是否买断'] = np.where(df_md.has_actual == 1, 1, 0)

In [ ]:
# A2024110123471583  第4期 实付日期new 为空，但系统有显示已还

### (2) 2025.8(已失效)

In [101]:
df_goal_temp.columns

Index(['order_number', '下单日期', '租赁方案', '当前期数', '应付日期', '当前应付租金', '实付金额',
       '实付日期new', '买断价', '应付总租金', '买断尾款', '订单状态_new', '当月已收买断金', '当前累计已收买断金',
       '剩余总应收买断金', '提前状态', 'MOB', 'overdue_days_mob', '当前累计应付租金', '当前累计实付租金',
       'search_time', '剩余总应收租金', '剩余总应收', '当前累计实付', '总应收', '当前累计实付租金_all',
       '续租总期数', 'view_month', 'purchase_amount', 'rembursement_status',
       '剩余采购成本', '最大期数', '最大期数_Bin', 'MOB_new', 'MOB_new_1', '是否拒量', 'price',
       'residue_money', 'unrecovered_cost', '剩余采购成本_old', '剩余采购成本_new',
       'is_relet', 'is_relet_max', '期数', '买断价_max', 'id_card_num',
       'province_name', 'age', '性别'],
      dtype='object')

In [108]:
import pandas as pd

# 假设你的DataFrame名为df
# 获取列名（返回Index对象）
columns = df_xz_concat.columns
# 转换为列表形式（更直观）
column_names = df_xz_concat.columns.tolist()

# 打印列名
print("列名列表：", column_names)

列名列表： ['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '实付日期', 'status_x', '当前期数', 'order_pay_id', 'create_time_x', 'update_time_x', 'type_x', 'advance_time', 'merchant_account_id', 'order_relet_id', 'is_relet', 'pay_type', 'sesame_promo_money', 'sesame_promo_money_pay', 'promo_money', 'plat_subsidy', 'promo_money_show', '应付日期', 'id_y', 'order_number', 'order_time', 'search_time', 'end_date', 'merchant_id', 'merchant_name', 'channel_id', 'channel_name', 'model_number', 'specification', 'purchase_amount', 'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages', 'buyout_balance_payment', 'advance_periods', 'advance_sum', 'order_create_time', 'rembursement_status', 'paid_periods', 'begin_overdue_time', 'overdue_periods', 'max_overdue_days', 'now_overdue_days', 'total_receivable', 'contract_price', 'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment', 'total_received', 'total_unreceived', 'unrecovered_cost', 'overdue_type', 'gross_profit_receivable'

In [205]:
## 观察点2025-8月底逾期，即条件为view_month = '2025-08'&overdue_days_mob > 0
# 观察点2025-8月底的应收未收金额统计， view_month = '2025-08'&实付日期new<"2025-08-31",如果end

temp1 = df_xz_concat[['order_number','end_date']]
temp1 = temp1.drop_duplicates(subset = 'order_number')
temp2 = df_goal_temp.merge(temp1, on = 'order_number', how = 'left')

sql = '''-- 租后应收监控  
SELECT  tprm.order_number
,ym.name -- 订单归属资方
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
;
'''

temp3 = query(sql)
temp2 = temp2.merge(temp3, on = 'order_number', how = 'left')

conditions_1 = (temp2.view_month == '2025-08')
conditions_2 = (temp2.view_month == '2025-08') & ((temp2.实付日期new>"2025-08-31")|(temp2.实付日期new.isnull()))

temp4 = temp2[conditions_2][['order_number','name','剩余总应收租金','剩余总应收','end_date']]

temp4.loc[:, "资方"] = np.where(temp4['name']=='维客自营账户', '维客自营账户', np.where(temp4['name']=='海南鸿丰商业保理有限公司', '海南鸿丰商业保理有限公司','其他'))
temp4['应收未收'] = np.where(temp4.end_date<='2025-08-31', temp4.剩余总应收, temp4.剩余总应收租金)

temp5 = temp2[conditions_1][['order_number','view_month','overdue_days_mob','search_time','下单日期','rembursement_status']]

temp6 = temp5.merge(temp4, on = 'order_number', how = 'left')

temp6['年份'] = temp6['下单日期'].dt.year
# 筛选出观察点2025-08-31逾期订单
# df_merge['是否逾期'] = np.where((df_merge['年份'].isin([2022, 2023]))&(df_merge['status']==8), 0, df_merge['是否逾期'])
temp6.loc[:, "是否逾期"] = np.where((temp6['overdue_days_mob']>0), 1, 0)#&(temp6['overdue']==1)
conditions_3 = (temp6['rembursement_status']=='已完成')|(temp6['rembursement_status']=='已买断')
temp6['是否逾期'] = np.where(((temp6['年份'].isin([2022, 2023]))&conditions_3), 0, temp6['是否逾期'])

df_all = temp6[temp6.是否逾期 == 1]

df_all[(df_all['search_time']=='2023-11')&(df_all['资方'] == '维客自营账户')]




,order_number,view_month,overdue_days_mob,search_time,下单日期,rembursement_status,name,剩余总应收租金,剩余总应收,end_date,资方,应收未收,年份,是否逾期
2261,A20231101005524111,2025-08,510.0,2023-11,2023-11-01,已逾期,维客自营账户,5307.60,5307.60,2024-11-03,维客自营账户,5307.60,2023,1
2262,A2023110107482670,2025-08,388.0,2023-11,2023-11-01,已逾期,维客自营账户,900.23,5099.21,2024-11-03,维客自营账户,5099.21,2023,1
2263,A2023110108403958,2025-08,510.0,2023-11,2023-11-01,已逾期,维客自营账户,2369.10,5821.24,2024-11-03,维客自营账户,5821.24,2023,1
2264,A2023110110032411,2025-08,541.0,2023-11,2023-11-01,已逾期,维客自营账户,3517.45,7339.18,2024-11-03,维客自营账户,7339.18,2023,1
2265,A2023110110264054,2025-08,632.0,2023-11,2023-11-01,已逾期,维客自营账户,6662.90,11662.40,2024-11-03,维客自营账户,11662.40,2023,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49245,A2023112820562846,2025-08,154.0,2023-11,2023-11-28,已逾期,维客自营账户,2990.19,2990.19,2025-10-15,维客自营账户,2990.19,2023,1
49250,A2023112912180637,2025-08,59.0,2023-11,2023-11-29,已逾期,维客自营账户,599.20,599.20,2025-09-03,维客自营账户,599.20,2023,1
49258,A2023112917465175,2025-08,176.0,2023-11,2023-11-29,已逾期,维客自营账户,2604.96,2604.96,2025-09-08,维客自营账户,2604.96,2023,1
49259,A2023112920270028,2025-08,209.0,2023-11,2023-11-29,已逾期,维客自营账户,3182.06,3182.06,2025-08-11,维客自营账户,3182.06,2023,1


In [204]:
temp2[temp2.order_number == 'A2023110214064522'][['order_number','name','剩余总应收租金','剩余总应收','end_date','view_month','实付日期new']]



,order_number,name,剩余总应收租金,剩余总应收,end_date,view_month,实付日期new
116869,A2023110214064522,维客自营账户,5955.97,12455.38,2025-08-17,2023-11,2023-11-02
116870,A2023110214064522,维客自营账户,5414.65,11914.06,2025-08-17,2023-12,2023-12-05
116871,A2023110214064522,维客自营账户,4332.01,10831.42,2025-08-17,2024-01,2024-01-04
116872,A2023110214064522,维客自营账户,3790.69,10290.10,2025-08-17,2024-02,2024-01-22
116873,A2023110214064522,维客自营账户,3249.37,9748.78,2025-08-17,2024-03,2024-02-22
...,...,...,...,...,...,...,...
611719,A2023110214064522,维客自营账户,1994.05,1994.05,2025-08-17,2025-06,NaT
611720,A2023110214064522,维客自营账户,1994.05,1994.05,2025-08-17,2025-07,NaT
611721,A2023110214064522,维客自营账户,1994.05,1994.05,2025-08-17,2025-08,2025-09-03
611722,A2023110214064522,维客自营账户,1994.05,1994.05,2025-08-17,2025-09,NaT


#### 维客自营账户

In [219]:

df_wk = df_all[(df_all['资方'] == '维客自营账户')]
df_wk_g = df_wk.groupby(['search_time']).agg({'是否逾期': 'sum', '应收未收': 'sum'}).rename(columns={'是否逾期': '逾期订单数', '应收未收': '应收未收金额'})
df_wk_g.loc[:, '订单平均逾期金额'] = df_wk_g['应收未收金额'] / df_wk_g['逾期订单数']

df_wk_g.loc['汇总', :] = df_wk_g.sum()
df_wk_g[:20]

,逾期订单数,应收未收金额,订单平均逾期金额
search_time,,,
2022-07,7.0,53970.48,7710.068571
2022-08,29.0,191852.78,6615.613103
2022-09,20.0,110617.87,5530.893500
2022-10,12.0,67358.52,5613.210000
2022-11,13.0,76380.96,5875.458462
2022-12,3.0,17905.20,5968.400000
2023-01,80.0,440527.60,5506.595000
2023-02,60.0,397995.12,6633.252000
2023-03,30.0,229450.60,7648.353333


In [213]:
# df_wk[(df_wk['search_time']=='2023-11')].to_excel('F:\需求\总需求/tmp.xlsx', index = False)


In [175]:
# # df_mob_temp[df_mob_temp['order_number']=='A202207200249265'][['view_month','overdue_days_mob']]
# tmp2 = df_mob_temp[(df_mob_temp['is_relet']==0)&(df_mob_temp['view_month']=='2025-09')&(df_mob_temp['overdue_days_mob']!=0)].drop_duplicates(subset='order_number',keep='last')
# df_t =  tmp2.groupby('search_time').agg(
#         {'order_number':'count','剩余总应收租金':'sum','剩余总应收买断金':'sum'})
# df_t.to_excel('F:\需求\总需求/tmp2.xlsx', index = False)

#### 海南鸿丰商业保理有限公司

In [220]:
df_hf = df_all[(df_all['资方'] == '海南鸿丰商业保理有限公司')&(df_all['是否逾期']==1)]
df_hf_g = df_hf.groupby(['search_time']).agg({'是否逾期': 'sum', '应收未收': 'sum'}).rename(columns={'是否逾期': '逾期订单数', '应收未收': '应收未收金额'})
df_hf_g.loc[:, '订单平均逾期金额'] = df_hf_g['应收未收金额'] / df_hf_g['逾期订单数']
df_hf_g.loc['汇总', :] = df_hf_g.sum()
df_hf_g 

,逾期订单数,应收未收金额,订单平均逾期金额
search_time,,,
2024-12,82.0,326175.82,3977.753902
2025-01,20.0,85520.17,4276.008500
2025-02,22.0,92433.61,4201.527727
2025-03,60.0,248487.62,4141.460333
2025-04,66.0,312529.93,4735.301970
2025-05,64.0,336327.84,5255.122500
2025-06,62.0,316653.28,5107.310968
2025-07,43.0,237806.45,5530.382558
汇总,419.0,1955934.72,37224.868459


In [221]:
df_qt = df_all[(df_all['资方'] == '其他')&(df_all['是否逾期']==1)]
df_qt_g = df_qt.groupby(['search_time']).agg({'是否逾期': 'sum', '应收未收': 'sum'}).rename(columns={'是否逾期': '逾期订单数', '应收未收': '应收未收金额'})
df_qt_g.loc[:, '订单平均逾期金额'] = df_qt_g['应收未收金额'] / df_qt_g['逾期订单数']
df_qt_g.loc['汇总', :] = df_qt_g.sum()
df_qt_g 

,逾期订单数,应收未收金额,订单平均逾期金额
search_time,,,
2022-12,20.0,136662.50,6833.125000
2023-01,7.0,39852.25,5693.178571
2023-02,53.0,341360.27,6440.759811
2023-03,82.0,513778.77,6265.594756
2023-04,20.0,183757.78,9187.889000
...,...,...,...
2025-04,750.0,4236267.32,5648.356427
2025-05,655.0,3795070.10,5794.000153
2025-06,515.0,3018135.45,5860.457184


In [222]:
# with pd.ExcelWriter('F:\需求\倩姐需求\自营逾期业务量_10.22.xlsx') as writer:
#     df_wk_g.to_excel(writer, sheet_name='维客自营账户')
#     df_hf_g.to_excel(writer, sheet_name='海南鸿丰商业保理有限公司')
#     df_qt_g.to_excel(writer, sheet_name='其他')
    

### (3) 2025-8月底逾期订单，即条件为view_month = '2025-08'（2025-11）&overdue_days_mob > 0

按年月(search_time)，维客壹佰、鸿丰、金锤、越蓉、代自营分（merchant_name），只统计逾期(overdue_days_mob > 0)的订单 
* 1、首租(is_relet == 0)： 
    - （1）应收未收租金：只要逾期了，所有的未收租金都包含在内 
    - （2）租送残值：采购成本（purchase_amount）/12未交租期数（实付日期为空的最大期数-最小期数）
    - （3）租归残值1：采购成本/24未交租期数 
    - （4）租归残值2：采购成本/2
* 2、续租(is_relet != 0)： 
    + （1）应收未收租金：只要逾期了，所有的未收租金都包含在内 
    + （2）续租残值3：采购成本/2/续租期数*未交租期数
* 3、交付
下单年月 应收未收租金 租送残值 租归残值1 租归残值2 续租残值3  总残值，5个sheet页，其中 总残值 = 租送残值 + 租归残值1 + 租归残值2 + 续租残值3

* 4、预收
    + 应付日期>11.30 & 实付日期不为空 & 当前期数不为1

In [82]:
# df_xz_concat[df_xz_concat.order_id==186902][['search_time', 'merchant_name', 'is_relet', '逾期天数', 'purchase_amount', '实付日期', '应付日期', '当前期数', 'all_rental', 'real_pay_money']]
# 拼接观察点2025-08的数据
df_cw_merge = df_xz_concat[['order_id', '当前期数', '实付日期new', '应付日期', 'is_relet', 'order_number', 'search_time', 'merchant_name', 'all_rental', 'purchase_amount', 'rembursement_status']].merge(
    df_goal_temp[df_goal_temp.view_month == '2025-11'][['下单日期', 'order_number', '租赁方案', 'overdue_days_mob', '剩余总应收租金', '剩余总应收', 'view_month']], on = ['order_number'], how = 'left')
# 未交期数=实付日期为空的最大当前期数-实付日期为空的最小当前期数
# 1. 计算每个order_number下实付日期为空的最大sort
max_sort_0 = df_cw_merge[df_cw_merge['实付日期new'].isnull()].groupby('order_number')['当前期数'].max()
# 2. 计算每个order_number下实付日期为空的最小sort
min_sort_0 = df_cw_merge[df_cw_merge['实付日期new'].isnull()].groupby('order_number')['当前期数'].min()
df_cw_merge['未交期数'] = df_cw_merge['order_number'].map(max_sort_0) - df_cw_merge['order_number'].map(min_sort_0) + 1
df_cw_merge['未交期数'] = df_cw_merge['未交期数'].fillna(0)
# 1. 计算每个order_number下is_relet=0的最大sort（首租最大）
max_sort_sz = df_cw_merge[df_cw_merge['is_relet'] == 0].groupby('order_number')['当前期数'].max().fillna(0)

# 2. 计算每个order_number下is_relet=1的最大sort（续租最大）
max_sort_xz = df_cw_merge[df_cw_merge['is_relet'] != 0].groupby('order_number')['当前期数'].max().fillna(0)

# 3. 计算续租期数（续租最大 - 首租最大），并通过order_number映射回原DataFrame
df_cw_merge['续租期数'] = np.where(df_cw_merge.is_relet != 0, df_cw_merge['order_number'].map(max_sort_xz) - df_cw_merge['order_number'].map(max_sort_sz), 0)
# 4. 统计当期是否逾期（实付日期new>应付日期），并计算每个order_number的逾期期数 2025-12-11
df_cw_merge['是否逾期'] = np.where(df_cw_merge['实付日期new'] > df_cw_merge['应付日期'], 1, 0)
df_cw_merge['逾期期数'] = df_cw_merge.groupby('order_number')['是否逾期'].transform('sum')
# 5.统计是否预收，应付日期>11.30 & 实付日期不为空 & 当前sort不为1，，并计算每个order_number的预收期数 2025-12-12
df_cw_merge['是否预收'] = np.where((df_cw_merge['应付日期'] > '2025-11-30') & (df_cw_merge['实付日期new'] < '2025-11-30') & (df_cw_merge['当前期数'] != 1), 1, 0)
df_cw_merge['预收期数'] = df_cw_merge.groupby('order_number')['是否预收'].transform('sum')

# 按order_number,当前期数升序排序
df_cw_merge = df_cw_merge.sort_values(by=['order_number', '当前期数'], ascending=[True, True])
# 按order_number,当前期数去重
df_cw_merge = df_cw_merge.drop_duplicates(subset=['order_number'], keep='last')
# df_cw_merge[df_cw_merge.order_id == 186902]
df_cw_merge[df_cw_merge.逾期期数 >= 1].tail()

,order_id,当前期数,实付日期new,应付日期,is_relet,order_number,search_time,merchant_name,all_rental,purchase_amount,rembursement_status,下单日期,租赁方案,overdue_days_mob,剩余总应收租金,剩余总应收,view_month,未交期数,续租期数,是否逾期,逾期期数,是否预收,预收期数
1016168,4795334,12.0,2025-12-11,2026-11-08,0.0,A20251205184716410,2025-12,武汉中科国研,0.0,0.0,已完成,NaT,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,1,0,0
1016648,4797178,12.0,2025-12-11,2026-11-09,0.0,A2025120603110722,2025-12,山东融越（再生纪元）,0.0,0.0,已完成,NaT,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,1,0,0
1016996,4798506,12.0,2025-12-11,2026-11-09,0.0,A20251206111726182,2025-12,山东融越（再生纪元）,0.0,0.0,已完成,NaT,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,1,0,0
1017068,4798858,12.0,2025-12-11,2026-11-09,0.0,A2025120612021717,2025-12,山东融越（再生纪元）,0.0,0.0,已完成,NaT,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,1,0,0
1017800,4800562,12.0,2025-12-10,2026-11-09,0.0,A20251206152517266,2025-12,山东融越（再生纪元）,0.0,0.0,已完成,NaT,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,1,0,0


In [83]:
df_cw_merge[df_cw_merge.order_id == 4795334]

,order_id,当前期数,实付日期new,应付日期,is_relet,order_number,search_time,merchant_name,all_rental,purchase_amount,rembursement_status,下单日期,租赁方案,overdue_days_mob,剩余总应收租金,剩余总应收,view_month,未交期数,续租期数,是否逾期,逾期期数,是否预收,预收期数
1016168,4795334,12.0,2025-12-11,2026-11-08,0.0,A20251205184716410,2025-12,武汉中科国研,0.0,0.0,已完成,NaT,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,1,0,0


In [156]:
df_cw_merge[df_cw_merge.未交期数.isna()]

,order_id,当前期数,实付日期new,应付日期,is_relet,order_number,search_time,merchant_name,all_rental,purchase_amount,rembursement_status,下单日期,租赁方案,overdue_days_mob,剩余总应收租金,剩余总应收,view_month,未交期数,续租期数


In [89]:
df_cw_merge.loc[:, '是否逾期'] = np.where((df_cw_merge['overdue_days_mob'] > 0), 1, 0)
# 仅获取逾期订单
df_cw_merge['年份'] = df_cw_merge['下单日期'].dt.year
conditions_3 = (df_cw_merge['rembursement_status']=='已完成')|(df_cw_merge['rembursement_status']=='已买断')
df_cw_merge['是否逾期'] = np.where(((df_cw_merge['年份'].isin([2022, 2023]))&conditions_3), 0, df_cw_merge['是否逾期'])
df_cw_merge_t = df_cw_merge[df_cw_merge['是否逾期'] == 1]
# 计算首租订单各残值
    # - （2）租送残值：采购成本（purchase_amount）/12未交租期数（实付日期为空的最大期数-最小期数）
    # - （3）租归残值1：采购成本/24未交租期数 
    # - （4）租归残值2：采购成本/2
df_cw_merge_t.loc[:, '租送残值'] = np.where(df_cw_merge_t.租赁方案=='租完即送', df_cw_merge_t['purchase_amount'] / 12 * df_cw_merge_t['未交期数'], 0)
    # - （5）租送逾期残值：采购成本/12*逾期期数 2025-12-11
df_cw_merge_t.loc[:, '租送逾期残值'] = np.where(df_cw_merge_t.租赁方案=='租完即送', df_cw_merge_t['purchase_amount'] / 12 * df_cw_merge_t['逾期期数'], 0)
    # + （6）租送预收残值：采购成本/12*预收期数 2025-12-11
df_cw_merge_t.loc[:, '租送预收残值'] = np.where(df_cw_merge_t.租赁方案=='租完即送', df_cw_merge_t['purchase_amount'] / 12 * df_cw_merge_t['预收期数'], 0)
df_cw_merge_t.loc[:, '租归残值1'] = np.where(df_cw_merge_t.租赁方案=='租完归还', df_cw_merge_t['purchase_amount'] / 24 * df_cw_merge_t['未交期数'], 0)
df_cw_merge_t.loc[:, '租归残值2'] = np.where(df_cw_merge_t.租赁方案=='租完归还', df_cw_merge_t['purchase_amount'] / 2, 0)
    # - （7）租归逾期残值：采购成本/24*逾期期数 
df_cw_merge_t.loc[:, '租归逾期残值'] = np.where((df_cw_merge_t.租赁方案=='租完归还'), df_cw_merge_t['purchase_amount'] / 24 * df_cw_merge_t['逾期期数'], 0)
    # + （8）租归预收残值：采购成本/24*预收期数
df_cw_merge_t.loc[:, '租归预收残值'] = np.where((df_cw_merge_t.租赁方案=='租完归还'), df_cw_merge_t['purchase_amount'] / 24 * df_cw_merge_t['预收期数'], 0)
# 计算续租订单各残值
    # + （2）续租残值3：采购成本/2/续租期数*未交租期数 续租仅有租归
df_cw_merge_t.loc[:, '续租残值3'] = np.where((df_cw_merge_t.租赁方案=='租完归还')&(df_cw_merge_t['is_relet'] != 0), df_cw_merge_t['purchase_amount'] / 2 / df_cw_merge_t['续租期数'] * df_cw_merge_t['未交期数'], 0)
    # + （3）续租逾期残值：采购成本/2/续租期数*逾期期数
df_cw_merge_t.loc[:, '续租逾期残值'] = np.where((df_cw_merge_t.租赁方案=='租完归还')&(df_cw_merge_t['is_relet'] != 0), df_cw_merge_t['purchase_amount'] / 2 / df_cw_merge_t['续租期数'] * df_cw_merge_t['逾期期数'], 0)
    # + （4）续租预收残值：采购成本/2/续租期数*预收期数
df_cw_merge_t.loc[:, '续租预收残值'] = np.where((df_cw_merge_t.租赁方案=='租完归还')&(df_cw_merge_t['is_relet'] != 0), df_cw_merge_t['purchase_amount'] / 2 / df_cw_merge_t['续租期数'] * df_cw_merge_t['预收期数'], 0)

# 计算总残值
df_cw_merge_t.loc[:, '总残值'] = df_cw_merge_t['租送残值'] + df_cw_merge_t['租归残值1'] + df_cw_merge_t['租归残值2'] + df_cw_merge_t['续租残值3']
df_cw_merge_t.loc[:, '首租逾期残值'] = df_cw_merge_t['租送逾期残值'] + df_cw_merge_t['租归逾期残值']
df_cw_merge_t.loc[:, '总预收残值'] = df_cw_merge_t['租送预收残值'] + df_cw_merge_t['租归预收残值'] + df_cw_merge_t['续租预收残值']
# 资方 '杭州金锤网络信息服务有限公司','成都越蓉金科科技有限公司','海南鸿丰商业保理有限公司','
df_cw_merge_t.loc[:, '资方'] = np.where(df_cw_merge_t['merchant_name']=='迪瓜优选', '维客自营账户', 
                                    np.where(df_cw_merge_t['merchant_name']=='海南鸿丰商业保理有限公司', '海南鸿丰商业保理有限公司',
                                            np.where(df_cw_merge_t['merchant_name']== '杭州金锤网络信息服务有限公司', '杭州金锤网络信息服务有限公司',
                                                    np.where(df_cw_merge_t['merchant_name'] == '成都越蓉金科科技有限公司', '成都越蓉金科科技有限公司', '代自营'))))


In [81]:
# 筛选出所有重复的订单记录
df_cw_merge_t[(df_cw_merge_t.search_time=='2022-07')&(df_cw_merge_t['资方'] == '维客自营账户')&(df_cw_merge_t['租赁方案']=='租完归还')]


,order_id,当前期数,实付日期new,应付日期,is_relet,order_number,search_time,merchant_name,all_rental,purchase_amount,rembursement_status,下单日期,租赁方案,overdue_days_mob,剩余总应收租金,剩余总应收,view_month,未交期数,续租期数,是否逾期,逾期期数,年份,租送残值,租送逾期残值,租归残值1,租归残值2,租归逾期残值,续租残值3,续租逾期残值,总残值,资方
249,11564,12.0,2022-07-29,2023-06-27,0.0,A202207272231334,2022-07,迪瓜优选,6119.98,8280.0,已逾期,2022-07-27,租完归还,1177.0,4798.62,10377.64,2025-11,9.0,0.0,1,1,2022.0,0.0,0.0,3105.0,4140.0,345.00,0.0,0.0,7245.0,维客自营账户
298,11667,12.0,2022-07-29,2023-06-28,0.0,A2022072823485315,2022-07,迪瓜优选,6957.62,9030.0,续租中,2022-07-28,租完归还,870.0,0.00,5781.38,2025-11,0.0,0.0,1,1,2022.0,0.0,0.0,0.0,4515.0,376.25,0.0,0.0,4515.0,维客自营账户


#### 维客自营账户

In [90]:
df_wk = df_cw_merge_t[(df_cw_merge_t['资方'] == '维客自营账户')]
df_wk_g = df_wk.groupby(['search_time']).agg({'是否逾期': 'sum', '剩余总应收租金': 'sum', '租送残值': 'sum', '租归残值1': 'sum', '租归残值2': 'sum', '续租残值3': 'sum', 
                                            '总残值': 'sum', '租送逾期残值':'sum', '租归逾期残值': 'sum', '首租逾期残值':'sum', '续租逾期残值': 'sum', '总预收残值': 'sum'}
                                            ).rename(columns={'search_time': '下单年月', '是否逾期': '逾期订单数', '剩余总应收租金': '应收未收租金'})
# df_wk_g.loc[:, '订单平均逾期金额'] = df_wk_g['应收未收租金'] / df_wk_g['逾期订单数']
df_wk_g.loc['汇总', :] = df_wk_g.select_dtypes(include=['float64', 'int64']).sum()

df_wk_g

,逾期订单数,应收未收租金,租送残值,租归残值1,租归残值2,续租残值3,总残值,租送逾期残值,租归逾期残值,首租逾期残值,续租逾期残值,总预收残值
search_time,,,,,,,,,,,,
2022-07,7.0,42610.08,2.538792e+04,3.105000e+03,8.655000e+03,0.000000e+00,3.714792e+04,1171.666667,7.212500e+02,1.892917e+03,0.000000e+00,0.000000
2022-08,28.0,88209.32,1.744125e+04,4.031746e+04,8.592400e+04,7.987942e+03,1.516706e+05,3394.583333,2.094825e+04,2.434283e+04,5.356124e+03,0.000000
2022-09,20.0,47154.00,0.000000e+00,2.778067e+04,6.324553e+04,6.699033e+03,9.772523e+04,0.000000,1.725323e+04,1.725323e+04,9.319227e+03,0.000000
2022-10,12.0,29874.73,0.000000e+00,1.897902e+04,4.120788e+04,4.793397e+03,6.498029e+04,0.000000,1.080560e+04,1.080560e+04,2.001799e+03,0.000000
2022-11,13.0,47261.36,1.189750e+04,2.043471e+04,4.502109e+04,9.012455e+03,8.636576e+04,5077.500000,1.228141e+04,1.735890e+04,4.994045e+03,0.000000
2022-12,3.0,7692.00,6.126667e+03,0.000000e+00,9.210000e+03,0.000000e+00,1.533667e+04,1531.666667,3.236667e+03,4.768333e+03,0.000000e+00,0.000000
2023-01,80.0,230585.50,7.241292e+04,6.653354e+04,2.075850e+05,2.293845e+04,3.694699e+05,34865.833333,4.687771e+04,8.174354e+04,2.051274e+04,0.000000
2023-02,60.0,176220.33,1.565833e+04,8.006250e+04,1.998325e+05,2.046000e+04,3.160133e+05,4954.166667,3.734562e+04,4.229979e+04,1.418861e+04,0.000000
2023-03,30.0,104235.04,1.474875e+04,4.155633e+04,1.004640e+05,8.394167e+03,1.651632e+05,1280.000000,4.934375e+03,6.214375e+03,6.608333e+02,0.000000


In [104]:
df_wk[(df_wk.search_time=='2025-08')&(df_wk.是否预收==1)][['order_number','预收期数', 'purchase_amount']]

,order_number,预收期数,purchase_amount
794444,A20250805224700268,2,8500.0


#### 海南鸿丰商业保理有限公司

In [92]:
df_hf = df_cw_merge_t[(df_cw_merge_t['资方'] == '海南鸿丰商业保理有限公司')]
df_hf_g = df_hf.groupby(['search_time']).agg({'是否逾期': 'sum', '剩余总应收租金': 'sum', '租送残值': 'sum', '租归残值1': 'sum', '租归残值2': 'sum', '续租残值3': 'sum', '总残值': 'sum', '租送逾期残值':'sum', '租归逾期残值': 'sum', '首租逾期残值':'sum', '续租逾期残值': 'sum', '总预收残值': 'sum'}
                                            ).rename(columns={'search_time': '下单年月', '是否逾期': '逾期订单数', '剩余总应收租金': '应收未收租金'})
# df_hf_g.loc[:, '订单平均逾期金额'] = df_hf_g['应收未收租金'] / df_hf_g['逾期订单数']
df_hf_g.loc['汇总', :] = df_hf_g.select_dtypes(include=['float64', 'int64']).sum()

df_hf_g

,逾期订单数,应收未收租金,租送残值,租归残值1,租归残值2,续租残值3,总残值,租送逾期残值,租归逾期残值,首租逾期残值,续租逾期残值,总预收残值
search_time,,,,,,,,,,,,
2024-12,91.0,317571.82,11616.666667,1.887983e+05,410115.0,0.0,6.105300e+05,7871.666667,52790.000000,60661.666667,0.0,0.000000
2025-01,32.0,92480.73,7500.000000,5.074646e+04,132737.5,0.0,1.909840e+05,0.000000,20204.166667,20204.166667,0.0,10085.625000
2025-02,36.0,105126.70,0.000000,6.819583e+04,159377.5,0.0,2.275733e+05,0.000000,23297.500000,23297.500000,0.0,12543.125000
2025-03,94.0,315372.27,0.000000,2.041242e+05,407666.5,0.0,6.117908e+05,0.000000,43699.208333,43699.208333,0.0,33527.000000
2025-04,115.0,429963.83,6607.500000,2.659629e+05,500437.5,0.0,7.730079e+05,734.166667,60263.958333,60998.125000,0.0,43478.541667
2025-05,107.0,488021.08,5188.750000,2.913510e+05,467172.5,0.0,7.637123e+05,741.250000,40839.583333,41580.833333,0.0,43972.916667
2025-06,167.0,719606.80,6240.000000,4.411673e+05,692112.5,0.0,1.139520e+06,0.000000,55662.916667,55662.916667,0.0,49312.916667
2025-07,238.0,1098100.08,14650.000000,6.695358e+05,1002375.0,0.0,1.686561e+06,0.000000,71698.750000,71698.750000,0.0,82462.500000
2025-08,143.0,749836.11,29454.166667,4.362717e+05,598772.5,0.0,1.064498e+06,1791.666667,27021.666667,28813.333333,0.0,51388.125000


In [106]:
df_hf[(df_hf.search_time=='2025-08')&(df_hf.是否预收==1)][['order_number','预收期数', 'purchase_amount']]

,order_number,预收期数,purchase_amount
784784,A20250801095108346,1,8400.0
782984,A20250801104440653,1,8420.0
783044,A2025080111012124,1,8420.0
783380,A20250801124946731,1,8420.0
784196,A20250801164009859,1,8420.0
...,...,...,...
822740,A20250818150426114,1,8770.0
822884,A20250818154123946,1,8760.0
823148,A202508181658071189,1,8770.0
823196,A20250818170520100,1,8410.0


#### 杭州金锤网络信息服务有限公司

In [93]:
df_jc = df_cw_merge_t[(df_cw_merge_t['资方'] == '杭州金锤网络信息服务有限公司')]
df_jc_g = df_jc.groupby(['search_time']).agg({'是否逾期': 'sum', '剩余总应收租金': 'sum', '租送残值': 'sum', '租归残值1': 'sum', '租归残值2': 'sum', '续租残值3': 'sum', '总残值': 'sum', '租送逾期残值':'sum', '租归逾期残值': 'sum', '首租逾期残值':'sum', '续租逾期残值': 'sum', '总预收残值': 'sum'}
                                            ).rename(columns={'search_time': '下单年月', '是否逾期': '逾期订单数', '剩余总应收租金': '应收未收租金'})
# df_jc_g.loc[:, '订单平均逾期金额'] = df_jc_g['应收未收租金'] / df_jc_g['逾期订单数']
df_jc_g.loc['汇总', :] = df_jc_g.select_dtypes(include=['float64', 'int64']).sum()

df_jc_g

,逾期订单数,应收未收租金,租送残值,租归残值1,租归残值2,续租残值3,总残值,租送逾期残值,租归逾期残值,首租逾期残值,续租逾期残值,总预收残值
search_time,,,,,,,,,,,,
2024-08,2.0,10400.32,0.000000,6.015875e+03,7577.0,0.000000,1.359288e+04,0.000000,0.000000,0.000000,0.000000,0.0
2024-09,292.0,971843.64,60890.583333,4.203069e+05,1037206.0,133495.953588,1.651899e+06,9704.833333,185319.333333,195024.166667,55314.044247,0.0
2024-10,111.0,341147.67,12450.000000,1.528346e+05,439480.0,55754.166667,6.605188e+05,16600.833333,54468.958333,71069.791667,9163.333333,0.0
2024-11,217.0,596943.53,51740.833333,2.784842e+05,900267.5,47507.678571,1.278000e+06,11055.833333,115721.041667,126776.875000,14180.571429,0.0
2024-12,128.0,431214.98,25385.833333,2.469396e+05,559162.5,0.000000,8.314879e+05,6510.000000,76012.500000,82522.500000,0.000000,0.0
汇总,750.0,2351550.14,150467.250000,1.104581e+06,2943693.0,236757.798826,4.435499e+06,43871.500000,431521.833333,475393.333333,78657.949009,0.0


In [ ]:
df_jc[df_jc.search_time=='2024-08']
### 成都越蓉金科科技有限公司

#### 成都越蓉金科科技有限公司

In [94]:
df_yr = df_cw_merge_t[(df_cw_merge_t['资方'] == '成都越蓉金科科技有限公司')]
df_yr_g = df_yr.groupby(['search_time']).agg({'是否逾期': 'sum', '剩余总应收租金': 'sum', '租送残值': 'sum', '租归残值1': 'sum', '租归残值2': 'sum', '续租残值3': 'sum', '总残值': 'sum', '租送逾期残值':'sum', '租归逾期残值': 'sum', '首租逾期残值':'sum', '续租逾期残值': 'sum', '总预收残值': 'sum'}
                                            ).rename(columns={'search_time': '下单年月', '是否逾期': '逾期订单数', '剩余总应收租金': '应收未收租金'})
# df_yr_g.loc[:, '订单平均逾期金额'] = df_yr_g['应收未收租金'] / df_yr_g['逾期订单数']
df_yr_g.loc['汇总', :] = df_yr_g.select_dtypes(include=['float64', 'int64']).sum()

df_yr_g

,逾期订单数,应收未收租金,租送残值,租归残值1,租归残值2,续租残值3,总残值,租送逾期残值,租归逾期残值,首租逾期残值,续租逾期残值,总预收残值
search_time,,,,,,,,,,,,
2024-04,159.0,451828.07,0.0,235816.875000,625420.5,103001.655303,9.642390e+05,0.0,126944.291667,126944.291667,87070.643434,0.000000
2024-05,96.0,231162.38,0.0,122635.208333,361153.5,41594.686364,5.253834e+05,0.0,57111.291667,57111.291667,19010.571970,623.700758
2024-06,107.0,311111.93,0.0,159899.583333,387195.0,75829.310606,6.229239e+05,0.0,76430.625000,76430.625000,46006.765152,0.000000
2024-07,56.0,189645.90,0.0,94829.083333,204446.0,41669.833333,3.409449e+05,0.0,33274.458333,33274.458333,16516.106061,0.000000
汇总,418.0,1183748.28,0.0,613180.750000,1578215.0,262095.485606,2.453491e+06,0.0,293760.666667,293760.666667,168604.086616,623.700758


#### 代自营

In [95]:
df_dzy = df_cw_merge_t[(df_cw_merge_t['资方'] == '代自营')]
df_dzy_g = df_dzy.groupby(['search_time']).agg({'是否逾期': 'sum', '剩余总应收租金': 'sum', '租送残值': 'sum', '租归残值1': 'sum', '租归残值2': 'sum', '续租残值3': 'sum', '总残值': 'sum', '租送逾期残值':'sum', '租归逾期残值': 'sum', '首租逾期残值':'sum', '续租逾期残值': 'sum', '总预收残值': 'sum'}
                                            ).rename(columns={'search_time': '下单年月', '是否逾期': '逾期订单数', '剩余总应收租金': '应收未收租金'})
# df_dzy_g.loc[:, '订单平均逾期金额'] = df_dzy_g['应收未收租金'] / df_dzy_g['逾期订单数']
df_dzy_g.loc['汇总', :] = df_dzy_g.select_dtypes(include=['float64', 'int64']).sum()

df_dzy_g

,逾期订单数,应收未收租金,租送残值,租归残值1,租归残值2,续租残值3,总残值,租送逾期残值,租归逾期残值,首租逾期残值,续租逾期残值,总预收残值
search_time,,,,,,,,,,,,
2022-12,20.0,70942.20,1.670050e+04,3.399129e+04,7.798600e+04,1.541336e+04,1.440912e+05,3.932917e+03,1.688146e+04,2.081438e+04,4.228045e+03,0.000000e+00
2023-01,6.0,21518.30,7.087333e+03,5.841750e+03,1.617100e+04,4.264500e+03,3.336458e+04,2.120667e+03,3.240542e+03,5.361208e+03,7.107500e+02,0.000000e+00
2023-02,53.0,113697.75,0.000000e+00,5.772629e+04,1.917970e+05,1.548417e+04,2.650075e+05,0.000000e+00,4.070654e+04,4.070654e+04,1.095842e+04,0.000000e+00
2023-03,80.0,195847.02,1.519717e+04,9.026529e+04,2.499210e+05,2.076199e+04,3.761454e+05,5.968833e+03,4.781067e+04,5.377950e+04,2.985835e+04,0.000000e+00
2023-04,20.0,82408.27,4.877500e+03,3.871096e+04,8.006200e+04,9.389667e+03,1.330401e+05,0.000000e+00,1.553058e+04,1.553058e+04,4.249667e+03,0.000000e+00
2023-05,29.0,85602.63,1.851942e+04,3.176967e+04,7.711400e+04,1.424596e+04,1.416490e+05,2.292817e+04,1.645683e+04,3.938500e+04,5.974333e+03,0.000000e+00
2023-06,79.0,267197.74,5.410150e+04,1.049415e+05,2.569020e+05,4.593117e+04,4.618762e+05,9.746917e+03,8.361729e+04,9.336421e+04,3.937708e+04,0.000000e+00
2023-08,73.0,231656.19,2.776100e+04,1.128035e+05,2.622125e+05,6.284892e+04,4.656260e+05,1.709942e+04,8.182333e+04,9.892275e+04,4.671408e+04,0.000000e+00
2023-09,14.0,43181.30,5.399333e+03,2.025392e+04,5.216800e+04,1.113914e+04,8.896039e+04,0.000000e+00,1.437696e+04,1.437696e+04,1.160352e+04,0.000000e+00


In [96]:
with pd.ExcelWriter('F:\需求\倩姐需求\资方数据统计_自营逾期1212.xlsx') as writer:
    df_wk_g.to_excel(writer, sheet_name='维客自营账户')
    df_hf_g.to_excel(writer, sheet_name='海南鸿丰商业保理有限公司')
    df_jc_g.to_excel(writer, sheet_name='杭州金锤网络信息服务有限公司')
    df_yr_g.to_excel(writer, sheet_name='成都越蓉金科科技有限公司')
    df_dzy_g.to_excel(writer, sheet_name='代自营')